# 자세영상 CSV와 관절을 추출한다.

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
#print(pose)   # mediapipe.python.solutions.pose.Pose

video_path = 'sample\\<영상파일이름>.mp4'
cap = cv2.VideoCapture(video_path)


#check if the video capture is open
if(cap.isOpened() == False):
    print("Error Opening Video Stream Or File")

w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) #폭
h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) #높이
fps = cap.get(cv2.CAP_PROP_FPS) #프레임

#Colab은 C랑 C++사용
fourcc = cv2.VideoWriter_fourcc(*'DIVX') # 'D'. 'I'. 'V'. 'X' <- 이렇게 사용인데 *은 언팩 펼쳐짐  프레임을 파일에 넣을 수 있음

# 1프레임과 다음 프레임 사이의 간격 설정
delay = round(1000/fps)

# 영상 저장
# cv2.VideoWriter 객체 생성, 속성값 입력
out = cv2.VideoWriter('datamovie\\<영상파일이름>.mp4', fourcc, 20.0, (w, h))    #프레임수, 비디오 크기, 비디오파일 저장을 위한 스트림 생성

# landmark를 csv 파일에 저장하기 위한 자료구조
landmark_list = []  # 한개의 프레임에 포함된 관절좌표,visibility를 한행으로 표현(2차원 배열)

frame_cnt = 0

bone_idx = [11,12,13,14,15,16,23,24,25,25,27,28]

tmp_col = '''LS_x, LS_y, LS_z, RS_x, RS_y, RS_z, LFA_x, LFA_y, LFA_z, RFA_x, RFA_y, RFA_z, LH_x, LH_y, LH_z, RH_x, RH_y, RH_z,
             LUL_x,  LUL_y, LUL_z, RUL_x, RUL_y, RUL_z, LL_x, LL_y, LL_z, RL_x, RL_y, RL_z, LF_x, LF_y, LF_z, RF_x, RF_y, RF_z'''
columns = [ tok.strip() for tok in tmp_col.split(',')]

df = pd.DataFrame( columns=columns)

while(cap.isOpened()):
    ret, frame = cap.read()
    #print('프레임 시작')
    if ret == True:

        #frame = cv2.flip(frame, 1)    # 영상을 좌우 반전할 때(반전 영상을 생성할 때 유용함)

        # cv2.imshow('Frame', frame) #처음만 이거하면 너무 느려서 주석처리 해야함
        frame_cnt += 1

        if cv2.waitKey(25)  == ord('q'):
            break

        #image = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)  # OpenCV는 BGR 형식이지만 MediaPipe는 RGB형식
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)      # results.pose_landmarks 에 랜드마크 정보가 저장됨 관절 정보
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)  # 랜드마크 정보를 사용하여 연결선을 그림

        out.write(image)   # 관절 위치가 적색으로 표시된 이미지를 비디오 파일에 저장 처음만 이것도 주석처리

        if results.pose_landmarks==None: # 없는경우 오류나서 사용
            print('frame',frame_cnt, None)
        else:
            df_row = []

            for idx in bone_idx: # 미디어 파이프가 가지고있는 관절의 번호
                '''
                frame_landmark.append((round(results.pose_landmarks.landmark[idx].x,5),
                                       round(results.pose_landmarks.landmark[idx].y,5),
                                       round(results.pose_landmarks.landmark[idx].z,5),
                                       round(results.pose_landmarks.landmark[idx].visibility,5)))'''
                # 한행에 컬럼을 추가
                df_row.extend([round(results.pose_landmarks.landmark[idx].x,5),  # 소수점 나오는데 5자리까지만해서 반올림
                               round(results.pose_landmarks.landmark[idx].y,5),  # append 원소 하나 추가임 [ [1], [2]] extend 원소 하나 확장임 [ 1, 2 ]
                               round(results.pose_landmarks.landmark[idx].z,5)]) # 한행을 써야하는데 2차원이면 힘듬

            df.loc[len(df.index)] = df_row   # 한 행 추가

        #break  # 첫 프레임만 확인하려면 break 활성화
    else:
        break

display(df)

# csv 파일에 저장
df.to_csv('datacsv\\<영상파일이름>.csv', header=True, index=False)

print('총 프레임 수={}'.format(len(df)))

cap.release()
out.release()
cv2.destroyAllWindows()

# LSTM 학습용 데이터 전처리 및 Training
* 72개의 데이터 csv를 로드한다

In [1]:
import pandas as pd
import numpy as np

In [2]:
sqaut_hand0_0 = pd.read_csv('datacsv\\sqaut_hand0_0.csv')
sqaut_hand0_0

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.53247,0.37793,-0.25261,0.53017,0.38226,0.08762,0.46889,0.38497,-0.36566,0.47351,...,-0.00648,0.54720,0.67025,-0.00648,0.54281,0.78612,0.12587,0.54447,0.77309,0.34984
1,0.53244,0.38154,-0.23441,0.53245,0.38736,0.10851,0.46765,0.38655,-0.32589,0.47512,...,-0.02403,0.54498,0.67089,-0.02403,0.54327,0.78619,0.09496,0.54604,0.77306,0.31309
2,0.53235,0.38889,-0.23689,0.53238,0.39805,0.10462,0.46689,0.38878,-0.33318,0.47348,...,-0.03134,0.54544,0.67206,-0.03134,0.54375,0.78511,0.08632,0.54620,0.77302,0.29948
3,0.53233,0.39059,-0.23409,0.53234,0.39967,0.10093,0.46576,0.38914,-0.32939,0.47264,...,-0.03496,0.54560,0.67233,-0.03496,0.54371,0.78486,0.07783,0.54617,0.77288,0.28747
4,0.52965,0.39657,-0.21984,0.53228,0.40399,0.10364,0.46043,0.39148,-0.30703,0.47048,...,-0.04629,0.54570,0.67433,-0.04629,0.54374,0.78442,0.05586,0.54614,0.77274,0.26394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,0.54861,0.35993,-0.23025,0.54413,0.36470,0.05102,0.54851,0.45503,-0.24432,0.54323,...,-0.00406,0.53877,0.66494,-0.00406,0.54437,0.78242,0.08686,0.54874,0.75796,0.24686
231,0.54887,0.35991,-0.23493,0.54416,0.36462,0.05024,0.54758,0.45510,-0.24983,0.54244,...,-0.00455,0.53877,0.66495,-0.00455,0.54411,0.78250,0.08493,0.54859,0.75800,0.24717
232,0.54889,0.35980,-0.24024,0.54443,0.36451,0.05129,0.54573,0.45542,-0.25666,0.54201,...,-0.00549,0.53808,0.66530,-0.00549,0.54372,0.78266,0.08537,0.54862,0.75797,0.25186
233,0.54894,0.35979,-0.24267,0.54459,0.36448,0.05087,0.54487,0.45569,-0.25935,0.54173,...,-0.00635,0.53747,0.66579,-0.00635,0.54346,0.78278,0.08721,0.54864,0.75788,0.25581


In [3]:
sqaut_hand25_0 = pd.read_csv('datacsv\\sqaut_hand25_0.csv')
sqaut_hand25_0

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.47168,0.37483,-0.27298,0.45301,0.36981,0.01680,0.41037,0.37332,-0.36395,0.40775,...,-0.07516,0.50467,0.61104,-0.07516,0.50345,0.69436,0.05445,0.47064,0.65114,0.34841
1,0.46950,0.37675,-0.26197,0.45322,0.37567,0.04420,0.40999,0.37762,-0.35979,0.40809,...,-0.07929,0.50648,0.61242,-0.07929,0.50333,0.69449,0.03132,0.46889,0.65112,0.30801
2,0.46801,0.38453,-0.28164,0.45323,0.38297,0.01949,0.40837,0.38588,-0.36877,0.40620,...,-0.09434,0.50647,0.61155,-0.09434,0.50453,0.69474,-0.00178,0.46802,0.65198,0.35002
3,0.46738,0.38646,-0.28020,0.45320,0.38578,0.01988,0.40799,0.38861,-0.36682,0.40509,...,-0.15372,0.50646,0.60894,-0.15372,0.50504,0.69475,-0.08711,0.46805,0.65318,0.26948
4,0.46745,0.39346,-0.27639,0.45208,0.39364,0.01661,0.40618,0.39480,-0.35762,0.40016,...,-0.25576,0.50529,0.60492,-0.25576,0.50632,0.69476,-0.19351,0.46809,0.65473,0.26954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,0.48462,0.33251,-0.17530,0.47145,0.32790,0.08385,0.44635,0.41268,-0.11214,0.45229,...,-0.04069,0.49001,0.58808,-0.04069,0.50350,0.69015,0.01052,0.48172,0.66026,0.27443
451,0.48316,0.33246,-0.17789,0.47144,0.32776,0.08333,0.45110,0.40886,-0.11475,0.44985,...,-0.04536,0.49003,0.58807,-0.04536,0.50370,0.68993,0.00661,0.47998,0.65884,0.29354
452,0.48233,0.32975,-0.17865,0.47184,0.32518,0.08662,0.45085,0.40070,-0.11667,0.45006,...,-0.05570,0.49236,0.58649,-0.05570,0.50398,0.68863,-0.00954,0.47857,0.65865,0.29379
453,0.48166,0.32872,-0.17963,0.47212,0.32430,0.08711,0.43996,0.40149,-0.11775,0.45029,...,-0.05685,0.49357,0.58616,-0.05685,0.50411,0.68804,-0.01018,0.47781,0.65808,0.29905


In [4]:
sqaut_hand45_0 = pd.read_csv('datacsv\\sqaut_hand45_0.csv')
sqaut_hand45_0

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.47410,0.34103,-0.27140,0.43603,0.35351,0.08401,0.40412,0.36099,-0.40240,0.37594,...,-0.12705,0.49481,0.67699,-0.12705,0.50108,0.80806,-0.02376,0.45280,0.76133,0.38584
1,0.47413,0.34450,-0.27984,0.43586,0.35523,0.06137,0.40412,0.36076,-0.40573,0.37200,...,-0.16388,0.49502,0.67650,-0.16388,0.50109,0.80793,-0.08285,0.45534,0.76140,0.43888
2,0.47292,0.35104,-0.27117,0.43610,0.35882,0.06639,0.40186,0.36277,-0.38887,0.37041,...,-0.24452,0.49508,0.67672,-0.24452,0.50126,0.80793,-0.14017,0.45994,0.76263,0.38882
3,0.47235,0.35405,-0.27132,0.43616,0.36120,0.06497,0.40082,0.36315,-0.39144,0.36877,...,-0.22923,0.49484,0.67818,-0.22923,0.50130,0.80790,-0.10556,0.46208,0.76512,0.36866
4,0.47178,0.36335,-0.27108,0.43594,0.37074,0.05910,0.39814,0.36685,-0.38447,0.36630,...,-0.21474,0.49490,0.68284,-0.21474,0.50130,0.80741,-0.06284,0.46238,0.76504,0.37661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,0.50127,0.33220,-0.24417,0.44772,0.32146,0.04548,0.50570,0.43519,-0.26322,0.45301,...,-0.08132,0.47837,0.65792,-0.08132,0.49263,0.79610,-0.02790,0.44132,0.75840,0.30814
304,0.50090,0.33244,-0.24131,0.44699,0.32222,0.04832,0.50575,0.43528,-0.26276,0.45300,...,-0.08175,0.47837,0.65936,-0.08175,0.49260,0.79626,-0.02961,0.44139,0.75846,0.31038
305,0.50063,0.33248,-0.24805,0.44687,0.32216,0.04818,0.50584,0.43520,-0.27052,0.45310,...,-0.07840,0.47854,0.65954,-0.07840,0.49285,0.79619,-0.01996,0.44140,0.75699,0.34168
306,0.50035,0.33305,-0.26292,0.44740,0.32495,0.04585,0.50612,0.43597,-0.29199,0.45363,...,-0.07401,0.47865,0.66218,-0.07401,0.49324,0.79669,-0.00893,0.44091,0.75834,0.33470


In [5]:
sqaut_hand60_0 = pd.read_csv('datacsv\\sqaut_hand60_0.csv')
sqaut_hand60_0

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.50930,0.34447,-0.17849,0.43308,0.36041,0.04412,0.49179,0.35480,-0.41553,0.38487,...,-0.07298,0.51492,0.66982,-0.07298,0.52930,0.78326,0.07589,0.43409,0.76106,0.38077
1,0.50769,0.34666,-0.17997,0.43281,0.36738,0.03598,0.46390,0.35656,-0.36422,0.38507,...,-0.12604,0.51615,0.67020,-0.12604,0.52930,0.78437,0.01205,0.43407,0.76234,0.37497
2,0.50450,0.35191,-0.20358,0.43272,0.37202,0.02210,0.44058,0.36385,-0.36709,0.38528,...,-0.20355,0.51651,0.66873,-0.20355,0.52923,0.78428,-0.03882,0.43406,0.76244,0.34589
3,0.50406,0.35398,-0.19919,0.43223,0.37588,0.02718,0.44923,0.36750,-0.36256,0.38531,...,-0.23177,0.51678,0.66812,-0.23177,0.52904,0.78403,-0.07621,0.43408,0.76251,0.35683
4,0.50394,0.36147,-0.18389,0.43202,0.38513,0.03532,0.44672,0.36921,-0.35808,0.38535,...,-0.26527,0.51686,0.66721,-0.26527,0.52885,0.78350,-0.09574,0.43399,0.76270,0.26743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.52978,0.33294,-0.13296,0.44135,0.32975,-0.07730,0.53748,0.42753,-0.09532,0.42850,...,-0.01017,0.51478,0.66066,-0.01017,0.52544,0.78357,0.12875,0.43402,0.76003,0.23222
296,0.53009,0.33387,-0.13330,0.44133,0.33100,-0.07114,0.53696,0.42855,-0.09707,0.43023,...,-0.01810,0.51487,0.66078,-0.01810,0.52538,0.78444,0.12055,0.43403,0.76010,0.26389
297,0.53018,0.33430,-0.13432,0.44132,0.33156,-0.07064,0.53673,0.42872,-0.10090,0.43113,...,-0.02058,0.51482,0.66085,-0.02058,0.52528,0.78510,0.11798,0.43419,0.75980,0.27854
298,0.53024,0.33459,-0.13479,0.44131,0.33187,-0.07480,0.53651,0.42907,-0.09824,0.43143,...,-0.01820,0.51477,0.66089,-0.01820,0.52513,0.78547,0.12089,0.43431,0.75969,0.27227


In [6]:
sqaut_hand90_0 = pd.read_csv('datacsv\\sqaut_hand90_0.csv')
sqaut_hand90_0

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.49087,0.33499,-0.06445,0.41259,0.33056,-0.05103,0.51560,0.38362,-0.19755,0.38939,...,-0.04110,0.49230,0.58667,-0.04110,0.50299,0.67103,0.15808,0.41631,0.68348,0.09136
1,0.49087,0.33466,-0.10853,0.41291,0.33027,-0.13544,0.51555,0.37894,-0.30623,0.38990,...,-0.02072,0.49270,0.58597,-0.02072,0.50329,0.67108,0.17748,0.41653,0.68348,0.09493
2,0.49026,0.33439,-0.15481,0.41365,0.33041,-0.16711,0.50966,0.36390,-0.40821,0.39004,...,0.00469,0.49270,0.58154,0.00469,0.50340,0.67108,0.20685,0.41697,0.68302,0.03698
3,0.48988,0.33420,-0.15552,0.41408,0.33051,-0.16744,0.50777,0.36079,-0.40673,0.39009,...,0.00696,0.49269,0.57978,0.00696,0.50350,0.67108,0.20866,0.41729,0.68269,0.03211
4,0.48946,0.33446,-0.17080,0.41465,0.33184,-0.17800,0.50682,0.35895,-0.43915,0.39715,...,-0.08482,0.49203,0.57303,-0.08482,0.50358,0.67089,0.10389,0.41769,0.68187,0.01056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,0.49187,0.33157,-0.07808,0.41276,0.32795,-0.08838,0.49787,0.40274,-0.02898,0.41101,...,0.03870,0.49148,0.56988,0.03870,0.50161,0.66377,0.17906,0.41693,0.67821,0.11052
290,0.49174,0.33158,-0.07887,0.41281,0.32723,-0.08878,0.49829,0.40320,-0.03460,0.41068,...,0.04904,0.49147,0.56975,0.04904,0.50161,0.66380,0.18266,0.41686,0.67822,0.09522
291,0.49168,0.33149,-0.09559,0.41285,0.32721,-0.10462,0.49914,0.40444,-0.05019,0.41001,...,0.05378,0.49144,0.56997,0.05378,0.50162,0.66381,0.20489,0.41684,0.67831,0.10183
292,0.49167,0.33162,-0.11066,0.41287,0.32753,-0.11264,0.50103,0.40469,-0.06641,0.40681,...,0.05245,0.49145,0.57022,0.05245,0.50162,0.66384,0.19636,0.41685,0.67843,0.11006


In [7]:
sqaut_hand105_0 = pd.read_csv('datacsv\\sqaut_hand105_0.csv')
sqaut_hand105_0

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.57061,0.35923,0.04753,0.49068,0.34916,-0.16430,0.61764,0.37829,-0.05890,0.51434,...,0.12697,0.56113,0.65912,0.12697,0.56500,0.76243,0.33212,0.47298,0.78797,-0.00176
1,0.57060,0.36134,0.04818,0.49119,0.35009,-0.15027,0.62177,0.37788,-0.05018,0.51655,...,0.04403,0.56416,0.65549,0.04403,0.56417,0.76202,0.18056,0.47295,0.78884,0.08630
2,0.57074,0.36770,0.03208,0.49120,0.35618,-0.15021,0.62245,0.37822,-0.08643,0.52064,...,0.00844,0.56727,0.65477,0.00844,0.56416,0.76209,0.15192,0.47295,0.78880,0.05481
3,0.57080,0.36991,0.03089,0.49125,0.35850,-0.15223,0.62236,0.37862,-0.08628,0.52251,...,-0.01008,0.56924,0.65462,-0.01008,0.56423,0.76206,0.13949,0.47297,0.78823,0.00691
4,0.57162,0.37655,0.02078,0.49125,0.36500,-0.15133,0.62284,0.37930,-0.11920,0.52401,...,-0.15311,0.57046,0.65458,-0.15311,0.56477,0.76192,0.01153,0.47298,0.78541,-0.11969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.56018,0.33175,-0.02830,0.47488,0.33021,-0.11981,0.57066,0.42580,0.03710,0.46441,...,0.10896,0.55428,0.65062,0.10896,0.56541,0.76404,0.28751,0.47683,0.78157,0.09235
296,0.56020,0.33316,-0.03396,0.47440,0.33100,-0.13014,0.57020,0.42603,0.03130,0.46494,...,0.12494,0.55443,0.64943,0.12494,0.56524,0.76368,0.31176,0.47696,0.78168,0.09227
297,0.56015,0.33326,-0.04854,0.47428,0.33134,-0.14022,0.56984,0.42620,0.02625,0.46530,...,0.12333,0.55452,0.64784,0.12333,0.56516,0.76364,0.29920,0.47727,0.78164,0.08234
298,0.56016,0.33317,-0.04781,0.47425,0.33128,-0.13937,0.56978,0.42628,0.02519,0.46555,...,0.12568,0.55462,0.64698,0.12568,0.56511,0.76303,0.29977,0.47726,0.78174,0.07198


In [8]:
sqaut_hand135_0 = pd.read_csv('datacsv\\sqaut_hand135_0.csv')
sqaut_hand135_0

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.57122,0.35223,0.06053,0.52231,0.34176,-0.18820,0.63078,0.35855,0.10330,0.58991,...,0.01709,0.55916,0.71091,0.01709,0.53948,0.74959,0.31225,0.49859,0.80558,0.07970
1,0.57118,0.35504,0.05716,0.52277,0.34491,-0.20218,0.63301,0.35899,0.10238,0.59105,...,0.08974,0.56035,0.66577,0.08974,0.53936,0.75311,0.28320,0.49856,0.80481,0.12335
2,0.57080,0.36174,0.05338,0.52484,0.35052,-0.22487,0.63424,0.36147,0.11559,0.59430,...,0.08832,0.56170,0.67047,0.08832,0.53562,0.75377,0.28899,0.49848,0.80475,0.15073
3,0.56928,0.36248,0.05189,0.52510,0.35469,-0.22668,0.63427,0.36104,0.11789,0.59528,...,0.08804,0.56173,0.66987,0.08804,0.53543,0.75470,0.29359,0.49848,0.80467,0.14923
4,0.56856,0.36838,0.05001,0.52622,0.36307,-0.23625,0.63349,0.36380,0.12663,0.59902,...,0.10271,0.56185,0.66719,0.10271,0.53591,0.75617,0.30638,0.49828,0.80462,0.16888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,0.55254,0.31762,0.05499,0.50598,0.32189,-0.25189,0.54185,0.41716,0.08140,0.49605,...,0.22453,0.55513,0.64002,0.22453,0.55883,0.75795,0.38788,0.50264,0.80299,-0.02011
304,0.55264,0.31708,0.05290,0.50643,0.32152,-0.25120,0.54179,0.41729,0.08026,0.49603,...,0.22285,0.55517,0.64002,0.22285,0.55875,0.75792,0.37930,0.50325,0.80289,-0.02385
305,0.55291,0.31649,0.04826,0.50649,0.32136,-0.25314,0.54180,0.41726,0.07973,0.49560,...,0.22266,0.55527,0.64003,0.22266,0.55873,0.75778,0.37919,0.50297,0.80308,-0.02288
306,0.55293,0.31823,0.06007,0.50592,0.32296,-0.23977,0.54299,0.41784,0.09312,0.49519,...,0.20378,0.55557,0.64222,0.20378,0.55867,0.76100,0.35249,0.50272,0.80398,-0.02098


In [9]:
sqaut_hand155_0 = pd.read_csv('datacsv\\sqaut_hand155_0.csv')
sqaut_hand155_0

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.54330,0.36512,0.04931,0.53294,0.37755,-0.26638,0.56181,0.36107,0.24553,0.59309,...,0.20864,0.53848,0.57390,0.20864,0.53435,0.64444,0.41966,0.49341,0.68756,0.19903
1,0.54397,0.37094,0.01764,0.53165,0.37857,-0.27941,0.56802,0.36813,0.17810,0.59258,...,0.16795,0.53859,0.56629,0.16795,0.53421,0.64996,0.37613,0.49336,0.69156,0.07961
2,0.54487,0.37694,-0.00797,0.52987,0.38565,-0.30332,0.55957,0.38908,0.19064,0.59164,...,0.18359,0.53878,0.56596,0.18359,0.53401,0.65371,0.41909,0.49283,0.69189,-0.01502
3,0.54556,0.37785,-0.02195,0.52886,0.38852,-0.31626,0.55842,0.38366,0.17813,0.59023,...,0.16455,0.53889,0.56459,0.16455,0.53372,0.65463,0.39053,0.49215,0.69213,0.00698
4,0.54810,0.38489,-0.01643,0.52870,0.39365,-0.25510,0.59175,0.38984,0.09492,0.59027,...,0.00187,0.54014,0.56521,0.00187,0.53326,0.65643,0.18813,0.49136,0.69087,-0.19392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,0.53374,0.32545,0.09007,0.51514,0.33072,-0.16958,0.55016,0.38865,0.02963,0.53459,...,0.20843,0.53778,0.56788,0.20843,0.52903,0.65445,0.32784,0.49266,0.68590,-0.01098
451,0.53373,0.32564,0.09614,0.51642,0.33059,-0.16694,0.55253,0.38621,0.03535,0.54010,...,0.20374,0.53773,0.56762,0.20374,0.52933,0.65384,0.32006,0.49283,0.68599,-0.00855
452,0.53190,0.32534,0.10067,0.51954,0.32938,-0.16661,0.55253,0.38406,0.04121,0.54962,...,0.20309,0.53729,0.56668,0.20309,0.52930,0.65298,0.31948,0.49278,0.68601,-0.01630
453,0.53181,0.32488,0.09989,0.52096,0.32903,-0.16611,0.55287,0.38330,0.03861,0.54769,...,0.20295,0.53695,0.56648,0.20295,0.52928,0.65251,0.31959,0.49256,0.68606,-0.01406


In [10]:
sqaut_hand180_0 = pd.read_csv('datacsv\\sqaut_hand180_0.csv')
sqaut_hand180_0

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.47584,0.38125,0.10461,0.46958,0.38192,-0.22159,0.53454,0.38629,0.18653,0.53784,...,0.17109,0.45254,0.66297,0.17109,0.45104,0.77419,0.27364,0.45424,0.78784,0.06319
1,0.47348,0.38255,0.10025,0.46958,0.38448,-0.21798,0.52967,0.38590,0.17346,0.53403,...,0.18205,0.45254,0.66296,0.18205,0.45174,0.77369,0.29629,0.45430,0.78713,0.09070
2,0.47340,0.38739,0.09108,0.47021,0.38985,-0.22278,0.52957,0.38680,0.15031,0.53415,...,0.18346,0.45241,0.66838,0.18346,0.45175,0.77299,0.29747,0.45435,0.78636,0.09023
3,0.47373,0.39014,0.08886,0.47034,0.39094,-0.22600,0.52992,0.38712,0.15256,0.53423,...,0.19252,0.45282,0.66802,0.19252,0.45176,0.77267,0.31401,0.45429,0.78635,0.11026
4,0.47380,0.39501,0.08711,0.47173,0.39660,-0.22571,0.53160,0.38940,0.14262,0.53608,...,0.19178,0.45461,0.66839,0.19178,0.45195,0.77255,0.31103,0.45418,0.78634,0.10736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,0.45435,0.36214,0.09793,0.45375,0.36191,-0.22342,0.45196,0.45544,0.10595,0.45049,...,0.18734,0.45879,0.65352,0.18734,0.45042,0.76111,0.30643,0.45471,0.77996,0.09990
231,0.45378,0.36212,0.09799,0.45378,0.36200,-0.22308,0.45223,0.45555,0.10618,0.45216,...,0.18462,0.45864,0.65329,0.18462,0.44977,0.76079,0.30100,0.45488,0.77993,0.09682
232,0.45331,0.36216,0.09846,0.45381,0.36236,-0.22726,0.45233,0.45610,0.10869,0.45370,...,0.18514,0.45828,0.65352,0.18514,0.44950,0.76080,0.30357,0.45500,0.77981,0.09671
233,0.45306,0.36220,0.10000,0.45382,0.36262,-0.22791,0.45243,0.45644,0.11183,0.45505,...,0.18518,0.45809,0.65363,0.18518,0.44934,0.76081,0.30359,0.45503,0.77977,0.09480


In [11]:
sqaut_0_0 = pd.read_csv('datacsv\\sqaut_0_0.csv')
sqaut_0_0

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.43990,0.36163,-0.18514,0.44356,0.35883,0.08377,0.38182,0.38254,-0.14733,0.39673,...,-0.02525,0.46015,0.60368,-0.02525,0.46743,0.71228,0.05270,0.47113,0.69062,0.29616
1,0.43984,0.36479,-0.17854,0.44295,0.36461,0.08254,0.38235,0.38492,-0.14783,0.39405,...,-0.05261,0.45894,0.60252,-0.05261,0.46686,0.71211,-0.00004,0.47158,0.69226,0.26324
2,0.43984,0.37545,-0.17888,0.44308,0.37089,0.08447,0.38234,0.38998,-0.15903,0.39479,...,-0.06733,0.45823,0.60254,-0.06733,0.46685,0.71211,-0.02918,0.47193,0.69419,0.23552
3,0.43987,0.38340,-0.17686,0.44308,0.37698,0.08450,0.38072,0.39820,-0.15684,0.39537,...,-0.07739,0.45621,0.60226,-0.07739,0.46681,0.71181,-0.05442,0.47317,0.69411,0.25934
4,0.43987,0.39031,-0.16195,0.44225,0.38398,0.09534,0.38123,0.40467,-0.14575,0.39581,...,-0.12448,0.45459,0.60130,-0.12448,0.46726,0.71045,-0.13443,0.47381,0.69254,0.23128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,0.44946,0.32710,-0.22866,0.45053,0.32531,0.03519,0.46550,0.41173,-0.24509,0.45889,...,-0.01623,0.45837,0.59488,-0.01623,0.46657,0.70902,0.06334,0.46686,0.68862,0.22734
321,0.44929,0.32715,-0.23369,0.45053,0.32522,0.03138,0.46574,0.41178,-0.25245,0.45876,...,-0.01742,0.45790,0.59557,-0.01742,0.46662,0.70926,0.06234,0.46730,0.68880,0.23178
322,0.44928,0.32681,-0.23106,0.45053,0.32495,0.03355,0.46563,0.41190,-0.25003,0.45865,...,-0.01975,0.45788,0.59562,-0.01975,0.46702,0.70927,0.05875,0.46767,0.68883,0.22914
323,0.44900,0.32688,-0.23307,0.45066,0.32481,0.03381,0.46536,0.41205,-0.25105,0.45863,...,-0.01997,0.45789,0.59600,-0.01997,0.46721,0.71003,0.05906,0.46787,0.69007,0.23184


In [12]:
sqaut_25_0 = pd.read_csv('datacsv\\sqaut_25_0.csv')
sqaut_25_0

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.48449,0.33991,-0.20386,0.48155,0.34521,0.10912,0.44641,0.39149,-0.20381,0.45616,...,-0.08631,0.49881,0.66642,-0.08631,0.49931,0.80085,-0.04300,0.50187,0.78402,0.25501
1,0.48351,0.34473,-0.20449,0.48183,0.34961,0.11577,0.44050,0.39160,-0.18875,0.45554,...,-0.08508,0.49879,0.66834,-0.08508,0.49951,0.80145,-0.04260,0.50228,0.78470,0.27183
2,0.48175,0.35181,-0.22939,0.48196,0.35146,0.10334,0.44175,0.39419,-0.23545,0.45610,...,-0.07102,0.50032,0.67198,-0.07102,0.50078,0.80140,-0.00245,0.50194,0.78475,0.26126
3,0.48096,0.35300,-0.22233,0.48204,0.35249,0.10391,0.44258,0.39539,-0.25713,0.45854,...,-0.07692,0.50082,0.67269,-0.07692,0.50087,0.79975,-0.00053,0.50075,0.78474,0.21130
4,0.47939,0.35881,-0.22146,0.48200,0.35987,0.12947,0.44003,0.40022,-0.23111,0.45173,...,-0.10344,0.50127,0.67831,-0.10344,0.50036,0.79920,0.01506,0.49962,0.78585,0.22077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,0.50212,0.31105,-0.18847,0.49959,0.31438,0.15908,0.49461,0.41366,-0.20732,0.48545,...,-0.08655,0.49259,0.65998,-0.08655,0.49775,0.80339,-0.05516,0.50594,0.77790,0.21570
449,0.50159,0.30896,-0.18030,0.49911,0.31427,0.16204,0.49837,0.40873,-0.27861,0.48633,...,-0.08451,0.49287,0.65955,-0.08451,0.49824,0.80340,-0.04664,0.50558,0.77802,0.16839
450,0.50127,0.30782,-0.18365,0.49925,0.31336,0.16181,0.49960,0.40653,-0.30007,0.48719,...,-0.07810,0.49464,0.65987,-0.07810,0.49854,0.80347,-0.04287,0.50537,0.77825,0.18306
451,0.50094,0.30818,-0.18425,0.49903,0.31375,0.16364,0.49851,0.40773,-0.25553,0.49312,...,-0.06692,0.49756,0.66010,-0.06692,0.49904,0.80362,-0.01146,0.50384,0.77973,0.20066


In [13]:
sqaut_45_0 = pd.read_csv('datacsv\\sqaut_45_0.csv')
sqaut_45_0

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.50278,0.33061,-0.20662,0.48051,0.33222,0.09142,0.43921,0.37550,-0.16385,0.44026,...,-0.06092,0.50238,0.66334,-0.06092,0.51824,0.79167,0.03072,0.48390,0.74264,0.30057
1,0.50219,0.33202,-0.20904,0.48333,0.33270,0.08975,0.43425,0.37413,-0.15494,0.43161,...,-0.05813,0.50239,0.66577,-0.05813,0.51773,0.79166,0.01264,0.48519,0.74349,0.32872
2,0.50098,0.33431,-0.21305,0.48335,0.33411,0.08579,0.42711,0.37352,-0.15521,0.42918,...,-0.05066,0.50387,0.66939,-0.05066,0.51755,0.79188,0.03387,0.48563,0.74536,0.35632
3,0.50073,0.33552,-0.21434,0.48329,0.33575,0.08439,0.42667,0.37265,-0.15455,0.42809,...,-0.05061,0.50511,0.67142,-0.05061,0.51720,0.79282,0.04081,0.48584,0.74688,0.35179
4,0.49965,0.33683,-0.21369,0.48346,0.33832,0.08523,0.42346,0.37082,-0.15266,0.42938,...,-0.06260,0.50828,0.67111,-0.06260,0.51718,0.79286,-0.01439,0.48443,0.74799,0.34947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,0.52727,0.33565,-0.22666,0.47197,0.33113,0.05954,0.53022,0.43929,-0.24955,0.46778,...,-0.07039,0.50492,0.66080,-0.07039,0.51919,0.79129,0.00150,0.47872,0.75257,0.33794
342,0.52693,0.33558,-0.23224,0.47054,0.33112,0.05926,0.52842,0.43882,-0.26082,0.46744,...,-0.08372,0.50396,0.66018,-0.08372,0.51871,0.79063,-0.02823,0.47904,0.75155,0.34103
343,0.52670,0.33546,-0.22923,0.46977,0.33111,0.06098,0.52825,0.43830,-0.25728,0.46712,...,-0.08106,0.50353,0.65965,-0.08106,0.51816,0.78956,-0.02389,0.47918,0.75005,0.33596
344,0.52710,0.33626,-0.21929,0.46976,0.33167,0.06296,0.53005,0.43782,-0.24949,0.46674,...,-0.07658,0.50363,0.66296,-0.07658,0.51888,0.78960,-0.01214,0.48093,0.75025,0.28955


In [14]:
sqaut_60_0 = pd.read_csv('datacsv\\sqaut_60_0.csv')
sqaut_60_0

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.51656,0.32791,-0.08956,0.44090,0.33346,0.00446,0.46737,0.38269,-0.16776,0.41140,...,-0.02841,0.51193,0.66531,-0.02841,0.52495,0.78506,0.11654,0.43108,0.76221,0.31593
1,0.51620,0.32810,-0.08957,0.44083,0.33514,0.00873,0.47009,0.38373,-0.16303,0.41127,...,-0.00531,0.51237,0.66257,-0.00531,0.52539,0.78497,0.11112,0.43170,0.76190,0.29050
2,0.51483,0.32927,-0.09697,0.44060,0.34048,0.01154,0.46040,0.38254,-0.14695,0.41124,...,-0.00716,0.51337,0.66232,-0.00716,0.52598,0.78492,0.11022,0.43187,0.76158,0.29847
3,0.51410,0.33024,-0.09892,0.44043,0.34245,0.01192,0.45755,0.38164,-0.15039,0.41091,...,-0.00982,0.51375,0.66237,-0.00982,0.52625,0.78520,0.10513,0.43188,0.76146,0.31569
4,0.51332,0.33404,-0.08669,0.44043,0.34635,0.01194,0.45883,0.38546,-0.10143,0.41069,...,-0.03260,0.51514,0.66238,-0.03260,0.52641,0.78539,0.07407,0.43187,0.76088,0.30394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,0.52849,0.33027,-0.14684,0.43888,0.32876,-0.05744,0.54018,0.43103,-0.10809,0.43842,...,-0.00922,0.51084,0.66265,-0.00922,0.52564,0.78697,0.11213,0.43538,0.76134,0.25995
404,0.52853,0.33030,-0.15019,0.43887,0.32872,-0.05725,0.53992,0.43094,-0.11139,0.43818,...,-0.00896,0.51082,0.66264,-0.00896,0.52564,0.78698,0.11377,0.43544,0.76109,0.25612
405,0.52856,0.33017,-0.15326,0.43887,0.32833,-0.05856,0.53987,0.43047,-0.11382,0.43804,...,-0.01893,0.51076,0.66229,-0.01893,0.52594,0.78717,0.09605,0.43544,0.75969,0.26050
406,0.52860,0.33076,-0.15403,0.43888,0.32851,-0.05893,0.53986,0.43096,-0.11424,0.43734,...,-0.01832,0.51063,0.66258,-0.01832,0.52585,0.78838,0.09716,0.43533,0.76016,0.24683


In [15]:
sqaut_90_0 = pd.read_csv('datacsv\\sqaut_90_0.csv')
sqaut_90_0

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.50870,0.34448,-0.07621,0.43327,0.34367,-0.05462,0.50750,0.37535,-0.23193,0.43039,...,-0.12058,0.51457,0.58579,-0.12058,0.52811,0.67933,0.08148,0.43182,0.68721,-0.05384
1,0.50870,0.34562,-0.07767,0.43296,0.34611,-0.04761,0.50713,0.37781,-0.23751,0.43040,...,-0.07370,0.51836,0.60715,-0.07370,0.52925,0.67926,0.22453,0.43184,0.68729,-0.09725
2,0.50832,0.35003,-0.07042,0.43243,0.35061,-0.04228,0.50704,0.38142,-0.22647,0.43067,...,-0.08798,0.51876,0.60261,-0.08798,0.52973,0.67932,0.20434,0.43136,0.68777,-0.05301
3,0.50791,0.35764,-0.07741,0.43233,0.35799,-0.04338,0.50473,0.38546,-0.24108,0.43007,...,-0.09509,0.51919,0.60007,-0.09509,0.52972,0.67920,0.19841,0.43031,0.68838,-0.05735
4,0.50724,0.36484,-0.07962,0.43251,0.36380,-0.05639,0.50483,0.39302,-0.24861,0.42911,...,-0.21456,0.52000,0.59489,-0.21456,0.52977,0.67967,0.02845,0.42986,0.68907,-0.05798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,0.51587,0.33055,-0.09722,0.43343,0.32596,-0.11766,0.52271,0.41487,-0.05168,0.42610,...,0.07874,0.51618,0.58763,0.07874,0.53171,0.68220,0.25769,0.42817,0.68795,0.15743
287,0.51609,0.33067,-0.09920,0.43355,0.32592,-0.11746,0.52276,0.41510,-0.05302,0.42611,...,0.07863,0.51624,0.58764,0.07863,0.53183,0.68243,0.25823,0.42817,0.68799,0.16120
288,0.51634,0.33029,-0.10506,0.43363,0.32537,-0.11768,0.52276,0.41455,-0.05375,0.42611,...,0.06982,0.51625,0.58585,0.06982,0.53190,0.68204,0.24572,0.42888,0.68799,0.17189
289,0.51638,0.32747,-0.10542,0.43369,0.32263,-0.11742,0.52265,0.41092,-0.05266,0.42617,...,0.06007,0.51622,0.57958,0.06007,0.53227,0.67855,0.22501,0.42939,0.68645,0.16680


In [16]:
sqaut_105_0 = pd.read_csv('datacsv\\sqaut_105_0.csv')
sqaut_105_0

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.55921,0.33499,0.03540,0.48923,0.33347,-0.11283,0.58833,0.37356,-0.13551,0.52670,...,0.10351,0.55898,0.64846,0.10351,0.56725,0.75780,0.26700,0.47119,0.78939,0.09310
1,0.55931,0.33556,0.02340,0.48854,0.33392,-0.10879,0.58782,0.37724,-0.13549,0.51881,...,0.08979,0.55897,0.64866,0.08979,0.56745,0.75940,0.25712,0.47173,0.78644,0.14126
2,0.55907,0.33945,0.02492,0.48851,0.33600,-0.10984,0.58846,0.38225,-0.12741,0.51980,...,0.07919,0.55884,0.64867,0.07919,0.56752,0.76028,0.24409,0.47192,0.78543,0.13741
3,0.55846,0.34109,0.02500,0.48853,0.33685,-0.11233,0.58860,0.38384,-0.12834,0.52040,...,0.07611,0.55872,0.64862,0.07611,0.56753,0.76024,0.24839,0.47210,0.78455,0.14009
4,0.55694,0.34465,0.02800,0.48898,0.34030,-0.10988,0.58946,0.38773,-0.12597,0.52327,...,0.06744,0.55851,0.64869,0.06744,0.56752,0.76021,0.22878,0.47210,0.78268,0.13102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,0.56106,0.33240,0.00670,0.47659,0.32894,-0.12215,0.55583,0.42732,0.06791,0.46425,...,0.13309,0.55608,0.64841,0.13309,0.56421,0.76165,0.29943,0.47673,0.78706,0.09831
404,0.56100,0.33238,0.00499,0.47653,0.32895,-0.12122,0.55715,0.42725,0.06689,0.46439,...,0.13145,0.55609,0.64828,0.13145,0.56423,0.76171,0.29672,0.47675,0.78736,0.09337
405,0.56105,0.33078,0.00179,0.47640,0.32881,-0.12113,0.55966,0.42503,0.06675,0.46449,...,0.13278,0.55711,0.64769,0.13278,0.56439,0.75976,0.32003,0.47693,0.78707,0.09703
406,0.56105,0.33080,0.00187,0.47637,0.32923,-0.11729,0.56166,0.42480,0.06739,0.46453,...,0.13082,0.55716,0.64747,0.13082,0.56435,0.76000,0.30642,0.47694,0.78708,0.06684


In [17]:
sqaut_135_0 = pd.read_csv('datacsv\\sqaut_135_0.csv')
sqaut_135_0

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.51182,0.33188,0.12366,0.50993,0.33574,-0.20148,0.55025,0.35915,0.19552,0.57292,...,0.18115,0.51591,0.64435,0.18115,0.50556,0.74830,0.28076,0.48021,0.79546,0.03613
1,0.51555,0.33126,0.12438,0.50513,0.33435,-0.19117,0.55758,0.37283,0.05723,0.57076,...,0.19331,0.52342,0.64453,0.19331,0.51498,0.74816,0.30360,0.47990,0.79521,0.03620
2,0.52068,0.33613,0.12152,0.50308,0.33488,-0.19280,0.56276,0.37294,0.08778,0.57055,...,0.19629,0.52804,0.64522,0.19629,0.51902,0.74821,0.31119,0.47979,0.79526,0.04664
3,0.52359,0.33808,0.12103,0.50176,0.33488,-0.19148,0.56429,0.37498,0.05911,0.56697,...,0.19371,0.52939,0.64527,0.19371,0.52015,0.74821,0.30375,0.47942,0.79526,0.03954
4,0.52281,0.34148,0.11495,0.50233,0.33813,-0.20721,0.56410,0.37503,0.13745,0.56664,...,0.20555,0.53104,0.64704,0.20555,0.52108,0.74909,0.32850,0.47867,0.79590,0.06267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,0.53183,0.33095,0.05278,0.47709,0.32835,-0.24579,0.53701,0.42575,0.10613,0.47041,...,0.19705,0.52581,0.65113,0.19705,0.51490,0.75481,0.33651,0.48022,0.79882,0.10649
342,0.53413,0.33006,0.04472,0.47716,0.32813,-0.24196,0.53748,0.42575,0.10262,0.47312,...,0.18158,0.52982,0.65068,0.18158,0.51813,0.75299,0.31370,0.48039,0.79627,0.09286
343,0.53507,0.32909,0.04698,0.47720,0.32796,-0.23756,0.53781,0.42536,0.10209,0.47372,...,0.16527,0.52994,0.65055,0.16527,0.51812,0.75262,0.28794,0.48103,0.79466,0.08855
344,0.53491,0.32906,0.05799,0.47627,0.32866,-0.22934,0.53804,0.42550,0.11175,0.47298,...,0.16761,0.53035,0.65078,0.16761,0.51797,0.75366,0.28850,0.48105,0.79476,0.08053


In [18]:
sqaut_155_0 = pd.read_csv('datacsv\\sqaut_155_0.csv')
sqaut_155_0

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.52307,0.35333,0.12872,0.51865,0.34491,-0.20355,0.56487,0.38451,0.20305,0.56480,...,0.16330,0.50255,0.66388,0.16330,0.49990,0.78697,0.24744,0.50283,0.80373,0.08192
1,0.52299,0.35637,0.13162,0.51868,0.34943,-0.20366,0.56632,0.39128,0.21857,0.56720,...,0.16694,0.50281,0.66481,0.16694,0.50119,0.78777,0.25638,0.50302,0.80414,0.09783
2,0.52514,0.36185,0.13083,0.51912,0.35555,-0.20645,0.56906,0.39416,0.22879,0.56819,...,0.18167,0.50316,0.66593,0.18167,0.50250,0.78732,0.28666,0.50310,0.80390,0.12828
3,0.52651,0.36254,0.12747,0.51945,0.35766,-0.20999,0.56916,0.39400,0.22376,0.56865,...,0.17861,0.50308,0.66698,0.17861,0.50314,0.78746,0.27629,0.50373,0.80440,0.12054
4,0.52680,0.36515,0.13295,0.52066,0.36278,-0.21370,0.56898,0.40411,0.24185,0.57459,...,0.19551,0.50307,0.67187,0.19551,0.50312,0.78907,0.30577,0.50332,0.80545,0.14653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,0.50900,0.32024,0.18468,0.49999,0.31405,-0.17761,0.51914,0.37213,0.24370,0.50234,...,0.13877,0.49930,0.65412,0.13877,0.49911,0.78057,0.18143,0.50441,0.80309,-0.03575
449,0.50918,0.31991,0.17919,0.49955,0.31145,-0.18284,0.51594,0.37534,0.23681,0.49696,...,0.15408,0.49935,0.65375,0.15408,0.49885,0.77854,0.21054,0.50384,0.80232,-0.00815
450,0.50610,0.32027,0.17764,0.49976,0.31005,-0.18287,0.51047,0.37938,0.26176,0.49826,...,0.16248,0.49926,0.65349,0.16248,0.49871,0.77804,0.23329,0.50389,0.80246,0.01620
451,0.50504,0.32047,0.17203,0.49984,0.31044,-0.19751,0.50981,0.38028,0.27626,0.49842,...,0.18368,0.49948,0.65453,0.18368,0.49858,0.77808,0.28422,0.50386,0.80248,0.06426


In [19]:
sqaut_180_0 = pd.read_csv('datacsv\\sqaut_180_0.csv')
sqaut_180_0

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.55786,0.36234,0.09553,0.55792,0.36437,-0.19202,0.59028,0.35872,0.17458,0.61627,...,0.20454,0.53919,0.59963,0.20454,0.53697,0.67845,0.31929,0.53200,0.71144,0.06626
1,0.55783,0.36363,0.08848,0.55806,0.36981,-0.19085,0.59025,0.35556,0.17310,0.61635,...,0.19582,0.53778,0.60163,0.19582,0.53389,0.68047,0.30883,0.53193,0.71235,0.14368
2,0.55794,0.37057,0.08980,0.55909,0.37965,-0.18118,0.59343,0.36167,0.17424,0.61948,...,0.18865,0.53692,0.60315,0.18865,0.53174,0.68346,0.29389,0.53161,0.71235,0.12999
3,0.55845,0.37162,0.09834,0.55930,0.38079,-0.17379,0.59322,0.37045,0.17476,0.61955,...,0.18279,0.53714,0.60273,0.18279,0.53106,0.68381,0.27925,0.53121,0.71222,0.06010
4,0.55822,0.37776,0.10838,0.55961,0.38692,-0.17393,0.59195,0.37683,0.18847,0.61978,...,0.18453,0.53777,0.60112,0.18453,0.52982,0.68530,0.27936,0.53119,0.71140,0.06425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,0.54583,0.31966,0.08410,0.55221,0.32889,-0.20198,0.52838,0.40687,0.08614,0.53416,...,0.15211,0.54576,0.58889,0.15211,0.54003,0.69190,0.24943,0.52939,0.70549,0.07591
402,0.54538,0.31867,0.08381,0.55226,0.32879,-0.20608,0.52823,0.40678,0.08674,0.53424,...,0.15889,0.54532,0.58888,0.15889,0.53936,0.69157,0.26252,0.52952,0.70518,0.08344
403,0.54519,0.31802,0.08354,0.55226,0.32863,-0.20759,0.52837,0.40676,0.08702,0.53429,...,0.15937,0.54508,0.58871,0.15937,0.53909,0.69141,0.26283,0.52962,0.70492,0.08316
404,0.54524,0.31932,0.08401,0.55234,0.32943,-0.20699,0.53101,0.40666,0.08739,0.53433,...,0.15924,0.54506,0.58879,0.15924,0.53917,0.69141,0.26186,0.52981,0.70516,0.08255


In [20]:
sqaut_bad_hand0_1 = pd.read_csv('datacsv\\sqaut_bad_hand0_1.csv')
sqaut_bad_hand0_1

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.53600,0.38191,-0.20642,0.53708,0.38651,0.10502,0.47956,0.41379,-0.27309,0.49222,...,0.02670,0.51583,0.67875,0.02670,0.54182,0.78351,0.15265,0.55225,0.74393,0.36757
1,0.53615,0.38201,-0.20622,0.53695,0.38633,0.10837,0.47922,0.41329,-0.27253,0.49315,...,0.01024,0.51647,0.67850,0.01024,0.54194,0.78403,0.13418,0.55318,0.75452,0.34542
2,0.53616,0.38389,-0.20506,0.53645,0.38738,0.11261,0.47653,0.41048,-0.27125,0.49318,...,0.00097,0.51639,0.68029,0.00097,0.54205,0.78410,0.13841,0.55378,0.75512,0.35099
3,0.53616,0.38475,-0.20479,0.53619,0.38799,0.11326,0.47506,0.40946,-0.27130,0.49319,...,-0.00245,0.51633,0.68074,-0.00245,0.54207,0.78420,0.13575,0.55405,0.75472,0.35052
4,0.53634,0.38569,-0.20499,0.53518,0.39048,0.11329,0.47526,0.40797,-0.29233,0.48978,...,-0.02316,0.51481,0.68102,-0.02316,0.54207,0.78421,0.11460,0.55379,0.74623,0.34017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,0.54020,0.36313,-0.22195,0.53304,0.36842,0.06970,0.53567,0.46194,-0.24193,0.53488,...,-0.02613,0.53454,0.66659,-0.02613,0.54324,0.78341,0.05904,0.54692,0.75971,0.22868
293,0.54030,0.36242,-0.22237,0.53302,0.36802,0.06963,0.53537,0.46113,-0.24184,0.53474,...,-0.02631,0.53437,0.66596,-0.02631,0.54316,0.78340,0.06027,0.54669,0.75994,0.23076
294,0.54029,0.36244,-0.21659,0.53217,0.36857,0.07794,0.53404,0.45959,-0.23693,0.53154,...,-0.03110,0.53409,0.66574,-0.03110,0.54269,0.78342,0.05037,0.54676,0.76069,0.22915
295,0.54027,0.36242,-0.22066,0.53191,0.36884,0.07574,0.53369,0.45969,-0.23962,0.53153,...,-0.02983,0.53407,0.66664,-0.02983,0.54267,0.78372,0.05513,0.54681,0.76167,0.23432


In [21]:
sqaut_bad_hand25_1 = pd.read_csv('datacsv\\sqaut_bad_hand25_1.csv')
sqaut_bad_hand25_1

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.50283,0.32654,-0.25566,0.47083,0.31880,0.03910,0.46061,0.38460,-0.30422,0.42941,...,-0.09671,0.50757,0.58754,-0.09671,0.52020,0.68103,0.01921,0.48058,0.64389,0.37523
1,0.50153,0.32731,-0.26292,0.47119,0.31929,0.03864,0.45818,0.38417,-0.32871,0.42894,...,-0.07300,0.51003,0.58982,-0.07300,0.52028,0.68293,0.05295,0.48344,0.64617,0.37923
2,0.49977,0.32868,-0.26204,0.47188,0.32036,0.03864,0.45344,0.38018,-0.33923,0.42894,...,-0.06716,0.51114,0.59002,-0.06716,0.52037,0.68359,0.05615,0.48514,0.64860,0.34604
3,0.49859,0.32985,-0.26182,0.47209,0.32141,0.04264,0.45157,0.37910,-0.33771,0.42871,...,-0.06425,0.51182,0.59010,-0.06425,0.52037,0.68387,0.05829,0.48574,0.64915,0.34144
4,0.49730,0.33368,-0.26985,0.47359,0.32395,0.04135,0.44777,0.37535,-0.34396,0.42869,...,-0.04885,0.51161,0.59192,-0.04885,0.52020,0.68466,0.09371,0.48592,0.64915,0.40899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,0.47098,0.46439,-0.32834,0.45636,0.44889,-0.08171,0.40872,0.47942,-0.42104,0.40406,...,-0.21771,0.46455,0.63946,-0.21771,0.51730,0.68033,-0.02112,0.48049,0.64828,0.24903
280,0.47114,0.46399,-0.32062,0.45625,0.45231,-0.06744,0.41266,0.47240,-0.41454,0.40383,...,-0.22115,0.46366,0.64073,-0.22115,0.51731,0.68019,-0.02693,0.48110,0.64857,0.24750
281,0.47123,0.46479,-0.32704,0.45534,0.45008,-0.07413,0.41265,0.47051,-0.42048,0.40275,...,-0.21586,0.46293,0.64137,-0.21586,0.51733,0.68019,-0.02253,0.48310,0.64868,0.25200
282,0.47110,0.46382,-0.32114,0.45573,0.44859,-0.06392,0.41258,0.46938,-0.41445,0.40282,...,-0.21477,0.46297,0.64213,-0.21477,0.51753,0.68020,-0.01696,0.48672,0.64867,0.25650


In [22]:
sqaut_bad_hand45_1 = pd.read_csv('datacsv\\sqaut_bad_hand45_1.csv')
sqaut_bad_hand45_1

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.49380,0.32318,-0.24868,0.46307,0.33099,0.09431,0.42272,0.30835,-0.38302,0.39716,...,-0.08015,0.49922,0.67093,-0.08015,0.52407,0.79588,0.06954,0.46655,0.76053,0.38485
1,0.50380,0.32606,-0.22903,0.45607,0.33400,0.10102,0.44246,0.31589,-0.39179,0.39613,...,-0.12970,0.49958,0.67956,-0.12970,0.52425,0.79501,0.04997,0.46869,0.76069,0.24802
2,0.50488,0.33273,-0.22918,0.45522,0.34275,0.10309,0.44710,0.31768,-0.40063,0.39500,...,-0.13134,0.49965,0.68671,-0.13134,0.52439,0.79513,0.05266,0.46990,0.76264,0.25907
3,0.50546,0.33636,-0.23117,0.45534,0.34391,0.10342,0.44376,0.32436,-0.39797,0.39502,...,-0.12602,0.49965,0.68774,-0.12602,0.52497,0.79539,0.04755,0.47247,0.76261,0.26100
4,0.50436,0.34114,-0.23291,0.45543,0.35223,0.10278,0.44273,0.32904,-0.39732,0.39288,...,-0.12592,0.49970,0.68831,-0.12592,0.52526,0.79480,0.04761,0.46814,0.76258,0.26457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,0.53126,0.31417,-0.18584,0.46498,0.32115,0.08763,0.53902,0.42152,-0.22650,0.47269,...,-0.08140,0.51825,0.67039,-0.08140,0.53427,0.79886,-0.01361,0.47575,0.76239,0.29682
307,0.53125,0.31418,-0.18638,0.46551,0.32165,0.08699,0.53913,0.42151,-0.22452,0.47369,...,-0.08080,0.51815,0.67038,-0.08080,0.53388,0.79886,-0.00861,0.47576,0.76239,0.29406
308,0.53124,0.31420,-0.18507,0.46562,0.32204,0.08709,0.53921,0.42149,-0.22242,0.47375,...,-0.08046,0.51804,0.67032,-0.08046,0.53362,0.79886,-0.00476,0.47575,0.76239,0.29270
309,0.53123,0.31417,-0.18154,0.46557,0.32204,0.08826,0.53933,0.42148,-0.21523,0.47352,...,-0.08049,0.51794,0.67039,-0.08049,0.53354,0.79898,-0.00452,0.47568,0.76210,0.29531


In [23]:
sqaut_bad_hand60_1 = pd.read_csv('datacsv\\sqaut_bad_hand60_1.csv')
sqaut_bad_hand60_1

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.49486,0.33853,-0.19779,0.42043,0.34094,0.07012,0.44989,0.33702,-0.39562,0.36792,...,-0.05476,0.48718,0.68741,-0.05476,0.50817,0.79703,0.15863,0.41503,0.78156,0.31614
1,0.49349,0.33854,-0.19325,0.42016,0.34787,0.07007,0.45202,0.32242,-0.39248,0.36749,...,-0.04455,0.48743,0.68734,-0.04455,0.50850,0.79691,0.20064,0.41560,0.77989,0.27901
2,0.49263,0.34257,-0.19278,0.42020,0.35516,0.06594,0.45231,0.32276,-0.38609,0.36681,...,-0.03346,0.48742,0.68915,-0.03346,0.50850,0.79659,0.22062,0.41560,0.77980,0.28378
3,0.49180,0.34419,-0.19201,0.42033,0.35768,0.06657,0.45444,0.32480,-0.38707,0.36672,...,-0.03432,0.48737,0.68912,-0.03432,0.50856,0.79545,0.20333,0.41572,0.77973,0.28383
4,0.49025,0.35073,-0.18739,0.42031,0.36467,0.06715,0.44393,0.33170,-0.36173,0.36964,...,-0.10168,0.48674,0.69008,-0.10168,0.50801,0.79028,0.16172,0.41577,0.77967,0.27083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,0.50670,0.32102,-0.13215,0.41685,0.32513,-0.01662,0.51768,0.42787,-0.11597,0.40922,...,-0.02821,0.49486,0.67004,-0.02821,0.51046,0.79557,0.11126,0.41461,0.77932,0.25229
319,0.50669,0.32102,-0.13564,0.41689,0.32508,-0.01491,0.51739,0.42762,-0.11823,0.40883,...,-0.02810,0.49467,0.67057,-0.02810,0.51050,0.79638,0.10671,0.41462,0.77931,0.25223
320,0.50676,0.32106,-0.14039,0.41703,0.32496,-0.01663,0.51718,0.42743,-0.12019,0.40865,...,-0.02798,0.49461,0.67056,-0.02798,0.51050,0.79639,0.09908,0.41475,0.77922,0.25217
321,0.50697,0.32182,-0.13999,0.41707,0.32513,-0.02137,0.51700,0.42843,-0.11454,0.40850,...,-0.02571,0.49460,0.67081,-0.02571,0.51061,0.79678,0.09969,0.41499,0.77924,0.25194


In [24]:
sqaut_bad_hand90_1 = pd.read_csv('datacsv\\sqaut_bad_hand90_1.csv')
sqaut_bad_hand90_1

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.54183,0.32745,-0.09736,0.45993,0.32474,-0.08688,0.56313,0.37985,-0.25066,0.43747,...,-0.24762,0.53774,0.56878,-0.24762,0.54678,0.65327,-0.03120,0.46424,0.66075,-0.06580
1,0.54181,0.32801,-0.09296,0.45989,0.32476,-0.08244,0.56325,0.37983,-0.23942,0.43741,...,-0.15203,0.53775,0.57059,-0.15203,0.54699,0.65345,0.10970,0.46400,0.66179,-0.02545
2,0.54143,0.32897,-0.08692,0.45989,0.32543,-0.07802,0.56318,0.37852,-0.22837,0.43703,...,-0.14485,0.53776,0.57097,-0.14485,0.54713,0.65335,0.10700,0.46358,0.66329,0.00357
3,0.54123,0.32961,-0.08536,0.45989,0.32585,-0.07639,0.56314,0.37805,-0.22591,0.43688,...,-0.14520,0.53777,0.57124,-0.14520,0.54722,0.65294,0.11006,0.46333,0.66393,0.00643
4,0.54046,0.33238,-0.08654,0.45981,0.32891,-0.07670,0.56278,0.37777,-0.23713,0.43674,...,-0.23720,0.53792,0.57047,-0.23720,0.54716,0.65302,0.00837,0.46315,0.66461,-0.01505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,0.54228,0.31867,-0.08688,0.46302,0.31280,-0.11691,0.54846,0.39900,-0.03975,0.45663,...,0.08981,0.53724,0.56254,0.08981,0.54851,0.65447,0.26880,0.46030,0.66113,0.17444
343,0.54238,0.31876,-0.08758,0.46308,0.31278,-0.11768,0.54843,0.39922,-0.04038,0.45663,...,0.08983,0.53715,0.56322,0.08983,0.54839,0.65499,0.27047,0.46029,0.66098,0.17647
344,0.54250,0.31825,-0.08592,0.46320,0.31195,-0.11467,0.54848,0.39894,-0.03950,0.45665,...,0.07940,0.53712,0.56318,0.07940,0.54842,0.65497,0.25831,0.46030,0.66013,0.17893
345,0.54253,0.31832,-0.08567,0.46326,0.31195,-0.11451,0.54849,0.39917,-0.03940,0.45666,...,0.08074,0.53712,0.56330,0.08074,0.54836,0.65496,0.26638,0.46029,0.66014,0.17926


In [25]:
sqaut_bad_hand105_1 = pd.read_csv('datacsv\\sqaut_bad_hand105_1.csv')
sqaut_bad_hand105_1

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.57912,0.34391,0.07040,0.50597,0.33593,-0.17673,0.63259,0.33716,-0.01344,0.53929,...,0.05329,0.59554,0.66411,0.05329,0.58634,0.77920,0.21944,0.49442,0.79366,0.09712
1,0.57913,0.34556,0.07301,0.50601,0.33695,-0.17404,0.63261,0.33891,0.06382,0.53833,...,0.04122,0.59914,0.68622,0.04122,0.58589,0.77767,0.32477,0.49307,0.79979,0.10957
2,0.57987,0.35604,0.07135,0.50606,0.34531,-0.17157,0.63327,0.34081,0.05394,0.53736,...,0.01649,0.60502,0.69518,0.01649,0.58474,0.77681,0.32315,0.49319,0.79953,0.17754
3,0.58045,0.35701,0.06445,0.50606,0.34659,-0.17438,0.63348,0.34566,0.05468,0.53576,...,0.01922,0.60618,0.68872,0.01922,0.58403,0.77676,0.28461,0.49321,0.79797,0.19925
4,0.58150,0.36676,0.06188,0.50610,0.35640,-0.17880,0.63470,0.35026,0.05664,0.53576,...,-0.02165,0.60907,0.68820,-0.02165,0.58385,0.77676,0.24905,0.49325,0.79805,0.22221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,0.58638,0.32806,0.00390,0.49528,0.31800,-0.13251,0.59684,0.42621,0.06755,0.48606,...,0.13644,0.57095,0.65327,0.13644,0.58314,0.77440,0.32961,0.49115,0.79180,0.10929
319,0.58631,0.32803,0.00076,0.49528,0.31807,-0.13474,0.59716,0.42637,0.06750,0.48604,...,0.13490,0.57106,0.65340,0.13490,0.58314,0.77497,0.32212,0.49155,0.79306,0.11035
320,0.58596,0.32804,-0.00904,0.49508,0.31824,-0.14477,0.59733,0.42695,0.06179,0.48604,...,0.13792,0.57100,0.65339,0.13792,0.58301,0.77490,0.32600,0.49143,0.79319,0.11071
321,0.58595,0.32859,-0.00691,0.49500,0.31938,-0.14087,0.59744,0.42802,0.06159,0.48617,...,0.13595,0.57082,0.65362,0.13595,0.58308,0.77529,0.31669,0.49064,0.79397,0.08713


In [26]:
sqaut_bad_hand135_1 = pd.read_csv('datacsv\\sqaut_bad_hand135_1.csv')
sqaut_bad_hand135_1

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.54817,0.33867,0.08550,0.49428,0.32270,-0.22506,0.60772,0.34316,0.12258,0.55762,...,0.20645,0.55626,0.65101,0.20645,0.53809,0.76552,0.40564,0.47355,0.79629,0.04758
1,0.54782,0.33905,0.08318,0.49283,0.32563,-0.22009,0.60874,0.33927,0.13970,0.54826,...,0.13012,0.55775,0.65668,0.13012,0.53736,0.76632,0.32866,0.47477,0.79762,0.08181
2,0.54703,0.34266,0.07467,0.49225,0.33414,-0.21987,0.60854,0.33969,0.13784,0.54805,...,0.12211,0.55981,0.65833,0.12211,0.53670,0.76637,0.29915,0.47526,0.79727,0.15196
3,0.54614,0.34329,0.07151,0.49213,0.33478,-0.22015,0.60851,0.33968,0.13502,0.54859,...,0.13062,0.56070,0.65791,0.13062,0.53646,0.76644,0.30213,0.47621,0.79631,0.16375
4,0.54781,0.35133,0.07261,0.49190,0.34276,-0.22029,0.61283,0.34302,0.13899,0.54923,...,0.11833,0.56275,0.65980,0.11833,0.53602,0.76656,0.29941,0.47603,0.79626,0.20796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,0.53531,0.31702,0.07371,0.47227,0.31144,-0.22141,0.52915,0.42135,0.10870,0.46151,...,0.20289,0.53063,0.64278,0.20289,0.52621,0.75636,0.36941,0.46807,0.79714,0.04776
307,0.53445,0.31693,0.07512,0.47231,0.31175,-0.22321,0.52799,0.42131,0.10953,0.46148,...,0.20080,0.53066,0.64280,0.20080,0.52640,0.75637,0.36137,0.46808,0.79772,0.05552
308,0.53399,0.31676,0.07557,0.47235,0.31196,-0.22381,0.52744,0.42122,0.10998,0.46148,...,0.19883,0.53069,0.64278,0.19883,0.52640,0.75636,0.35089,0.46817,0.79803,0.05996
309,0.53408,0.31655,0.07591,0.47259,0.31194,-0.22462,0.52771,0.42118,0.11153,0.46144,...,0.19917,0.53115,0.64254,0.19917,0.52600,0.75643,0.35617,0.46812,0.79818,0.04308


In [27]:
sqaut_bad_hand155_1 = pd.read_csv('datacsv\\sqaut_bad_hand155_1.csv')
sqaut_bad_hand155_1

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.53086,0.31978,0.01476,0.49753,0.32385,-0.24671,0.56966,0.36093,0.07202,0.53595,...,0.17011,0.52645,0.56332,0.17011,0.51716,0.64494,0.33833,0.47581,0.68423,0.04427
1,0.52898,0.31765,0.03244,0.49854,0.32433,-0.24785,0.56914,0.35620,0.11533,0.53776,...,0.20913,0.52672,0.56384,0.20913,0.51695,0.64747,0.38268,0.47536,0.68437,0.06703
2,0.52883,0.31903,0.04674,0.49884,0.32505,-0.23393,0.57079,0.35513,0.12336,0.54249,...,0.20824,0.52712,0.56406,0.20824,0.51632,0.64890,0.37236,0.47480,0.68445,0.06346
3,0.52878,0.32000,0.04925,0.49903,0.32562,-0.23085,0.57175,0.35468,0.12400,0.54433,...,0.20655,0.52725,0.56446,0.20655,0.51573,0.65005,0.36725,0.47443,0.68454,0.06691
4,0.52797,0.32502,0.04105,0.49952,0.32996,-0.25353,0.57297,0.35446,0.12109,0.54883,...,0.23742,0.52721,0.56772,0.23742,0.51540,0.65105,0.42044,0.47379,0.68512,0.12074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,0.54404,0.45045,-0.04729,0.52498,0.45781,-0.29519,0.60405,0.45363,-0.02189,0.59002,...,0.01942,0.57398,0.59606,0.01942,0.52769,0.64271,0.18863,0.47809,0.67880,0.00322
280,0.54325,0.45003,-0.04901,0.52481,0.45944,-0.30019,0.60346,0.45336,-0.02199,0.58997,...,0.01823,0.57423,0.59667,0.01823,0.52528,0.64150,0.18089,0.47811,0.67842,0.01458
281,0.54256,0.44843,-0.05146,0.52517,0.45926,-0.30340,0.60328,0.45298,-0.02281,0.58991,...,0.01904,0.57389,0.59646,0.01904,0.52343,0.64005,0.20248,0.47811,0.67850,0.02177
282,0.54265,0.44700,-0.04688,0.52558,0.45930,-0.30148,0.60429,0.45343,-0.01694,0.59081,...,0.02612,0.57417,0.59642,0.02612,0.52361,0.64093,0.19232,0.47826,0.67855,0.00150


In [28]:
sqaut_bad_hand180_1 = pd.read_csv('datacsv\\sqaut_bad_hand180_1.csv')
sqaut_bad_hand180_1

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.46405,0.37883,0.11201,0.46591,0.37873,-0.22090,0.50722,0.43321,0.15849,0.52947,...,0.21634,0.47803,0.65821,0.21634,0.45467,0.75316,0.35677,0.45674,0.78406,0.11042
1,0.46480,0.37914,0.10762,0.46526,0.38012,-0.22906,0.50906,0.43051,0.16495,0.52913,...,0.22997,0.47806,0.65829,0.22997,0.45422,0.75280,0.38093,0.45673,0.78429,0.13710
2,0.46479,0.38153,0.10538,0.46508,0.38238,-0.22856,0.51266,0.42727,0.16665,0.52982,...,0.22699,0.47873,0.65903,0.22699,0.45428,0.75588,0.37359,0.45678,0.78431,0.13468
3,0.46484,0.38260,0.10490,0.46491,0.38336,-0.22775,0.51420,0.42609,0.16617,0.53033,...,0.22489,0.47908,0.65995,0.22489,0.45431,0.75475,0.37091,0.45690,0.78419,0.13314
4,0.46566,0.38550,0.10449,0.46539,0.38679,-0.22431,0.51677,0.42237,0.16429,0.53440,...,0.21531,0.48241,0.66268,0.21531,0.45489,0.76033,0.35372,0.45697,0.78404,0.13003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,0.46799,0.36856,0.08780,0.46030,0.36974,-0.23856,0.46200,0.46123,0.10400,0.45921,...,0.18319,0.46827,0.66394,0.18319,0.45238,0.76465,0.30255,0.45396,0.77997,0.10056
293,0.46848,0.36861,0.08535,0.45996,0.36975,-0.24068,0.46268,0.46127,0.10289,0.45968,...,0.18168,0.46931,0.66317,0.18168,0.45293,0.76477,0.30101,0.45389,0.77975,0.10015
294,0.46847,0.36909,0.08484,0.46018,0.36962,-0.23641,0.46368,0.46140,0.10337,0.46285,...,0.16462,0.46914,0.66037,0.16462,0.45342,0.76491,0.26528,0.45397,0.77978,0.06147
295,0.46829,0.36907,0.08180,0.46026,0.36932,-0.23809,0.46353,0.46147,0.10228,0.46243,...,0.17008,0.46684,0.65995,0.17008,0.45338,0.76519,0.27897,0.45405,0.78001,0.06635


In [29]:
sqaut_bad_0_1 = pd.read_csv('datacsv\\sqaut_bad_0_1.csv')
sqaut_bad_0_1

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.46470,0.36128,-0.17180,0.46821,0.37298,0.13001,0.42418,0.37618,-0.16386,0.45263,...,-0.07755,0.43546,0.68326,-0.07755,0.47622,0.81543,-0.04392,0.47666,0.75776,0.34321
1,0.46470,0.36014,-0.18433,0.46474,0.37103,0.13001,0.42950,0.38601,-0.24380,0.44787,...,-0.09411,0.43543,0.68644,-0.09411,0.47514,0.80942,-0.04067,0.47596,0.75642,0.31608
2,0.46451,0.36147,-0.20195,0.46327,0.37231,0.10388,0.43663,0.38979,-0.26011,0.44659,...,-0.09448,0.43476,0.69118,-0.09448,0.47431,0.80671,-0.01186,0.47489,0.75543,0.33582
3,0.46398,0.36198,-0.19364,0.46278,0.37232,0.10935,0.43568,0.39457,-0.24697,0.44597,...,-0.10451,0.43378,0.69242,-0.10451,0.47426,0.80568,-0.02195,0.47938,0.76240,0.33589
4,0.46307,0.36300,-0.20794,0.46252,0.37241,0.10684,0.43628,0.39847,-0.28565,0.44391,...,-0.10378,0.43255,0.69742,-0.10378,0.47391,0.80483,-0.01850,0.48348,0.76209,0.35345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,0.46870,0.32107,-0.18408,0.46792,0.33326,0.13427,0.43500,0.38703,-0.26418,0.45512,...,-0.07165,0.46237,0.66116,-0.07165,0.46797,0.79973,-0.01783,0.46724,0.77494,0.20023
342,0.46864,0.32107,-0.18485,0.46797,0.33184,0.13427,0.43017,0.38763,-0.15120,0.45394,...,-0.05701,0.46256,0.66085,-0.05701,0.46797,0.80068,0.02422,0.46724,0.77343,0.23642
343,0.46860,0.32107,-0.18504,0.46800,0.33105,0.13411,0.42869,0.38789,-0.14842,0.45304,...,-0.05342,0.46269,0.66054,-0.05342,0.46797,0.79966,0.02969,0.46720,0.77247,0.24294
344,0.46935,0.32139,-0.18131,0.46694,0.33185,0.13836,0.42926,0.38983,-0.15370,0.44897,...,-0.06624,0.46300,0.66055,-0.06624,0.46874,0.79974,-0.00458,0.46723,0.77249,0.21244


In [30]:
sqaut_bad_25_1 = pd.read_csv('datacsv\\sqaut_bad_25_1.csv')
sqaut_bad_25_1

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.47437,0.36557,-0.21190,0.46736,0.36363,0.13295,0.42758,0.41194,-0.20922,0.44554,...,-0.09609,0.44035,0.70831,-0.09609,0.49020,0.80556,0.04756,0.47772,0.77975,0.25964
1,0.47566,0.36829,-0.20248,0.46521,0.36961,0.13703,0.43541,0.40919,-0.22920,0.43717,...,-0.13218,0.44012,0.70978,-0.13218,0.49025,0.80250,0.04281,0.47729,0.78667,0.26961
2,0.47564,0.37430,-0.18938,0.46207,0.37549,0.14621,0.43440,0.41007,-0.17177,0.42148,...,-0.15338,0.43990,0.71165,-0.15338,0.49025,0.79914,0.01392,0.47770,0.79031,0.24481
3,0.47562,0.37575,-0.18594,0.46057,0.37762,0.14759,0.43464,0.40998,-0.16453,0.42115,...,-0.15670,0.43986,0.71251,-0.15670,0.49012,0.79593,0.00993,0.47781,0.79070,0.23239
4,0.47550,0.38025,-0.16920,0.45676,0.38133,0.15364,0.43611,0.40279,-0.15287,0.41423,...,-0.18181,0.43991,0.71509,-0.18181,0.48992,0.79205,0.00800,0.47661,0.79046,0.20668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,0.48019,0.32821,-0.20308,0.47451,0.32595,0.13444,0.43550,0.40184,-0.16291,0.43974,...,-0.06003,0.46350,0.66583,-0.06003,0.48441,0.79781,-0.01616,0.47730,0.75671,0.32375
323,0.48018,0.32822,-0.20315,0.47450,0.32612,0.13453,0.43524,0.40189,-0.16263,0.43977,...,-0.06013,0.46345,0.66577,-0.06013,0.48436,0.79771,-0.01214,0.47731,0.75673,0.32395
324,0.48000,0.32832,-0.20425,0.47436,0.32628,0.12783,0.43484,0.40181,-0.16134,0.43978,...,-0.06047,0.46704,0.66632,-0.06047,0.48453,0.79771,-0.01742,0.47731,0.75644,0.32416
325,0.47996,0.32852,-0.20747,0.47425,0.32644,0.12303,0.43439,0.40183,-0.16087,0.43983,...,-0.06090,0.46859,0.66701,-0.06090,0.48463,0.79772,-0.01855,0.47714,0.75570,0.34171


In [31]:
sqaut_bad_45_1 = pd.read_csv('datacsv\\sqaut_bad_45_1.csv')
sqaut_bad_45_1

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.48937,0.33414,-0.10574,0.43311,0.33589,0.10272,0.44921,0.40206,-0.08089,0.40929,...,-0.07111,0.47338,0.66452,-0.07111,0.49492,0.78911,0.00894,0.43307,0.76279,0.21414
1,0.48953,0.33306,-0.12440,0.43250,0.33563,0.10012,0.45011,0.40562,-0.07663,0.40788,...,-0.06783,0.47394,0.66595,-0.06783,0.49706,0.79254,0.01446,0.43223,0.76125,0.29506
2,0.48946,0.33167,-0.12900,0.43242,0.33522,0.10009,0.44915,0.40546,-0.07577,0.40661,...,-0.06726,0.47387,0.66596,-0.06726,0.49761,0.79333,0.01421,0.43140,0.76065,0.28204
3,0.48945,0.33108,-0.12980,0.43236,0.33490,0.10010,0.44845,0.40537,-0.07566,0.40585,...,-0.06650,0.47383,0.66597,-0.06650,0.49784,0.79370,0.01393,0.43091,0.76040,0.28147
4,0.48909,0.33028,-0.12879,0.43228,0.33479,0.10046,0.44735,0.40410,-0.07610,0.40515,...,-0.06416,0.47382,0.66596,-0.06416,0.49783,0.79359,0.02081,0.43056,0.76066,0.27063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,0.49793,0.32689,-0.09683,0.43285,0.32777,0.08111,0.45540,0.38098,-0.07690,0.40695,...,-0.07700,0.48354,0.65624,-0.07700,0.50226,0.79064,-0.02826,0.43443,0.75605,0.25068
327,0.49779,0.32702,-0.08863,0.43282,0.32718,0.08215,0.45597,0.38036,-0.06240,0.40695,...,-0.07661,0.48223,0.65436,-0.07661,0.50218,0.79031,-0.02311,0.43445,0.75575,0.24681
328,0.49765,0.32710,-0.08660,0.43279,0.32667,0.08254,0.45745,0.38072,-0.05628,0.40696,...,-0.08370,0.48177,0.65394,-0.08370,0.50223,0.79030,-0.03740,0.43450,0.75552,0.24967
329,0.49732,0.32716,-0.08801,0.43277,0.32662,0.08196,0.45645,0.38080,-0.05781,0.40696,...,-0.08165,0.48177,0.65399,-0.08165,0.50258,0.79034,-0.03190,0.43478,0.75544,0.25040


In [32]:
sqaut_bad_60_1 = pd.read_csv('datacsv\\sqaut_bad_60_1.csv')
sqaut_bad_60_1

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.50524,0.32539,-0.09097,0.43310,0.31901,0.05912,0.48247,0.41009,-0.07526,0.40917,...,-0.03633,0.49124,0.66604,-0.03633,0.51091,0.79386,0.06508,0.42780,0.76303,0.24091
1,0.50534,0.32538,-0.09230,0.43289,0.32077,0.07533,0.48078,0.40450,-0.09017,0.40899,...,-0.03881,0.49127,0.66883,-0.03881,0.51309,0.79609,0.06493,0.42782,0.76311,0.28096
2,0.50540,0.32536,-0.09189,0.43242,0.32199,0.07951,0.48048,0.40266,-0.09030,0.40889,...,-0.03877,0.49131,0.67066,-0.03877,0.51406,0.79692,0.07062,0.42782,0.76353,0.27450
3,0.50545,0.32532,-0.09006,0.43198,0.32272,0.08126,0.48032,0.40141,-0.08967,0.40881,...,-0.03931,0.49131,0.67165,-0.03931,0.51444,0.79718,0.06958,0.42784,0.76390,0.27292
4,0.50549,0.32536,-0.08828,0.43168,0.32326,0.08176,0.47932,0.40095,-0.08867,0.40857,...,-0.03948,0.49123,0.67214,-0.03948,0.51441,0.79714,0.06846,0.42785,0.76476,0.26760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,0.50202,0.32328,-0.08729,0.42903,0.32081,0.06856,0.47712,0.39830,-0.09151,0.40528,...,-0.05112,0.49271,0.66112,-0.05112,0.51454,0.79753,0.04260,0.42581,0.76098,0.28549
334,0.50165,0.32217,-0.08495,0.42881,0.31953,0.06748,0.47482,0.39158,-0.09984,0.40508,...,-0.05323,0.49256,0.66051,-0.05323,0.51444,0.79758,0.03427,0.42575,0.76099,0.28713
335,0.50168,0.32114,-0.07981,0.42874,0.31883,0.06894,0.47461,0.38707,-0.10588,0.40504,...,-0.05562,0.49256,0.65955,-0.05562,0.51447,0.79744,0.02979,0.42573,0.76099,0.28357
336,0.50180,0.32098,-0.08920,0.42925,0.31874,0.06883,0.47458,0.38751,-0.10140,0.40531,...,-0.05181,0.49257,0.65961,-0.05181,0.51449,0.79742,0.04359,0.42568,0.76149,0.28360


In [33]:
sqaut_bad_90_1 = pd.read_csv('datacsv\\sqaut_bad_90_1.csv')
sqaut_bad_90_1

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.58119,0.33245,-0.03325,0.49332,0.33177,-0.00753,0.58277,0.41620,-0.12242,0.48628,...,0.05532,0.58768,0.67544,0.05532,0.59941,0.78354,0.26059,0.48503,0.79463,0.19434
1,0.58125,0.33316,-0.05236,0.49307,0.33202,-0.03848,0.58188,0.41742,-0.16946,0.48628,...,0.04818,0.58849,0.67704,0.04818,0.59938,0.78398,0.27133,0.48362,0.79447,0.19937
2,0.58143,0.33576,-0.04984,0.49216,0.33361,-0.03594,0.58051,0.42239,-0.16386,0.48628,...,0.02505,0.59002,0.68332,0.02505,0.59937,0.78501,0.27301,0.48358,0.79408,0.21248
3,0.58150,0.33757,-0.04844,0.49171,0.33459,-0.03806,0.58006,0.42379,-0.16303,0.48628,...,0.02542,0.59091,0.68611,0.02542,0.59938,0.78565,0.27207,0.48357,0.79364,0.21148
4,0.58163,0.34268,-0.05124,0.49121,0.33936,-0.03926,0.57953,0.42824,-0.16606,0.48642,...,0.02742,0.59238,0.68781,0.02742,0.59940,0.78590,0.28637,0.48359,0.79318,0.21985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,0.58490,0.33322,-0.09797,0.49046,0.32844,-0.11618,0.60150,0.43195,-0.02528,0.47557,...,0.02460,0.57945,0.66794,0.02460,0.59775,0.78549,0.20197,0.48907,0.78584,0.18848
342,0.58503,0.33265,-0.10111,0.49048,0.32770,-0.11524,0.60128,0.43072,-0.02984,0.47690,...,0.04409,0.57942,0.66414,0.04409,0.59776,0.78302,0.20027,0.49031,0.78196,0.18577
343,0.58525,0.33244,-0.10349,0.49049,0.32721,-0.11261,0.60119,0.43031,-0.03148,0.47759,...,0.04479,0.57920,0.66281,0.04479,0.59761,0.78252,0.19678,0.49109,0.78029,0.18699
344,0.58526,0.33248,-0.10342,0.49042,0.32722,-0.10916,0.60077,0.43034,-0.03310,0.47804,...,0.04078,0.57896,0.66309,0.04078,0.59780,0.78261,0.19764,0.49106,0.78131,0.19194


In [34]:
sqaut_bad_105_1 = pd.read_csv('datacsv\\sqaut_bad_105_1.csv')
sqaut_bad_105_1

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.57256,0.32584,0.04389,0.49271,0.32736,-0.08826,0.59561,0.40988,-0.06909,0.51360,...,0.13590,0.57147,0.65476,0.13590,0.57213,0.76472,0.33463,0.49155,0.79011,0.17874
1,0.57212,0.32589,0.05810,0.49271,0.32727,-0.07987,0.59560,0.41069,-0.05750,0.51017,...,0.13105,0.57106,0.65058,0.13105,0.57214,0.76368,0.33469,0.49172,0.79057,0.13897
2,0.57186,0.32593,0.06541,0.49273,0.32718,-0.07831,0.59568,0.41091,-0.05016,0.50924,...,0.13125,0.57087,0.64906,0.13125,0.57218,0.76326,0.33610,0.49172,0.79119,0.12909
3,0.57168,0.32595,0.06556,0.49275,0.32709,-0.07893,0.59577,0.41103,-0.04991,0.50890,...,0.13158,0.57080,0.64836,0.13158,0.57222,0.76299,0.34132,0.49172,0.79170,0.13080
4,0.57168,0.32605,0.06914,0.49280,0.32696,-0.07666,0.59596,0.41125,-0.04550,0.50880,...,0.13179,0.57077,0.64804,0.13179,0.57243,0.76289,0.34156,0.49170,0.79229,0.12940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,0.57393,0.32548,0.06137,0.49647,0.32415,-0.09265,0.60057,0.40466,-0.06326,0.51895,...,0.16529,0.56811,0.64751,0.16529,0.57537,0.75974,0.35832,0.48840,0.79224,0.12309
334,0.57383,0.32512,0.06386,0.49677,0.32341,-0.09830,0.60057,0.40446,-0.05956,0.51982,...,0.16316,0.56813,0.64713,0.16316,0.57538,0.75908,0.35346,0.48868,0.79156,0.12781
335,0.57378,0.32467,0.06355,0.49689,0.32293,-0.10357,0.60052,0.40371,-0.06064,0.52107,...,0.16405,0.56814,0.64643,0.16405,0.57535,0.75786,0.35331,0.48886,0.79155,0.12651
336,0.57353,0.32467,0.06243,0.49645,0.32292,-0.10268,0.60053,0.40383,-0.06494,0.52004,...,0.16305,0.56778,0.64789,0.16305,0.57491,0.75814,0.35679,0.48883,0.79251,0.11115


In [35]:
sqaut_bad_135_1 = pd.read_csv('datacsv\\sqaut_bad_135_1.csv')
sqaut_bad_135_1

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.56968,0.33789,0.08011,0.51026,0.32695,-0.13644,0.60037,0.41344,-0.04329,0.53880,...,0.17349,0.57431,0.65140,0.17349,0.56732,0.75608,0.35495,0.50179,0.79109,0.08288
1,0.56968,0.33858,0.07206,0.51030,0.32841,-0.16270,0.60028,0.41332,-0.05941,0.53951,...,0.20556,0.57334,0.65227,0.20556,0.56686,0.75689,0.38254,0.50160,0.79136,0.08412
2,0.56975,0.33896,0.07304,0.51052,0.32930,-0.16742,0.60054,0.41324,-0.05752,0.54111,...,0.21458,0.57320,0.65386,0.21458,0.56674,0.75731,0.39816,0.50149,0.79138,0.10027
3,0.56976,0.33896,0.07818,0.51092,0.32971,-0.16558,0.60071,0.41319,-0.05588,0.54169,...,0.21709,0.57316,0.65509,0.21709,0.56662,0.75770,0.40125,0.50149,0.79134,0.10026
4,0.57029,0.33875,0.07906,0.51128,0.32977,-0.16713,0.60179,0.41312,-0.05580,0.54166,...,0.21958,0.57316,0.65518,0.21958,0.56684,0.75769,0.40528,0.50163,0.79133,0.08448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,0.56970,0.33554,0.05984,0.50291,0.32748,-0.15664,0.59839,0.40859,-0.06510,0.53425,...,0.19067,0.56643,0.64422,0.19067,0.56419,0.75188,0.35665,0.49855,0.78865,0.09234
327,0.56972,0.33568,0.06316,0.50296,0.32752,-0.15634,0.59880,0.40813,-0.06450,0.53408,...,0.19898,0.56679,0.64492,0.19898,0.56422,0.75199,0.36450,0.49849,0.78865,0.07990
328,0.56974,0.33599,0.06421,0.50299,0.32758,-0.15726,0.59886,0.40800,-0.06480,0.53420,...,0.19967,0.56692,0.64584,0.19967,0.56423,0.75212,0.37113,0.49833,0.78867,0.08005
329,0.56976,0.33663,0.06402,0.50303,0.32761,-0.15937,0.59938,0.40799,-0.06771,0.53470,...,0.20554,0.56721,0.64627,0.20554,0.56424,0.75218,0.37702,0.49827,0.78908,0.07394


In [36]:
sqaut_bad_155_1 = pd.read_csv('datacsv\\sqaut_bad_155_1.csv')
sqaut_bad_155_1

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.53546,0.36441,0.13089,0.53757,0.36771,-0.17152,0.56928,0.43159,0.05345,0.58585,...,0.12504,0.56528,0.66994,0.12504,0.51942,0.78261,0.17197,0.51061,0.80993,-0.01082
1,0.53586,0.37145,0.13073,0.53561,0.36792,-0.19388,0.56945,0.43162,0.12774,0.57702,...,0.14824,0.57214,0.67025,0.14824,0.52508,0.77950,0.22527,0.51157,0.80567,0.05621
2,0.53615,0.37626,0.13021,0.53564,0.37566,-0.19846,0.57147,0.43295,0.12408,0.57864,...,0.15282,0.57489,0.67302,0.15282,0.52582,0.77929,0.21898,0.51153,0.80413,0.10398
3,0.53615,0.37769,0.13030,0.53569,0.37772,-0.20042,0.57144,0.43437,0.12454,0.57971,...,0.16054,0.57415,0.67451,0.16054,0.52544,0.77919,0.22771,0.51147,0.80321,0.10035
4,0.53856,0.38427,0.13084,0.53574,0.38102,-0.19494,0.57926,0.44747,0.11030,0.58055,...,0.15355,0.57870,0.67503,0.15355,0.52601,0.77898,0.21409,0.51146,0.80249,0.07550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,0.52657,0.32983,0.12454,0.52260,0.32556,-0.20620,0.56130,0.40830,0.03999,0.56973,...,0.23332,0.52985,0.65418,0.23332,0.52240,0.76431,0.35191,0.51668,0.80026,0.06654
323,0.52637,0.32987,0.12436,0.52274,0.32567,-0.20490,0.56036,0.40557,0.04066,0.56985,...,0.23266,0.52983,0.65415,0.23266,0.52239,0.76402,0.35173,0.51676,0.80032,0.06374
324,0.52639,0.32991,0.13575,0.52276,0.32547,-0.20174,0.56001,0.40588,0.04626,0.57024,...,0.23234,0.53095,0.65405,0.23234,0.52442,0.76388,0.35056,0.51700,0.80030,0.03663
325,0.52632,0.32991,0.13755,0.52275,0.32540,-0.20153,0.55962,0.40590,0.04852,0.57028,...,0.23175,0.53133,0.65404,0.23175,0.52457,0.76419,0.34602,0.51714,0.80029,0.04124


In [37]:
sqaut_bad_180_1 = pd.read_csv('datacsv\\sqaut_bad_180_1.csv')
sqaut_bad_180_1

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.54732,0.37247,0.10728,0.53484,0.35652,-0.15851,0.55780,0.38747,0.17507,0.53198,...,0.08501,0.54855,0.66627,0.08501,0.52130,0.77837,0.12844,0.53320,0.80237,-0.06142
1,0.54155,0.37264,0.12306,0.53570,0.36457,-0.17431,0.55605,0.39019,0.22816,0.55098,...,0.18716,0.55093,0.66559,0.18716,0.52546,0.75402,0.28667,0.53095,0.80238,-0.07550
2,0.53658,0.37151,0.12141,0.54110,0.36538,-0.19308,0.55587,0.39097,0.23183,0.56479,...,0.19487,0.55261,0.66612,0.19487,0.52540,0.75945,0.29986,0.53053,0.80322,-0.05292
3,0.53072,0.37040,0.12817,0.54473,0.36577,-0.18936,0.55309,0.39508,0.24767,0.57251,...,0.22135,0.55341,0.66611,0.22135,0.52438,0.75944,0.34592,0.52982,0.80469,-0.04112
4,0.53121,0.36982,0.12952,0.54384,0.36578,-0.19817,0.55452,0.39184,0.23771,0.56771,...,0.17760,0.56189,0.66740,0.17760,0.52798,0.75990,0.26947,0.52871,0.80641,-0.03978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,0.53832,0.33256,0.14102,0.53076,0.32868,-0.20365,0.56174,0.39087,0.09516,0.56244,...,0.17400,0.54051,0.66286,0.17400,0.53501,0.77661,0.25257,0.53629,0.80203,0.05471
342,0.53871,0.33237,0.14891,0.53072,0.32857,-0.18669,0.56110,0.39152,0.09477,0.55380,...,0.14765,0.54089,0.66083,0.14765,0.53509,0.77631,0.21226,0.53633,0.80156,0.02510
343,0.53880,0.33196,0.15030,0.53069,0.32826,-0.18074,0.56109,0.39161,0.08672,0.55238,...,0.14519,0.54114,0.65950,0.14519,0.53565,0.77558,0.21134,0.53658,0.80136,0.02290
344,0.53897,0.33200,0.14627,0.53078,0.32827,-0.19231,0.56262,0.39149,0.07213,0.55432,...,0.16438,0.54115,0.65948,0.16438,0.53563,0.77489,0.24942,0.53628,0.80085,0.05314


In [38]:
sqaut_bad_hand0_2 = pd.read_csv('datacsv\\sqaut_bad_hand0_2.csv')
sqaut_bad_hand0_2

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.51331,0.38277,-0.23466,0.51440,0.38588,0.09783,0.44900,0.42273,-0.29771,0.46556,...,-0.00742,0.53145,0.66461,-0.00742,0.53532,0.78245,0.10429,0.53704,0.75824,0.35992
1,0.51181,0.38644,-0.22053,0.51923,0.38629,0.10821,0.44799,0.42154,-0.27887,0.46832,...,-0.00744,0.53051,0.66662,-0.00744,0.53595,0.78352,0.10344,0.53916,0.75782,0.33298
2,0.51041,0.39065,-0.21888,0.52054,0.38899,0.10669,0.44699,0.42060,-0.26844,0.46739,...,-0.00135,0.53050,0.67113,-0.00135,0.53613,0.78423,0.11857,0.53986,0.75737,0.34096
3,0.50969,0.39256,-0.21494,0.52105,0.39046,0.10608,0.44624,0.41997,-0.26690,0.46690,...,-0.00078,0.53049,0.67276,-0.00078,0.53613,0.78470,0.11896,0.54040,0.75802,0.33816
4,0.50762,0.39603,-0.21580,0.52122,0.39374,0.10250,0.44241,0.42002,-0.26303,0.46250,...,-0.00624,0.53053,0.67220,-0.00624,0.53581,0.78471,0.09952,0.54073,0.75439,0.36315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,0.52897,0.36838,-0.23596,0.52325,0.37070,0.05961,0.52789,0.46393,-0.25504,0.52319,...,-0.00770,0.51979,0.67285,-0.00770,0.53880,0.78219,0.09665,0.53942,0.75839,0.27965
229,0.52837,0.36838,-0.23604,0.52321,0.37072,0.05954,0.52710,0.46421,-0.25682,0.52303,...,-0.00754,0.51984,0.67285,-0.00754,0.53910,0.78227,0.09781,0.53940,0.75861,0.28074
230,0.52755,0.36834,-0.23605,0.52311,0.37073,0.05951,0.52616,0.46427,-0.25679,0.52286,...,-0.00741,0.51996,0.67280,-0.00741,0.53908,0.78223,0.09804,0.53921,0.75844,0.28073
231,0.52714,0.36831,-0.23613,0.52294,0.37072,0.05962,0.52583,0.46429,-0.25685,0.52269,...,-0.00714,0.52004,0.67279,-0.00714,0.53898,0.78217,0.09857,0.53900,0.75833,0.28155


In [39]:
sqaut_bad_hand25_2 = pd.read_csv('datacsv\\sqaut_bad_hand25_2.csv')
sqaut_bad_hand25_2

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.51090,0.30440,-0.28840,0.49105,0.30791,0.07595,0.47484,0.39609,-0.31762,0.45865,...,-0.04219,0.50251,0.62103,-0.04219,0.50678,0.75116,0.07983,0.47872,0.70651,0.41641
1,0.51263,0.30434,-0.27184,0.49225,0.30782,0.09920,0.47487,0.37941,-0.31028,0.45866,...,-0.04286,0.50253,0.62548,-0.04286,0.50683,0.74971,0.08862,0.47834,0.70599,0.41845
2,0.51261,0.30410,-0.26727,0.49267,0.30784,0.10457,0.46037,0.36713,-0.29231,0.44929,...,-0.04286,0.50430,0.62883,-0.04286,0.50724,0.74887,0.11885,0.47801,0.70576,0.41734
3,0.51245,0.30391,-0.26824,0.49309,0.30784,0.10359,0.45647,0.36471,-0.28751,0.44682,...,-0.04285,0.50518,0.63015,-0.04285,0.50753,0.74838,0.12169,0.47757,0.70561,0.41666
4,0.51246,0.30402,-0.26855,0.49301,0.30571,0.10042,0.45094,0.35447,-0.30908,0.43439,...,-0.03324,0.50592,0.63017,-0.03324,0.50782,0.74794,0.12709,0.47706,0.70581,0.41893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,0.52738,0.30446,-0.25744,0.48415,0.30567,0.04151,0.52121,0.40543,-0.27567,0.47709,...,-0.04464,0.50250,0.62660,-0.04464,0.51147,0.74372,0.05129,0.47643,0.70501,0.38242
279,0.52704,0.30549,-0.25713,0.48515,0.30582,0.04187,0.52125,0.40579,-0.27575,0.47889,...,-0.04486,0.50252,0.62698,-0.04486,0.51149,0.74385,0.05032,0.47646,0.70515,0.38481
280,0.52680,0.30587,-0.25678,0.48548,0.30572,0.04393,0.52127,0.40605,-0.27619,0.47906,...,-0.04503,0.50248,0.62714,-0.04503,0.51149,0.74386,0.05026,0.47646,0.70519,0.38812
281,0.52661,0.30592,-0.25765,0.48557,0.30510,0.04400,0.52129,0.40624,-0.27688,0.47913,...,-0.04524,0.50247,0.62700,-0.04524,0.51150,0.74356,0.05043,0.47651,0.70508,0.38360


In [40]:
sqaut_bad_hand45_2 = pd.read_csv('datacsv\\sqaut_bad_hand45_2.csv')
sqaut_bad_hand45_2

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.53090,0.30771,-0.18789,0.46760,0.31589,0.06842,0.51933,0.39966,-0.23689,0.44044,...,-0.07219,0.50459,0.62736,-0.07219,0.51991,0.74554,0.04573,0.46652,0.70773,0.32137
1,0.53060,0.30728,-0.20194,0.46744,0.31576,0.05513,0.51933,0.39909,-0.25135,0.43859,...,-0.05444,0.50646,0.62821,-0.05444,0.51994,0.74519,0.08615,0.46480,0.70809,0.37060
2,0.52989,0.30626,-0.18885,0.46738,0.31524,0.06082,0.51840,0.39637,-0.23720,0.43373,...,-0.04888,0.50728,0.62882,-0.04888,0.51998,0.74482,0.10453,0.46380,0.70865,0.35142
3,0.52935,0.30548,-0.18563,0.46736,0.31480,0.06403,0.51772,0.39485,-0.23416,0.43220,...,-0.04763,0.50763,0.62913,-0.04763,0.51998,0.74460,0.10477,0.46311,0.70879,0.34717
4,0.52849,0.30423,-0.18436,0.46817,0.31357,0.07447,0.51110,0.39031,-0.23825,0.42885,...,-0.04998,0.50762,0.62897,-0.04998,0.51989,0.74411,0.09231,0.46266,0.70880,0.34358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,0.54200,0.30896,-0.16501,0.47165,0.31020,0.02889,0.54302,0.40476,-0.19169,0.46161,...,-0.05474,0.51140,0.62600,-0.05474,0.52294,0.74174,0.05431,0.46694,0.70537,0.32273
239,0.54193,0.30895,-0.16489,0.47070,0.31057,0.02521,0.54256,0.40478,-0.18995,0.46019,...,-0.05477,0.51140,0.62597,-0.05477,0.52283,0.74157,0.05521,0.46693,0.70537,0.32496
240,0.54178,0.30886,-0.16289,0.47011,0.31073,0.02833,0.54220,0.40476,-0.18630,0.45970,...,-0.05495,0.51142,0.62595,-0.05495,0.52279,0.74128,0.05342,0.46691,0.70530,0.32368
241,0.54174,0.30882,-0.16246,0.46963,0.31072,0.02664,0.54184,0.40471,-0.18277,0.45898,...,-0.05469,0.51123,0.62586,-0.05469,0.52270,0.74059,0.05746,0.46682,0.70458,0.32414


In [41]:
sqaut_bad_hand60_2 = pd.read_csv('datacsv\\sqaut_bad_hand60_2.csv')
sqaut_bad_hand60_2

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.52868,0.30618,-0.18155,0.45331,0.31790,0.04101,0.48109,0.31875,-0.35571,0.39652,...,-0.01817,0.51464,0.62998,-0.01817,0.52561,0.73431,0.17132,0.44891,0.71340,0.34910
1,0.52781,0.30576,-0.19813,0.45774,0.31750,0.04112,0.46552,0.31472,-0.34296,0.40021,...,-0.00545,0.51525,0.62875,-0.00545,0.52629,0.73452,0.15432,0.44896,0.71307,0.31589
2,0.52660,0.30692,-0.19917,0.45993,0.32037,0.03951,0.46890,0.31099,-0.35131,0.40309,...,0.00361,0.51690,0.62931,0.00361,0.52667,0.73485,0.15541,0.44896,0.71270,0.35860
3,0.52589,0.30714,-0.19826,0.46107,0.32157,0.03931,0.47060,0.30798,-0.35436,0.40399,...,0.00168,0.51795,0.62956,0.00168,0.52677,0.73503,0.15419,0.44896,0.71258,0.35657
4,0.52486,0.31016,-0.19644,0.46082,0.32569,0.03686,0.47152,0.30890,-0.33812,0.40406,...,-0.01613,0.51923,0.63161,-0.01613,0.52689,0.73473,0.15408,0.44896,0.71188,0.21264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,0.53857,0.30699,-0.17546,0.45680,0.31096,-0.06587,0.54802,0.40855,-0.15338,0.43869,...,-0.02780,0.51589,0.62919,-0.02780,0.52803,0.74038,0.12813,0.45143,0.71225,0.29591
244,0.53867,0.30721,-0.17684,0.45679,0.31094,-0.06675,0.54902,0.41021,-0.15491,0.43847,...,-0.02682,0.51573,0.62890,-0.02682,0.52801,0.74040,0.12767,0.45141,0.71230,0.30319
245,0.53870,0.30725,-0.17625,0.45679,0.31081,-0.06318,0.54936,0.41094,-0.15479,0.43841,...,-0.02599,0.51546,0.62844,-0.02599,0.52790,0.74017,0.12714,0.45141,0.71242,0.30315
246,0.53882,0.30734,-0.17752,0.45688,0.31081,-0.06222,0.54956,0.41119,-0.15868,0.43838,...,-0.02605,0.51526,0.62821,-0.02605,0.52791,0.74021,0.11769,0.45141,0.71244,0.31685


In [42]:
sqaut_bad_hand90_2 = pd.read_csv('datacsv\\sqaut_bad_hand90_2.csv')
sqaut_bad_hand90_2

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.54013,0.30555,-0.12085,0.45005,0.30507,-0.17289,0.59017,0.33936,-0.36432,0.41204,...,0.00767,0.54235,0.63424,0.00767,0.55955,0.73393,0.24392,0.45407,0.74934,0.23593
1,0.54001,0.30508,-0.10566,0.44929,0.30519,-0.15862,0.58853,0.33591,-0.34060,0.41256,...,0.03708,0.54252,0.63414,0.03708,0.55943,0.73407,0.28477,0.45427,0.74849,0.20609
2,0.54001,0.30532,-0.10951,0.44923,0.30566,-0.16061,0.58856,0.32309,-0.39905,0.41492,...,0.03280,0.54254,0.63121,0.03280,0.55924,0.73394,0.27475,0.45433,0.74665,0.17959
3,0.54001,0.30550,-0.11466,0.44920,0.30604,-0.16199,0.58858,0.31959,-0.40837,0.41621,...,0.03039,0.54254,0.62972,0.03039,0.55908,0.73384,0.26789,0.45438,0.74563,0.17599
4,0.54018,0.31061,-0.15027,0.44933,0.30996,-0.16443,0.58856,0.31422,-0.46406,0.41703,...,0.02140,0.54264,0.62880,0.02140,0.55896,0.73380,0.23434,0.45438,0.74479,0.18613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,0.54163,0.30488,-0.10847,0.44799,0.30432,-0.13994,0.55744,0.40369,-0.04428,0.43814,...,0.09681,0.53970,0.62502,0.09681,0.55776,0.73250,0.30889,0.45472,0.74135,0.20042
239,0.54163,0.30479,-0.10597,0.44810,0.30436,-0.13900,0.55739,0.40350,-0.04196,0.43800,...,0.09774,0.53967,0.62458,0.09774,0.55765,0.73212,0.31096,0.45491,0.74118,0.19570
240,0.54165,0.30479,-0.10259,0.44818,0.30439,-0.13560,0.55739,0.40350,-0.03941,0.43789,...,0.09659,0.53971,0.62440,0.09659,0.55760,0.73177,0.31560,0.45505,0.74084,0.20008
241,0.54218,0.30467,-0.10186,0.44834,0.30439,-0.13526,0.55751,0.40351,-0.03839,0.43781,...,0.09543,0.54006,0.62442,0.09543,0.55770,0.73169,0.31893,0.45510,0.74084,0.20310


In [43]:
sqaut_bad_hand105_2 = pd.read_csv('datacsv\\sqaut_bad_hand105_2.csv')
sqaut_bad_hand105_2

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.54911,0.31681,0.04187,0.47250,0.30818,-0.18497,0.59954,0.32444,-0.06024,0.51297,...,0.13893,0.54662,0.61630,0.13893,0.54897,0.71391,0.35611,0.47422,0.74285,0.06170
1,0.54715,0.31755,0.04892,0.47353,0.30793,-0.19518,0.59954,0.32256,0.06095,0.51546,...,0.12758,0.54671,0.61194,0.12758,0.54911,0.71370,0.31673,0.47419,0.74186,0.06296
2,0.54597,0.31850,0.05582,0.47619,0.30875,-0.19492,0.59885,0.32066,0.07183,0.52067,...,0.11439,0.54670,0.60579,0.11439,0.54923,0.71350,0.27875,0.47418,0.74083,0.05818
3,0.54533,0.31912,0.05905,0.47724,0.30923,-0.19421,0.59851,0.31964,0.07035,0.52155,...,0.10428,0.54670,0.60388,0.10428,0.54940,0.71350,0.25784,0.47412,0.73947,0.05778
4,0.54490,0.32485,0.01915,0.47823,0.31329,-0.23232,0.59843,0.31638,0.04294,0.51256,...,0.05618,0.54597,0.60249,0.05618,0.54919,0.71287,0.20521,0.47403,0.73746,0.06563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,0.54306,0.31151,-0.00754,0.46246,0.30414,-0.15807,0.56244,0.40468,0.06412,0.45236,...,0.14361,0.54181,0.61535,0.14361,0.55023,0.71423,0.33745,0.47410,0.74495,0.07756
244,0.54305,0.31144,-0.01092,0.46241,0.30426,-0.16202,0.56267,0.40466,0.06170,0.45166,...,0.14676,0.54190,0.61598,0.14676,0.55028,0.71438,0.34421,0.47416,0.74515,0.08076
245,0.54305,0.31102,-0.01088,0.46230,0.30427,-0.16505,0.56268,0.40464,0.06196,0.45130,...,0.16080,0.54186,0.61618,0.16080,0.55027,0.71420,0.37007,0.47424,0.74513,0.08504
246,0.54290,0.31105,-0.00416,0.46225,0.30464,-0.16451,0.56244,0.40485,0.06971,0.45087,...,0.17003,0.54186,0.61702,0.17003,0.55026,0.71441,0.37912,0.47421,0.74528,0.08284


In [44]:
sqaut_bad_hand135_2 = pd.read_csv('datacsv\\sqaut_bad_hand135_2.csv')
sqaut_bad_hand135_2

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.53494,0.31382,0.04567,0.47315,0.30181,-0.19913,0.55632,0.39718,0.11227,0.48145,...,0.21268,0.53686,0.61349,0.21268,0.53291,0.70893,0.39028,0.47929,0.74798,0.12025
1,0.53503,0.31314,0.05099,0.47315,0.30188,-0.19864,0.56014,0.39521,0.11327,0.48473,...,0.21320,0.53767,0.61120,0.21320,0.53460,0.70841,0.39250,0.47972,0.74795,0.13967
2,0.53578,0.31314,0.05904,0.47317,0.30206,-0.19873,0.56746,0.39295,0.11659,0.48588,...,0.21767,0.53783,0.61114,0.21767,0.53550,0.70790,0.40544,0.47981,0.74804,0.13751
3,0.53614,0.31316,0.05998,0.47320,0.30224,-0.19919,0.56928,0.39195,0.11770,0.48663,...,0.22038,0.53795,0.61119,0.22038,0.53609,0.70770,0.40940,0.47990,0.74808,0.13901
4,0.53598,0.31332,0.06284,0.47355,0.30224,-0.20037,0.57212,0.38651,0.12123,0.49545,...,0.22464,0.53836,0.61112,0.22464,0.53628,0.70771,0.42511,0.47990,0.74842,0.13651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,0.52737,0.30965,0.06858,0.46106,0.30285,-0.19708,0.53681,0.40273,0.12963,0.45925,...,0.18211,0.53242,0.60572,0.18211,0.53313,0.70806,0.34656,0.47710,0.74562,0.01647
239,0.52777,0.30965,0.06756,0.46120,0.30298,-0.19361,0.53749,0.40271,0.12896,0.45978,...,0.17619,0.53268,0.60569,0.17619,0.53319,0.70776,0.34269,0.47723,0.74534,0.02056
240,0.52792,0.30965,0.06708,0.46153,0.30298,-0.19280,0.53793,0.40269,0.12976,0.46007,...,0.17212,0.53279,0.60565,0.17212,0.53333,0.70738,0.33916,0.47726,0.74532,0.01999
241,0.52810,0.30956,0.06571,0.46203,0.30265,-0.19196,0.53801,0.40268,0.12878,0.46049,...,0.16865,0.53317,0.60552,0.16865,0.53359,0.70722,0.33843,0.47753,0.74528,0.02284


In [45]:
sqaut_bad_hand155_2 = pd.read_csv('datacsv\\sqaut_bad_hand155_2.csv')
sqaut_bad_hand155_2

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.51127,0.31495,0.04297,0.48515,0.30745,-0.27264,0.53428,0.39314,0.09242,0.53323,...,0.27052,0.52649,0.60527,0.27052,0.52511,0.70670,0.46894,0.49042,0.74949,0.18450
1,0.51325,0.31192,0.07450,0.48532,0.30738,-0.27058,0.53876,0.38496,0.12662,0.52993,...,0.27088,0.52670,0.60436,0.27088,0.52500,0.70621,0.46675,0.49040,0.74923,0.14241
2,0.51204,0.31055,0.07365,0.48792,0.30694,-0.26704,0.52512,0.38543,0.11509,0.54796,...,0.27283,0.52669,0.60131,0.27283,0.52486,0.70554,0.46682,0.49003,0.74870,0.13202
3,0.51147,0.31015,0.07408,0.48908,0.30680,-0.26336,0.52128,0.38643,0.11081,0.55185,...,0.27354,0.52618,0.60105,0.27354,0.52419,0.70549,0.46658,0.48987,0.74869,0.13324
4,0.51214,0.30907,0.06840,0.48937,0.30724,-0.27557,0.55510,0.36079,0.13352,0.55448,...,0.27197,0.52586,0.60098,0.27197,0.52347,0.70549,0.46217,0.48947,0.74862,0.14504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,0.52162,0.31041,0.08659,0.47418,0.30172,-0.24939,0.52866,0.40382,0.14680,0.47641,...,0.21105,0.52944,0.61665,0.21105,0.51955,0.70485,0.36716,0.48663,0.74758,0.14865
279,0.52058,0.31024,0.08228,0.47430,0.30172,-0.25391,0.52706,0.40373,0.14210,0.47639,...,0.21348,0.52930,0.61658,0.21348,0.51884,0.70507,0.37131,0.48658,0.74764,0.15431
280,0.52006,0.31008,0.08032,0.47431,0.30174,-0.25410,0.52608,0.40372,0.14038,0.47637,...,0.21391,0.52921,0.61664,0.21391,0.51876,0.70515,0.37227,0.48657,0.74770,0.15819
281,0.51992,0.30963,0.08036,0.47436,0.30162,-0.25496,0.52570,0.40370,0.14076,0.47636,...,0.21351,0.52921,0.61611,0.21351,0.51878,0.70502,0.37196,0.48661,0.74750,0.15566


In [46]:
sqaut_bad_hand180_2 = pd.read_csv('datacsv\\sqaut_bad_hand180_2.csv')
sqaut_bad_hand180_2

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.48804,0.38231,0.10296,0.49011,0.38890,-0.21972,0.53695,0.42306,0.18521,0.54869,...,0.20460,0.46891,0.65638,0.20460,0.46415,0.76161,0.33055,0.46259,0.78281,0.10797
1,0.48721,0.38367,0.08857,0.49039,0.39049,-0.22540,0.53706,0.42307,0.13746,0.55367,...,0.22023,0.46843,0.65997,0.22023,0.46415,0.76159,0.35721,0.46259,0.78375,0.14236
2,0.48739,0.38736,0.06918,0.49073,0.39229,-0.23793,0.54474,0.42152,0.11603,0.56024,...,0.21900,0.46799,0.66231,0.21900,0.46393,0.76249,0.35227,0.46251,0.78474,0.14093
3,0.48759,0.38837,0.06437,0.49109,0.39373,-0.24387,0.54630,0.41912,0.11581,0.56102,...,0.21809,0.46691,0.66353,0.21809,0.46371,0.76267,0.35025,0.46249,0.78497,0.14056
4,0.48934,0.39295,0.06231,0.49204,0.39692,-0.24674,0.55185,0.41814,0.11203,0.56229,...,0.21657,0.46580,0.66310,0.21657,0.46339,0.76211,0.34754,0.46243,0.78495,0.13419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,0.47511,0.36645,0.08861,0.47187,0.36569,-0.22216,0.46816,0.45691,0.09754,0.46832,...,0.18519,0.48050,0.65095,0.18519,0.46904,0.76039,0.30057,0.46417,0.78236,0.07739
229,0.47535,0.36646,0.08997,0.47259,0.36568,-0.22242,0.46841,0.45690,0.09901,0.46921,...,0.18111,0.48016,0.65098,0.18111,0.46541,0.75828,0.29060,0.46421,0.78267,0.07340
230,0.47535,0.36647,0.09436,0.47393,0.36558,-0.22044,0.46793,0.45689,0.10237,0.47088,...,0.17954,0.47993,0.65097,0.17954,0.46414,0.75703,0.28663,0.46421,0.78271,0.06871
231,0.47538,0.36647,0.09430,0.47467,0.36561,-0.22076,0.46759,0.45690,0.10277,0.47127,...,0.18015,0.47952,0.65093,0.18015,0.46328,0.75606,0.28872,0.46421,0.78274,0.07012


In [47]:
sqaut_bad_0_2 = pd.read_csv('datacsv\\sqaut_bad_0_2.csv')
sqaut_bad_0_2

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.49343,0.30860,-0.21373,0.50116,0.31125,0.11228,0.44445,0.35167,-0.15781,0.46901,...,-0.06829,0.49226,0.64167,-0.06829,0.49512,0.77164,-0.02242,0.49841,0.74716,0.27553
1,0.49381,0.30866,-0.21298,0.49967,0.31352,0.11828,0.44564,0.35199,-0.16979,0.46846,...,-0.06893,0.49416,0.64338,-0.06893,0.49523,0.77073,-0.01541,0.49881,0.74753,0.27265
2,0.49399,0.31060,-0.20941,0.49922,0.31763,0.11838,0.44910,0.35289,-0.17093,0.46819,...,-0.06894,0.49573,0.64472,-0.06894,0.49527,0.77015,-0.01474,0.49913,0.74763,0.26749
3,0.49399,0.31091,-0.20718,0.49886,0.31905,0.11793,0.45035,0.35326,-0.17003,0.46819,...,-0.06555,0.49666,0.64493,-0.06555,0.49521,0.76946,-0.00787,0.49938,0.74754,0.26777
4,0.49367,0.31499,-0.20088,0.49820,0.32259,0.12198,0.44379,0.35580,-0.14404,0.45924,...,-0.06405,0.49843,0.64779,-0.06405,0.49517,0.76934,-0.00801,0.49964,0.74852,0.26367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,0.50839,0.29397,-0.21397,0.51476,0.29833,0.12743,0.47809,0.39540,-0.14052,0.49668,...,-0.03602,0.48697,0.63442,-0.03602,0.49337,0.76580,0.04045,0.49841,0.74909,0.24715
281,0.50815,0.29402,-0.21394,0.51465,0.29824,0.12650,0.47818,0.39760,-0.13888,0.49595,...,-0.03497,0.48696,0.63440,-0.03497,0.49343,0.76575,0.04483,0.49840,0.74907,0.24982
282,0.50812,0.29424,-0.21879,0.51449,0.29780,0.12368,0.47810,0.39749,-0.14297,0.49574,...,-0.02952,0.48699,0.63447,-0.02952,0.49362,0.76559,0.05685,0.49828,0.74887,0.26040
283,0.50807,0.29446,-0.21991,0.51439,0.29749,0.12223,0.47787,0.39761,-0.14398,0.49547,...,-0.02796,0.48703,0.63448,-0.02796,0.49375,0.76544,0.05863,0.49820,0.74871,0.26210


In [48]:
sqaut_bad_25_2 = pd.read_csv('datacsv\\sqaut_bad_25_2.csv')
sqaut_bad_25_2

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.48435,0.34338,-0.19435,0.46659,0.35470,0.08967,0.42908,0.35614,-0.19272,0.41629,...,-0.18031,0.51132,0.64353,-0.18031,0.50330,0.76291,-0.18291,0.47318,0.70948,0.29332
1,0.48423,0.34473,-0.19343,0.46993,0.35528,0.09517,0.42630,0.36274,-0.19249,0.41927,...,-0.10475,0.50990,0.64607,-0.10475,0.50269,0.76264,-0.09265,0.47583,0.71138,0.27579
2,0.48353,0.34822,-0.18101,0.47130,0.35907,0.11658,0.42646,0.36304,-0.18995,0.42190,...,-0.12350,0.50752,0.64673,-0.12350,0.50241,0.76057,-0.12467,0.47886,0.71628,0.17852
3,0.48329,0.34969,-0.18120,0.47222,0.35982,0.11582,0.42976,0.36592,-0.19269,0.42370,...,-0.12438,0.50648,0.64716,-0.12438,0.50223,0.76044,-0.12098,0.47898,0.71577,0.21227
4,0.48224,0.35569,-0.17470,0.47194,0.36679,0.12608,0.42833,0.37105,-0.18433,0.43003,...,-0.18788,0.50549,0.64687,-0.18788,0.50232,0.75976,-0.20807,0.47893,0.71560,0.16952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,0.51998,0.29825,-0.19912,0.49644,0.29599,0.11549,0.47383,0.38823,-0.12800,0.47105,...,-0.05260,0.50272,0.62798,-0.05260,0.50739,0.75499,0.01363,0.48126,0.71285,0.33462
239,0.51997,0.29801,-0.19626,0.49628,0.29616,0.11543,0.47376,0.38757,-0.12464,0.47091,...,-0.05338,0.50254,0.62719,-0.05338,0.50734,0.75419,0.00899,0.48159,0.71283,0.33912
240,0.51993,0.29754,-0.19662,0.49620,0.29626,0.11509,0.47366,0.38615,-0.12529,0.47091,...,-0.05570,0.50240,0.62650,-0.05570,0.50731,0.75289,0.00447,0.48187,0.71280,0.33894
241,0.52009,0.29763,-0.19867,0.49629,0.29633,0.11232,0.47389,0.38604,-0.13092,0.47084,...,-0.05244,0.50229,0.62723,-0.05244,0.50727,0.75332,0.01843,0.48255,0.71336,0.33933


In [49]:
sqaut_bad_45_2 = pd.read_csv('datacsv\\sqaut_bad_45_2.csv')
sqaut_bad_45_2

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.50583,0.32114,-0.25854,0.46809,0.33434,0.02495,0.43230,0.36066,-0.24937,0.42515,...,-0.12713,0.51213,0.62936,-0.12713,0.52000,0.75574,-0.11326,0.45572,0.70607,0.47121
1,0.50712,0.32462,-0.24413,0.46739,0.33718,0.02743,0.43393,0.36645,-0.23534,0.41294,...,-0.15915,0.51255,0.62940,-0.15915,0.52006,0.75401,-0.16626,0.45590,0.70402,0.45224
2,0.50785,0.33060,-0.17777,0.46475,0.34724,0.07285,0.43824,0.37088,-0.19846,0.41237,...,-0.29384,0.51248,0.62292,-0.29384,0.52011,0.75014,-0.31343,0.45599,0.70156,0.40118
3,0.50839,0.33245,-0.17028,0.46344,0.34978,0.07747,0.43863,0.37136,-0.18297,0.40750,...,-0.28825,0.51249,0.62071,-0.28825,0.52008,0.74872,-0.30812,0.45621,0.70128,0.37621
4,0.50832,0.34158,-0.18341,0.46176,0.35514,0.06143,0.43911,0.37620,-0.18759,0.40596,...,-0.30351,0.51334,0.62027,-0.30351,0.52007,0.74808,-0.29927,0.45635,0.70105,0.36838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,0.53447,0.30184,-0.15473,0.46657,0.30493,0.06674,0.50898,0.38265,-0.11733,0.44139,...,-0.07084,0.50152,0.61879,-0.07084,0.51840,0.75144,-0.00544,0.45995,0.70723,0.33883
258,0.53430,0.30191,-0.15416,0.46661,0.30494,0.06677,0.50901,0.38275,-0.11699,0.44158,...,-0.07186,0.50149,0.61856,-0.07186,0.51840,0.75144,-0.00758,0.45995,0.70720,0.34064
259,0.53415,0.30191,-0.15367,0.46664,0.30494,0.06713,0.50960,0.38274,-0.11858,0.44185,...,-0.07106,0.50124,0.61803,-0.07106,0.51828,0.75129,-0.00419,0.45995,0.70719,0.33585
260,0.53405,0.30229,-0.15334,0.46665,0.30506,0.06761,0.50997,0.38334,-0.11795,0.44206,...,-0.07282,0.50113,0.61805,-0.07282,0.51827,0.75150,-0.00569,0.46006,0.70723,0.33431


In [50]:
sqaut_bad_60_2 = pd.read_csv('datacsv\\sqaut_bad_60_2.csv')
sqaut_bad_60_2

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.53618,0.30188,-0.15433,0.46707,0.31140,0.04478,0.47714,0.35509,-0.20983,0.42322,...,-0.05379,0.52444,0.64561,-0.05379,0.53485,0.77515,0.03242,0.45534,0.72823,0.50216
1,0.53588,0.30291,-0.14991,0.46726,0.31337,0.04714,0.47620,0.35197,-0.20863,0.42122,...,-0.05132,0.52495,0.64560,-0.05132,0.53329,0.77521,0.06743,0.45609,0.72958,0.40665
2,0.53494,0.30693,-0.15534,0.46569,0.31744,0.04065,0.47598,0.35395,-0.18318,0.41648,...,-0.06689,0.52661,0.64664,-0.06689,0.53332,0.77614,0.02749,0.45631,0.73048,0.50037
3,0.53429,0.30889,-0.15562,0.46527,0.31934,0.04025,0.47448,0.35423,-0.18558,0.41485,...,-0.06733,0.52756,0.64764,-0.06733,0.53348,0.77623,0.02927,0.45698,0.73045,0.50600
4,0.53324,0.31463,-0.16119,0.46487,0.32381,0.04031,0.47278,0.35934,-0.18623,0.41350,...,-0.05528,0.52858,0.64914,-0.05528,0.53380,0.77622,0.04124,0.45732,0.73019,0.53369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,0.53836,0.29234,-0.13800,0.45986,0.29976,0.02975,0.51745,0.38080,-0.12743,0.43819,...,-0.01754,0.51605,0.63401,-0.01754,0.53169,0.76743,0.10234,0.45589,0.72767,0.33914
227,0.53807,0.29219,-0.13907,0.45890,0.29979,0.03056,0.52066,0.38082,-0.13461,0.43784,...,-0.01701,0.51603,0.63399,-0.01701,0.53155,0.76673,0.10297,0.45583,0.72741,0.34355
228,0.53793,0.29210,-0.14163,0.45823,0.29981,0.02600,0.52216,0.38093,-0.14144,0.43760,...,-0.01461,0.51602,0.63397,-0.01461,0.53143,0.76620,0.10677,0.45571,0.72709,0.34975
229,0.53792,0.29334,-0.15039,0.45829,0.30105,0.02455,0.52160,0.38097,-0.14868,0.43712,...,-0.01398,0.51619,0.63698,-0.01398,0.53149,0.76748,0.11520,0.45566,0.72836,0.35134


In [51]:
sqaut_bad_90_2 = pd.read_csv('datacsv\\sqaut_bad_90_2.csv')
sqaut_bad_90_2

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.54502,0.37365,-0.12784,0.45137,0.37806,-0.10738,0.54200,0.38422,-0.38039,0.45384,...,-0.32270,0.56225,0.61658,-0.32270,0.56261,0.73220,-0.15190,0.46025,0.74200,-0.15340
1,0.54495,0.37590,-0.12995,0.45137,0.38013,-0.10965,0.54536,0.39203,-0.38914,0.45153,...,-0.33553,0.56232,0.61646,-0.33553,0.56322,0.73240,-0.16283,0.46025,0.74190,-0.16990
2,0.54423,0.37901,-0.11346,0.45135,0.38378,-0.09919,0.55018,0.40227,-0.35862,0.45069,...,-0.32815,0.56293,0.61791,-0.32815,0.56345,0.73274,-0.14255,0.46023,0.74199,-0.15525
3,0.54403,0.38037,-0.11606,0.45140,0.38530,-0.10025,0.55038,0.40298,-0.36522,0.45082,...,-0.32772,0.56359,0.62039,-0.32772,0.56361,0.73326,-0.14226,0.46021,0.74226,-0.15441
4,0.54329,0.38490,-0.11337,0.45162,0.39045,-0.09870,0.55092,0.40851,-0.36555,0.45087,...,-0.32165,0.56492,0.62396,-0.32165,0.56371,0.73355,-0.13462,0.46021,0.74239,-0.15026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,0.54996,0.30259,-0.08435,0.45603,0.30161,-0.13449,0.56462,0.40167,-0.01749,0.44080,...,0.10687,0.54539,0.63121,0.10687,0.56763,0.74336,0.33077,0.46008,0.74641,0.20178
207,0.54994,0.30247,-0.07722,0.45592,0.30156,-0.12963,0.56339,0.40224,-0.01359,0.44230,...,0.10530,0.54542,0.63116,0.10530,0.56642,0.74180,0.32529,0.45994,0.74640,0.19499
208,0.54994,0.30247,-0.07479,0.45588,0.30153,-0.12770,0.56279,0.40270,-0.01199,0.44321,...,0.10525,0.54545,0.63081,0.10525,0.56527,0.74016,0.32260,0.45978,0.74635,0.18967
209,0.55001,0.30245,-0.10109,0.45587,0.30138,-0.13873,0.56294,0.40314,-0.02992,0.44456,...,0.08070,0.54548,0.63026,0.08070,0.56470,0.73949,0.27863,0.45957,0.74655,0.20096


In [52]:
sqaut_bad_105_2 = pd.read_csv('datacsv\\sqaut_bad_105_2.csv')
sqaut_bad_105_2

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.53373,0.31130,0.03113,0.46931,0.30873,-0.17246,0.57476,0.34694,-0.14196,0.52441,...,0.11863,0.53868,0.62706,0.11863,0.54328,0.73030,0.32951,0.46991,0.77692,0.10142
1,0.53546,0.31168,0.04789,0.46937,0.30887,-0.14454,0.57465,0.34958,-0.11179,0.52150,...,0.08965,0.53770,0.62390,0.08965,0.54308,0.72990,0.29164,0.46986,0.77485,0.10143
2,0.54010,0.31635,0.04285,0.46928,0.31460,-0.13751,0.58179,0.35374,-0.12048,0.51697,...,0.09273,0.53658,0.62357,0.09273,0.54287,0.72955,0.29414,0.46961,0.77487,0.09937
3,0.54177,0.31849,0.03561,0.46917,0.31747,-0.14132,0.58481,0.35514,-0.12440,0.51602,...,0.08488,0.53651,0.62363,0.08488,0.54288,0.72931,0.29740,0.46948,0.77489,0.10006
4,0.54200,0.32298,0.02839,0.46909,0.32317,-0.13641,0.58754,0.35879,-0.12565,0.50933,...,0.08992,0.53610,0.62369,0.08992,0.54286,0.72934,0.29758,0.46883,0.77444,0.10120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,0.53876,0.30001,0.06173,0.46474,0.28864,-0.13806,0.56087,0.39021,-0.03843,0.49172,...,0.19607,0.54109,0.62038,0.19607,0.54423,0.72932,0.41891,0.46896,0.77173,0.12789
227,0.53968,0.29984,0.06523,0.46525,0.28856,-0.13795,0.56213,0.38959,-0.03580,0.49176,...,0.19470,0.54123,0.62037,0.19470,0.54425,0.72929,0.41488,0.46935,0.77170,0.12547
228,0.54036,0.29976,0.06528,0.46545,0.28850,-0.13709,0.56285,0.38923,-0.03581,0.49173,...,0.19518,0.54131,0.62037,0.19518,0.54426,0.72929,0.41361,0.46955,0.77171,0.12208
229,0.54016,0.30061,0.06927,0.46615,0.28870,-0.13955,0.56288,0.38957,-0.03561,0.49171,...,0.19378,0.54089,0.62067,0.19378,0.54415,0.73049,0.41344,0.46902,0.77270,0.12304


In [53]:
sqaut_bad_135_2 = pd.read_csv('datacsv\\sqaut_bad_135_2.csv')
sqaut_bad_135_2

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.53736,0.33710,0.01670,0.49102,0.32730,-0.21744,0.59038,0.34460,-0.11091,0.55432,...,0.08257,0.54160,0.58992,0.08257,0.54109,0.70530,0.22164,0.47936,0.75034,-0.07429
1,0.53949,0.33822,0.02008,0.49095,0.32885,-0.19698,0.59220,0.34547,-0.12407,0.55187,...,0.01599,0.54151,0.58731,0.01599,0.54114,0.70476,0.14048,0.47934,0.75014,-0.14577
2,0.54226,0.34421,0.02737,0.49093,0.33559,-0.17537,0.59287,0.35107,-0.13230,0.54834,...,-0.06464,0.54160,0.58660,-0.06464,0.54128,0.70469,0.05471,0.47934,0.74854,-0.21694
3,0.54290,0.34614,0.01439,0.49093,0.33868,-0.18923,0.59357,0.35190,-0.14527,0.54840,...,-0.04983,0.54199,0.58664,-0.04983,0.54141,0.70469,0.07762,0.47940,0.74722,-0.21918
4,0.54291,0.35451,0.00927,0.49092,0.34549,-0.19175,0.59472,0.35794,-0.14743,0.54833,...,-0.06475,0.54318,0.58805,-0.06475,0.54167,0.70534,0.05754,0.47965,0.74730,-0.16400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,0.53405,0.30458,0.06651,0.46901,0.30233,-0.14861,0.55714,0.39040,-0.00792,0.48431,...,0.22980,0.53882,0.60299,0.22980,0.54272,0.70834,0.40964,0.48143,0.75663,0.04660
258,0.53401,0.30459,0.06633,0.46914,0.30233,-0.14836,0.55712,0.39038,-0.00722,0.48404,...,0.22858,0.53879,0.60312,0.22858,0.54272,0.70834,0.40851,0.48142,0.75678,0.04640
259,0.53402,0.30460,0.06584,0.46919,0.30234,-0.14834,0.55719,0.39040,-0.00805,0.48436,...,0.22791,0.53883,0.60335,0.22791,0.54280,0.70834,0.40985,0.48143,0.75674,0.05030
260,0.53402,0.30499,0.06524,0.46923,0.30257,-0.15109,0.55746,0.39088,-0.00924,0.48437,...,0.23201,0.53894,0.60382,0.23201,0.54281,0.70837,0.41626,0.48141,0.75712,0.05155


In [54]:
sqaut_bad_155_2 = pd.read_csv('datacsv\\sqaut_bad_155_2.csv')
sqaut_bad_155_2

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.53659,0.35584,0.12229,0.51299,0.33892,-0.19044,0.58016,0.35701,0.16701,0.56893,...,0.11284,0.53481,0.59602,0.11284,0.52740,0.70005,0.26401,0.49695,0.76344,-0.12581
1,0.53676,0.35688,0.12348,0.51311,0.34345,-0.16639,0.57640,0.35786,0.18304,0.56946,...,0.10641,0.53499,0.59599,0.10641,0.52613,0.70032,0.23080,0.49734,0.76235,-0.17008
2,0.53914,0.36206,0.12467,0.51311,0.34935,-0.15023,0.57650,0.36525,0.17973,0.56938,...,0.07283,0.53585,0.59670,0.07283,0.52548,0.70075,0.18769,0.49746,0.76168,-0.17031
3,0.53903,0.36282,0.12435,0.51321,0.35202,-0.16069,0.57380,0.36644,0.18688,0.56996,...,0.07973,0.53622,0.59753,0.07973,0.52494,0.70120,0.20327,0.49805,0.76160,-0.15767
4,0.54059,0.37198,0.12496,0.51335,0.35796,-0.14562,0.57741,0.38062,0.15211,0.56995,...,0.03502,0.53785,0.59747,0.03502,0.52566,0.70093,0.13848,0.49885,0.76155,-0.15218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,0.50511,0.29438,0.12715,0.48527,0.29413,-0.21435,0.52377,0.38720,0.05086,0.51568,...,0.29164,0.52465,0.61095,0.29164,0.52144,0.71161,0.46827,0.49221,0.75686,-0.01711
239,0.50584,0.29444,0.12696,0.48483,0.29446,-0.21359,0.52393,0.38765,0.05240,0.51077,...,0.28434,0.52566,0.61094,0.28434,0.52227,0.71094,0.45662,0.49223,0.75726,-0.02105
240,0.50585,0.29472,0.12818,0.48498,0.29442,-0.21331,0.52408,0.38780,0.05649,0.51194,...,0.28626,0.52571,0.61082,0.28626,0.52228,0.70978,0.45873,0.49248,0.75726,-0.02362
241,0.50590,0.29499,0.12685,0.48478,0.29502,-0.21611,0.52425,0.38792,0.05260,0.51233,...,0.29558,0.52574,0.61074,0.29558,0.52264,0.70979,0.47632,0.49279,0.75761,-0.02024


In [55]:
sqaut_bad_180_2 = pd.read_csv('datacsv\\sqaut_bad_180_2.csv')
sqaut_bad_180_2

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.50747,0.30907,0.13958,0.50444,0.30688,-0.19580,0.54351,0.34439,0.12364,0.56093,...,0.12533,0.50766,0.62954,0.12533,0.50236,0.75101,0.16586,0.50254,0.76612,0.00853
1,0.50820,0.30897,0.12209,0.50391,0.30740,-0.20493,0.53204,0.36590,0.09265,0.56092,...,0.15048,0.50775,0.62958,0.15048,0.50424,0.75041,0.22515,0.50343,0.76605,0.06698
2,0.50865,0.31151,0.12225,0.50403,0.31155,-0.20628,0.53459,0.36133,0.09912,0.56522,...,0.14639,0.50674,0.63174,0.14639,0.50506,0.75052,0.21011,0.50384,0.76606,0.06585
3,0.50914,0.31218,0.11793,0.50410,0.31376,-0.21138,0.53753,0.35944,0.09098,0.56694,...,0.15452,0.50643,0.63277,0.15452,0.50516,0.75052,0.22509,0.50394,0.76606,0.08371
4,0.50946,0.31664,0.11722,0.50597,0.32010,-0.20905,0.53793,0.35862,0.09426,0.56918,...,0.14893,0.50376,0.63380,0.14893,0.50511,0.75086,0.21193,0.50394,0.76603,0.08017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,0.49237,0.30182,0.17191,0.49078,0.29006,-0.18302,0.49106,0.38601,0.17547,0.49139,...,0.15429,0.50922,0.62386,0.15429,0.50181,0.74757,0.22629,0.50107,0.76160,0.02831
281,0.49229,0.30181,0.16614,0.49118,0.29099,-0.18309,0.49168,0.38829,0.16840,0.49374,...,0.16707,0.50949,0.62405,0.16707,0.50189,0.74760,0.24831,0.50109,0.76162,0.05127
282,0.49220,0.30142,0.16453,0.49124,0.29132,-0.18714,0.49160,0.38773,0.17067,0.49427,...,0.17120,0.50989,0.62438,0.17120,0.50196,0.74754,0.25175,0.50111,0.76155,0.05229
283,0.49216,0.30120,0.16506,0.49125,0.29137,-0.18876,0.49126,0.38729,0.17427,0.49401,...,0.17098,0.51005,0.62459,0.17098,0.50199,0.74751,0.24931,0.50113,0.76151,0.04818


In [56]:
sqaut_bad_hand0_3 = pd.read_csv('datacsv\\sqaut_bad_hand0_3.csv')
sqaut_bad_hand0_3

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.48468,0.41833,-0.25338,0.48901,0.41308,0.08991,0.41924,0.45977,-0.31361,0.43079,...,0.00594,0.53931,0.67848,0.00594,0.53619,0.77686,0.18404,0.53549,0.75222,0.43093
1,0.48427,0.42158,-0.25742,0.48979,0.41666,0.07759,0.41925,0.46014,-0.31988,0.43094,...,0.01243,0.53867,0.68112,0.01243,0.53586,0.77740,0.16100,0.53549,0.75359,0.43029
2,0.48323,0.42835,-0.26893,0.48972,0.42345,0.05758,0.41716,0.46256,-0.34973,0.43052,...,0.00054,0.53878,0.68352,0.00054,0.53566,0.77833,0.12885,0.53548,0.75417,0.41608
3,0.48301,0.43152,-0.27545,0.48916,0.42623,0.04467,0.41723,0.46404,-0.35182,0.42915,...,-0.00350,0.53919,0.68373,-0.00350,0.53583,0.77870,0.12027,0.53548,0.75398,0.41884
4,0.48332,0.44067,-0.30039,0.48485,0.43601,0.01021,0.41613,0.46765,-0.38551,0.42343,...,-0.00571,0.53904,0.68387,-0.00571,0.53613,0.77907,0.12909,0.53531,0.75261,0.40894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,0.53173,0.36563,-0.23482,0.52498,0.37003,0.05399,0.52866,0.45965,-0.25261,0.52091,...,-0.01790,0.52768,0.66224,-0.01790,0.53736,0.78490,0.06635,0.54419,0.76573,0.24236
259,0.53181,0.36547,-0.23776,0.52512,0.36982,0.05389,0.52869,0.45973,-0.25512,0.52099,...,-0.01786,0.52881,0.66166,-0.01786,0.53734,0.78488,0.06694,0.54408,0.76553,0.24348
260,0.53179,0.36433,-0.23765,0.52590,0.36873,0.05598,0.52865,0.45921,-0.25423,0.52147,...,-0.01711,0.52879,0.66110,-0.01711,0.53693,0.78397,0.06982,0.54406,0.76423,0.24563
261,0.53178,0.36443,-0.23767,0.52592,0.36875,0.05594,0.52854,0.45926,-0.25381,0.52149,...,-0.01708,0.52927,0.66117,-0.01708,0.53696,0.78414,0.06985,0.54374,0.76412,0.24597


In [57]:
sqaut_bad_hand25_3 = pd.read_csv('datacsv\\sqaut_bad_hand25_3.csv')
sqaut_bad_hand25_3

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.48379,0.31853,-0.28983,0.46695,0.31521,0.07193,0.43554,0.38897,-0.34209,0.42968,...,-0.04456,0.49630,0.64231,-0.04456,0.49903,0.76569,0.08304,0.48510,0.70547,0.51499
1,0.48379,0.31951,-0.29845,0.46876,0.31430,0.07707,0.43019,0.38914,-0.34349,0.43011,...,-0.03467,0.50153,0.64059,-0.03467,0.49957,0.76503,0.08296,0.48111,0.70524,0.55127
2,0.48065,0.32491,-0.30574,0.46998,0.31448,0.07685,0.41854,0.38938,-0.35261,0.43005,...,-0.02583,0.50799,0.64052,-0.02583,0.50006,0.76500,0.08101,0.47812,0.70558,0.57763
3,0.48004,0.32610,-0.30363,0.47057,0.31456,0.07663,0.41878,0.38941,-0.35153,0.43033,...,-0.03716,0.51068,0.63969,-0.03716,0.50025,0.76500,0.05087,0.47514,0.70568,0.57210
4,0.47945,0.33076,-0.31114,0.46859,0.31880,0.05648,0.42028,0.38941,-0.36448,0.42783,...,-0.02746,0.51392,0.64016,-0.02746,0.50044,0.76505,0.07564,0.47543,0.70607,0.59790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,0.50954,0.28818,-0.28988,0.46738,0.29600,0.03807,0.54990,0.36869,-0.30509,0.48745,...,-0.04766,0.50064,0.63141,-0.04766,0.50390,0.75619,0.05016,0.48616,0.71380,0.45097
338,0.50895,0.28722,-0.29462,0.46752,0.29486,0.04139,0.54986,0.36769,-0.30555,0.48797,...,-0.04609,0.49997,0.63137,-0.04609,0.50386,0.75652,0.04798,0.48605,0.71500,0.44749
339,0.50759,0.28686,-0.29620,0.46768,0.29221,0.03512,0.55118,0.36339,-0.30314,0.49167,...,-0.03100,0.49935,0.63052,-0.03100,0.50396,0.75650,0.07014,0.48595,0.71486,0.47943
340,0.50698,0.28978,-0.30818,0.47046,0.28876,0.01721,0.55137,0.36428,-0.30206,0.49352,...,-0.01511,0.49859,0.62978,-0.01511,0.50420,0.75622,0.09286,0.48572,0.71309,0.48778


In [58]:
sqaut_bad_hand45_3 = pd.read_csv('datacsv\\sqaut_bad_hand45_3.csv')
sqaut_bad_hand45_3

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.51126,0.30556,-0.20358,0.45348,0.32164,0.08629,0.47218,0.35868,-0.35632,0.39999,...,-0.19104,0.51523,0.63125,-0.19104,0.52386,0.75165,-0.07517,0.45888,0.70759,0.48515
1,0.51083,0.30830,-0.21851,0.45347,0.32540,0.08011,0.45637,0.35869,-0.35122,0.39925,...,-0.12915,0.51646,0.63124,-0.12915,0.52453,0.75165,0.00073,0.45887,0.70746,0.50110
2,0.50976,0.31600,-0.25530,0.45353,0.33129,0.04694,0.44626,0.36189,-0.36603,0.39918,...,-0.10861,0.51787,0.63128,-0.10861,0.52470,0.75179,0.01663,0.45884,0.70728,0.52210
3,0.50848,0.31873,-0.27465,0.45407,0.33258,0.03759,0.44048,0.36534,-0.37262,0.39933,...,-0.08776,0.51847,0.63134,-0.08776,0.52484,0.75197,0.03950,0.45886,0.70716,0.53164
4,0.50828,0.32385,-0.26999,0.45266,0.33976,0.02569,0.44267,0.36554,-0.37624,0.39600,...,-0.11502,0.51934,0.63137,-0.11502,0.52494,0.75106,0.00465,0.45871,0.70584,0.54001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,0.53131,0.30230,-0.17786,0.45982,0.31022,0.03595,0.52882,0.40522,-0.20059,0.45024,...,-0.07222,0.50777,0.63139,-0.07222,0.52280,0.75556,0.01016,0.45713,0.70729,0.37846
281,0.53132,0.30242,-0.17768,0.45982,0.31027,0.03532,0.52905,0.40524,-0.20042,0.45024,...,-0.07096,0.50772,0.63135,-0.07096,0.52280,0.75559,0.01412,0.45713,0.70725,0.37944
282,0.53132,0.30261,-0.17412,0.45983,0.31030,0.03617,0.52907,0.40523,-0.19689,0.45008,...,-0.07109,0.50763,0.63072,-0.07109,0.52278,0.75559,0.01390,0.45708,0.70727,0.36208
283,0.53133,0.30283,-0.17303,0.45984,0.31031,0.03636,0.52909,0.40523,-0.19510,0.45001,...,-0.07092,0.50756,0.63017,-0.07092,0.52276,0.75558,0.01400,0.45702,0.70733,0.35710


In [59]:
sqaut_bad_hand60_3 = pd.read_csv('datacsv\\sqaut_bad_hand60_3.csv')
sqaut_bad_hand60_3

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.53941,0.30407,-0.16345,0.45235,0.30745,-0.03270,0.54668,0.39892,-0.16348,0.42396,...,-0.02580,0.51789,0.63116,-0.02580,0.53335,0.75019,0.10720,0.44417,0.71818,0.30633
1,0.53941,0.30372,-0.15258,0.45231,0.30735,-0.05233,0.54667,0.39727,-0.14500,0.42261,...,-0.01172,0.51866,0.63375,-0.01172,0.53335,0.75040,0.18477,0.44413,0.71817,0.30267
2,0.53948,0.30370,-0.14728,0.45197,0.30742,-0.02496,0.54665,0.39634,-0.17529,0.42016,...,-0.00875,0.51894,0.63627,-0.00875,0.53336,0.75070,0.19291,0.44405,0.71800,0.32235
3,0.53950,0.30366,-0.14469,0.45176,0.30749,-0.01413,0.54660,0.39565,-0.18554,0.41889,...,-0.01125,0.51900,0.63810,-0.01125,0.53339,0.75101,0.18534,0.44387,0.71803,0.33200
4,0.53951,0.30359,-0.14720,0.45121,0.30762,-0.00514,0.54694,0.39570,-0.20550,0.41596,...,-0.01072,0.51899,0.63933,-0.01072,0.53338,0.75125,0.18335,0.44372,0.71811,0.34559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,0.53812,0.29811,-0.18304,0.44868,0.30130,-0.09012,0.54736,0.39922,-0.15219,0.43942,...,-0.01725,0.52106,0.63358,-0.01725,0.53541,0.75007,0.14390,0.44104,0.72078,0.36086
319,0.53831,0.29821,-0.18380,0.44888,0.30126,-0.07078,0.54801,0.39924,-0.15906,0.44028,...,-0.01703,0.52140,0.63345,-0.01703,0.53542,0.74995,0.14075,0.44105,0.72058,0.36190
320,0.53859,0.29839,-0.18141,0.44908,0.30145,-0.06592,0.54856,0.39938,-0.15527,0.44235,...,-0.01533,0.52150,0.63334,-0.01533,0.53542,0.74979,0.13849,0.44114,0.72030,0.35541
321,0.53861,0.29837,-0.17569,0.44945,0.30141,-0.05371,0.54862,0.39947,-0.15129,0.44411,...,-0.01582,0.52150,0.63322,-0.01582,0.53532,0.74979,0.13084,0.44128,0.72007,0.35433


In [60]:
sqaut_bad_hand90_3 = pd.read_csv('datacsv\\sqaut_bad_hand90_3.csv')
sqaut_bad_hand90_3

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.54575,0.30881,-0.11866,0.45203,0.31012,-0.17976,0.56350,0.40952,-0.04851,0.44116,...,0.09718,0.54373,0.62832,0.09718,0.56252,0.73041,0.33922,0.45572,0.74304,0.22384
1,0.54596,0.30865,-0.12582,0.45203,0.30917,-0.17233,0.56359,0.40957,-0.06003,0.44062,...,0.09601,0.54370,0.62709,0.09601,0.56244,0.73040,0.33183,0.45583,0.74216,0.23824
2,0.54629,0.30866,-0.13109,0.45205,0.30860,-0.17036,0.56385,0.40961,-0.06639,0.44023,...,0.09583,0.54365,0.62628,0.09583,0.56235,0.73045,0.32785,0.45585,0.74200,0.23623
3,0.54646,0.30866,-0.13184,0.45207,0.30833,-0.16890,0.56404,0.40963,-0.06740,0.44000,...,0.09481,0.54365,0.62586,0.09481,0.56233,0.73050,0.32412,0.45586,0.74190,0.23459
4,0.54654,0.30871,-0.13187,0.45200,0.30819,-0.16665,0.56456,0.40966,-0.07049,0.43938,...,0.08937,0.54364,0.62586,0.08937,0.56233,0.73072,0.32243,0.45563,0.74192,0.23482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,0.54086,0.30082,-0.12360,0.44890,0.30041,-0.13096,0.55381,0.39942,-0.06188,0.44602,...,0.07814,0.53869,0.62249,0.07814,0.56599,0.73339,0.27276,0.45584,0.74105,0.23815
329,0.54146,0.30014,-0.11419,0.44916,0.29903,-0.13118,0.55399,0.39529,-0.04407,0.44386,...,0.06675,0.53911,0.62230,0.06675,0.56600,0.73334,0.26208,0.45584,0.74102,0.22811
330,0.54232,0.29888,-0.10438,0.44937,0.29667,-0.12877,0.55606,0.39135,-0.03819,0.43962,...,0.06335,0.53940,0.62150,0.06335,0.56605,0.73330,0.26058,0.45581,0.74079,0.22798
331,0.54298,0.30013,-0.09812,0.44934,0.29809,-0.11743,0.56052,0.39191,-0.03491,0.43505,...,0.06118,0.53965,0.62191,0.06118,0.56585,0.73357,0.26829,0.45569,0.74139,0.23752


In [61]:
sqaut_bad_hand105_3 = pd.read_csv('datacsv\\sqaut_bad_hand105_3.csv')
sqaut_bad_hand105_3

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.54786,0.30965,0.01472,0.46204,0.30151,-0.13056,0.57652,0.39387,0.07008,0.45426,...,0.10643,0.54312,0.62129,0.10643,0.55594,0.71818,0.32314,0.46911,0.75530,0.12253
1,0.54787,0.30913,0.00972,0.46206,0.30109,-0.14161,0.57652,0.39242,0.07012,0.45402,...,0.14627,0.54306,0.62114,0.14627,0.55575,0.71855,0.35253,0.46861,0.75532,0.11013
2,0.54787,0.30820,0.00799,0.46206,0.30078,-0.14305,0.57722,0.38681,0.06982,0.45349,...,0.15737,0.54306,0.62113,0.15737,0.55577,0.71903,0.36769,0.46831,0.75531,0.11704
3,0.54788,0.30762,0.00653,0.46206,0.30052,-0.14337,0.57762,0.38464,0.06910,0.45309,...,0.15777,0.54306,0.62106,0.15777,0.55579,0.71930,0.36810,0.46811,0.75529,0.11938
4,0.54807,0.30746,-0.00020,0.46212,0.30004,-0.14349,0.58005,0.38296,0.05767,0.45300,...,0.15527,0.54336,0.62064,0.15527,0.55585,0.71952,0.37061,0.46794,0.75544,0.11943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,0.55123,0.30209,-0.01332,0.46499,0.29744,-0.15629,0.56259,0.39732,0.05403,0.45541,...,0.16547,0.54302,0.61628,0.16547,0.55892,0.71637,0.40960,0.46636,0.75333,0.11692
319,0.55132,0.30205,-0.01594,0.46481,0.29754,-0.15477,0.56169,0.39692,0.05044,0.45408,...,0.16026,0.54296,0.61533,0.16026,0.55892,0.71635,0.40911,0.46649,0.75295,0.13989
320,0.55129,0.30215,-0.01802,0.46458,0.29788,-0.15370,0.55972,0.39691,0.04819,0.45352,...,0.15908,0.54262,0.61436,0.15908,0.55891,0.71635,0.40419,0.46650,0.75271,0.14544
321,0.55133,0.30224,-0.01675,0.46447,0.29789,-0.15114,0.55764,0.39705,0.04952,0.45347,...,0.15528,0.54253,0.61370,0.15528,0.55904,0.71641,0.40339,0.46652,0.75227,0.15932


In [62]:
sqaut_bad_hand135_3 = pd.read_csv('datacsv\\sqaut_bad_hand135_3.csv')
sqaut_bad_hand135_3

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.54630,0.32416,0.05275,0.48900,0.30947,-0.24549,0.59781,0.37970,0.07353,0.54639,...,0.15913,0.53480,0.61911,0.15913,0.54382,0.70911,0.45702,0.47716,0.75442,0.02534
1,0.54887,0.32481,0.03688,0.48938,0.30983,-0.23618,0.59991,0.37923,0.02513,0.54441,...,0.17516,0.53502,0.61676,0.17516,0.54101,0.70875,0.44358,0.47724,0.75475,-0.06828
2,0.55021,0.32892,0.01671,0.49040,0.31623,-0.24289,0.60397,0.37916,0.00623,0.54568,...,0.20987,0.53564,0.61336,0.20987,0.54070,0.70844,0.47016,0.47743,0.75508,-0.14449
3,0.55108,0.33157,0.02199,0.49093,0.31838,-0.24229,0.60492,0.37917,0.01622,0.54649,...,0.22557,0.53585,0.61339,0.22557,0.54081,0.70783,0.50386,0.47743,0.75578,-0.18171
4,0.55196,0.33665,0.01735,0.49231,0.32504,-0.22756,0.60665,0.38068,-0.03270,0.54965,...,0.20333,0.53586,0.61372,0.20333,0.54084,0.70630,0.50126,0.47746,0.75598,-0.21453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,0.54114,0.31154,0.04394,0.46930,0.30227,-0.22749,0.55213,0.40656,0.11487,0.47361,...,0.24254,0.54437,0.61196,0.24254,0.54459,0.71118,0.44869,0.47905,0.76069,-0.00817
281,0.54092,0.31170,0.04629,0.46932,0.30241,-0.22742,0.55192,0.40676,0.11752,0.47365,...,0.23983,0.54444,0.61182,0.23983,0.54477,0.71120,0.44555,0.47903,0.76067,-0.00740
282,0.54087,0.31173,0.04608,0.46937,0.30249,-0.22721,0.55191,0.40693,0.11734,0.47376,...,0.23928,0.54450,0.61184,0.23928,0.54487,0.71123,0.44593,0.47902,0.76067,-0.00689
283,0.54080,0.31177,0.04609,0.46941,0.30256,-0.22716,0.55190,0.40709,0.11774,0.47384,...,0.23708,0.54459,0.61190,0.23708,0.54491,0.71133,0.44272,0.47902,0.76061,-0.00515


In [63]:
sqaut_bad_hand155_3 = pd.read_csv('datacsv\\sqaut_bad_hand155_3.csv')
sqaut_bad_hand155_3

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.53449,0.30906,0.07394,0.52100,0.31289,-0.28064,0.56583,0.38632,0.14950,0.57105,...,0.24278,0.53001,0.61491,0.24278,0.51734,0.72253,0.42842,0.49910,0.77053,0.12284
1,0.53405,0.30945,0.07232,0.52104,0.31492,-0.28342,0.55887,0.39003,0.15083,0.57388,...,0.27094,0.52342,0.61651,0.27094,0.51708,0.72263,0.46402,0.49863,0.76881,0.19318
2,0.53425,0.31343,0.07534,0.52253,0.32267,-0.27766,0.55916,0.38912,0.15148,0.57958,...,0.27140,0.52415,0.61989,0.27140,0.51712,0.72286,0.45982,0.49837,0.76861,0.18151
3,0.53433,0.31455,0.07196,0.52336,0.32584,-0.27912,0.55941,0.38964,0.15189,0.58123,...,0.28218,0.52462,0.61997,0.28218,0.51826,0.72170,0.47821,0.49780,0.76760,0.18784
4,0.53709,0.31882,0.06838,0.52441,0.33084,-0.28157,0.55976,0.39441,0.16014,0.58277,...,0.27835,0.52423,0.61844,0.27835,0.51885,0.71860,0.47500,0.49748,0.76647,0.16958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,0.53786,0.29921,0.05967,0.49682,0.29069,-0.28505,0.51811,0.38060,0.11718,0.46043,...,0.27986,0.52813,0.60869,0.27986,0.51940,0.71088,0.47204,0.49369,0.75638,0.09951
338,0.53664,0.29932,0.06358,0.49693,0.29328,-0.28194,0.51622,0.38025,0.11817,0.46188,...,0.28275,0.52816,0.61068,0.28275,0.51920,0.71090,0.47254,0.49343,0.75667,0.16311
339,0.53326,0.29297,0.06908,0.49809,0.29170,-0.27965,0.50999,0.37182,0.12299,0.45948,...,0.28922,0.52823,0.61169,0.28922,0.51925,0.71152,0.47688,0.49317,0.75673,0.11473
340,0.52842,0.28712,0.07733,0.49874,0.29153,-0.28502,0.50493,0.36669,0.13848,0.46194,...,0.31035,0.52853,0.61219,0.31035,0.51934,0.71138,0.50980,0.49315,0.75682,0.10220


In [64]:
sqaut_bad_hand180_3 = pd.read_csv('datacsv\\sqaut_bad_hand180_3.csv')
sqaut_bad_hand180_3

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.51431,0.41589,0.04237,0.51341,0.42274,-0.28334,0.56897,0.45134,0.10828,0.56936,...,0.21610,0.46411,0.65919,0.21610,0.46393,0.75879,0.35650,0.46099,0.78618,0.13398
1,0.51483,0.41795,0.03702,0.51373,0.42475,-0.29001,0.56997,0.45205,0.09527,0.57292,...,0.24179,0.46440,0.65486,0.24179,0.46433,0.74852,0.41098,0.46103,0.78494,0.08619
2,0.51483,0.42438,0.02552,0.51583,0.43293,-0.30191,0.57456,0.45741,0.08037,0.57685,...,0.26730,0.46474,0.65024,0.26730,0.46446,0.73989,0.45944,0.46100,0.78248,0.10527
3,0.51545,0.42738,0.03768,0.51630,0.43512,-0.29588,0.57571,0.45900,0.08106,0.57678,...,0.25787,0.46470,0.64849,0.25787,0.46441,0.73994,0.43169,0.46100,0.78190,0.00014
4,0.51616,0.43388,0.03677,0.51770,0.44231,-0.29515,0.57802,0.46343,0.08381,0.58083,...,0.19626,0.46494,0.64933,0.19626,0.46387,0.74237,0.32645,0.46101,0.78192,-0.06958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,0.47257,0.36845,0.11056,0.47186,0.36841,-0.21236,0.46611,0.45939,0.11661,0.47333,...,0.16197,0.47277,0.65535,0.16197,0.45755,0.76546,0.25824,0.45859,0.78337,0.06042
259,0.47270,0.36848,0.11002,0.47172,0.36843,-0.21133,0.46649,0.45908,0.11608,0.47226,...,0.16225,0.47279,0.65531,0.16225,0.45768,0.76540,0.25855,0.45866,0.78305,0.06200
260,0.47280,0.36745,0.11078,0.47128,0.36734,-0.21168,0.46636,0.45842,0.11651,0.47176,...,0.16178,0.47281,0.65395,0.16178,0.45769,0.76511,0.25744,0.45867,0.78224,0.06203
261,0.47271,0.36753,0.11033,0.47127,0.36741,-0.20937,0.46641,0.45838,0.11540,0.47144,...,0.16117,0.47300,0.65447,0.16117,0.45771,0.76516,0.25645,0.45873,0.78227,0.06220


In [65]:
sqaut_bad_0_3 = pd.read_csv('datacsv\\sqaut_bad_0_3.csv')
sqaut_bad_0_3

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.50230,0.34957,-0.19627,0.49521,0.35095,0.11890,0.48627,0.39717,-0.26546,0.48566,...,-0.08202,0.54431,0.64402,-0.08202,0.52834,0.75622,0.02070,0.52636,0.72521,0.26569
1,0.50061,0.35285,-0.20056,0.49621,0.35415,0.11981,0.48423,0.40927,-0.22379,0.49204,...,-0.08373,0.54190,0.64321,-0.08373,0.52798,0.75534,0.03636,0.52712,0.72702,0.25094
2,0.50069,0.36915,-0.24204,0.49581,0.36336,0.07000,0.48904,0.42073,-0.33295,0.49470,...,-0.08322,0.53895,0.64332,-0.08322,0.52763,0.75372,0.05382,0.52741,0.72758,0.22471
3,0.50048,0.37052,-0.22713,0.49574,0.36720,0.08654,0.49082,0.41602,-0.33263,0.49544,...,-0.09082,0.53705,0.64378,-0.09082,0.52766,0.75347,0.04927,0.52887,0.72870,0.20815
4,0.49890,0.38406,-0.23024,0.49554,0.37153,0.08858,0.49055,0.45127,-0.25706,0.49553,...,-0.09115,0.53492,0.64386,-0.09115,0.52769,0.75350,0.04579,0.54060,0.72931,0.26124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,0.53125,0.29828,-0.22022,0.53683,0.30307,0.11420,0.50002,0.38060,-0.19590,0.52449,...,-0.02391,0.52356,0.62850,-0.02391,0.52784,0.75199,0.04719,0.52511,0.72324,0.32901
261,0.53110,0.29832,-0.21962,0.53662,0.30321,0.11319,0.49967,0.38066,-0.19645,0.52370,...,-0.02706,0.52356,0.62808,-0.02706,0.52781,0.75168,0.04029,0.52467,0.72312,0.32711
262,0.53075,0.29820,-0.21932,0.53652,0.30298,0.11308,0.49980,0.38089,-0.19899,0.52355,...,-0.02634,0.52359,0.62799,-0.02634,0.52771,0.75154,0.04358,0.52455,0.72317,0.31113
263,0.53051,0.29814,-0.21933,0.53641,0.30294,0.11269,0.49999,0.38085,-0.20885,0.52358,...,-0.02618,0.52361,0.62797,-0.02618,0.52763,0.75145,0.04444,0.52448,0.72323,0.30759


In [66]:
sqaut_bad_25_3 = pd.read_csv('datacsv\\sqaut_bad_25_3.csv')
sqaut_bad_25_3

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.55025,0.31274,-0.16175,0.49931,0.32566,0.06976,0.51412,0.39818,-0.11583,0.47491,...,-0.11764,0.55215,0.62463,-0.11764,0.56438,0.74569,-0.09804,0.50326,0.69294,0.49647
1,0.55126,0.31526,-0.16252,0.49820,0.32859,0.07198,0.51978,0.39862,-0.14228,0.47454,...,-0.10477,0.55594,0.62457,-0.10477,0.56609,0.74570,-0.07277,0.50593,0.69301,0.47569
2,0.55126,0.32371,-0.15897,0.49614,0.33604,0.07465,0.51861,0.39892,-0.14077,0.47301,...,-0.13110,0.55746,0.62409,-0.13110,0.56704,0.74573,-0.11450,0.50748,0.69283,0.51421
3,0.55128,0.32610,-0.16250,0.49567,0.33758,0.07446,0.51841,0.40152,-0.14359,0.47210,...,-0.11894,0.55817,0.62385,-0.11894,0.56741,0.74564,-0.09863,0.50755,0.69277,0.51797
4,0.55024,0.33103,-0.16091,0.49392,0.34446,0.07374,0.51653,0.40318,-0.14164,0.46978,...,-0.11882,0.56036,0.62386,-0.11882,0.56769,0.74535,-0.09588,0.50751,0.69275,0.51318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,0.57363,0.31260,-0.14056,0.51146,0.31600,0.06948,0.55836,0.40531,-0.12288,0.49067,...,-0.07415,0.54729,0.61330,-0.07415,0.56606,0.73997,-0.01089,0.50506,0.69503,0.35864
259,0.57346,0.31214,-0.14180,0.51146,0.31575,0.06941,0.55625,0.40055,-0.11936,0.49068,...,-0.07371,0.54702,0.61280,-0.07371,0.56597,0.73980,-0.00996,0.50463,0.69475,0.35762
260,0.57337,0.31194,-0.14192,0.51146,0.31549,0.07148,0.55630,0.39922,-0.11838,0.49084,...,-0.07602,0.54660,0.61232,-0.07602,0.56585,0.73972,-0.01576,0.50434,0.69452,0.35609
261,0.57359,0.31281,-0.14923,0.51247,0.31570,0.07139,0.55342,0.40131,-0.11932,0.49141,...,-0.07620,0.54700,0.61546,-0.07620,0.56603,0.74209,-0.01699,0.50459,0.69592,0.36960


In [67]:
sqaut_bad_45_3 = pd.read_csv('datacsv\\sqaut_bad_45_3.csv')
sqaut_bad_45_3

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.54228,0.40140,-0.25255,0.47538,0.42059,-0.09226,0.53718,0.46832,-0.22234,0.46518,...,-0.17496,0.56280,0.64103,-0.17496,0.56894,0.74322,-0.07072,0.48611,0.69894,0.22968
1,0.54223,0.41147,-0.28774,0.47543,0.42688,-0.14520,0.53827,0.48614,-0.37886,0.46519,...,-0.20848,0.56403,0.63969,-0.20848,0.56920,0.74153,-0.05839,0.48598,0.69884,0.15014
2,0.54223,0.42569,-0.30970,0.47547,0.43705,-0.15448,0.53818,0.50031,-0.35847,0.46583,...,-0.21183,0.56488,0.63975,-0.21183,0.56924,0.74193,-0.04977,0.48574,0.69879,0.09880
3,0.54219,0.42871,-0.31089,0.47555,0.43920,-0.15736,0.53844,0.50118,-0.36381,0.46583,...,-0.20994,0.56542,0.63977,-0.20994,0.57332,0.74174,-0.05443,0.48567,0.69792,0.14925
4,0.54219,0.44053,-0.29537,0.47555,0.44994,-0.15555,0.53876,0.51319,-0.27447,0.46599,...,-0.23710,0.56561,0.63814,-0.23710,0.57043,0.74390,-0.12082,0.48543,0.69734,0.03421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,0.57852,0.31291,-0.10961,0.50411,0.31255,0.01557,0.56014,0.39146,-0.13404,0.48592,...,-0.02122,0.55573,0.61882,-0.02122,0.56880,0.73922,0.08856,0.48815,0.69719,0.32021
218,0.57857,0.31299,-0.10853,0.50429,0.31239,0.01725,0.56033,0.39127,-0.13489,0.48570,...,-0.01759,0.55580,0.61867,-0.01759,0.56877,0.73932,0.09178,0.48848,0.69725,0.31421
219,0.57854,0.31285,-0.10790,0.50423,0.31245,0.01638,0.56038,0.38937,-0.13599,0.48548,...,-0.01674,0.55577,0.61814,-0.01674,0.56878,0.73930,0.09813,0.48865,0.69726,0.30373
220,0.57847,0.31195,-0.10848,0.50427,0.31237,0.02177,0.56076,0.38486,-0.13695,0.48546,...,-0.01886,0.55540,0.61793,-0.01886,0.56878,0.73929,0.08758,0.48889,0.69723,0.31085


In [68]:
sqaut_bad_60_3 = pd.read_csv('datacsv\\sqaut_bad_60_3.csv')
sqaut_bad_60_3

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.57294,0.30941,-0.08432,0.49066,0.31559,-0.03783,0.56428,0.39032,-0.15006,0.47744,...,-0.02657,0.55680,0.63466,-0.02657,0.57143,0.74495,0.12028,0.47416,0.70965,0.33541
1,0.57305,0.30969,-0.11303,0.49103,0.31421,-0.04789,0.56350,0.39272,-0.17318,0.47711,...,0.00024,0.55712,0.63280,0.00024,0.57192,0.74277,0.16850,0.47443,0.70946,0.35547
2,0.57307,0.30974,-0.11930,0.49107,0.31309,-0.05434,0.56315,0.39344,-0.18114,0.47710,...,0.00387,0.55742,0.63185,0.00387,0.57230,0.74166,0.17501,0.47473,0.70901,0.36563
3,0.57307,0.30979,-0.11879,0.49109,0.31237,-0.05466,0.56285,0.39400,-0.17933,0.47706,...,0.00415,0.55754,0.63119,0.00415,0.57251,0.74101,0.17433,0.47491,0.70881,0.36483
4,0.57307,0.30943,-0.11861,0.49125,0.31148,-0.05231,0.56245,0.39393,-0.17896,0.47702,...,0.00168,0.55755,0.63078,0.00168,0.57275,0.74078,0.16732,0.47523,0.70864,0.36234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,0.56290,0.31015,-0.11023,0.48174,0.31551,-0.05438,0.55434,0.39550,-0.17630,0.47482,...,-0.00840,0.55899,0.62377,-0.00840,0.57148,0.73670,0.16340,0.47285,0.70855,0.38726
251,0.56041,0.31078,-0.11052,0.47863,0.31593,-0.05146,0.55149,0.39687,-0.17144,0.47301,...,-0.00224,0.55710,0.62348,-0.00224,0.57066,0.73653,0.16770,0.47285,0.70738,0.39566
252,0.55819,0.31102,-0.09447,0.47692,0.31631,-0.03866,0.55200,0.39264,-0.16304,0.47203,...,-0.00766,0.55464,0.62276,-0.00766,0.57053,0.73656,0.13129,0.47297,0.70706,0.36868
253,0.55679,0.31074,-0.09297,0.47610,0.31654,-0.03623,0.55228,0.39026,-0.16589,0.47081,...,-0.01466,0.55401,0.62083,-0.01466,0.57044,0.73652,0.11607,0.47324,0.70666,0.36032


In [69]:
sqaut_bad_90_3 = pd.read_csv('datacsv\\sqaut_bad_90_3.csv')
sqaut_bad_90_3

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.52669,0.31255,-0.05539,0.44091,0.30976,-0.05768,0.51949,0.40086,-0.16689,0.44399,...,0.05288,0.52541,0.62869,0.05288,0.55009,0.72271,0.27626,0.43755,0.73263,0.20128
1,0.52675,0.31287,-0.05532,0.44093,0.30953,-0.05891,0.51949,0.40010,-0.16696,0.44344,...,0.06159,0.52569,0.62471,0.06159,0.54793,0.72231,0.28864,0.43729,0.73253,0.21358
2,0.52675,0.31282,-0.06420,0.44093,0.30920,-0.06660,0.51945,0.40104,-0.16961,0.44309,...,0.06707,0.52603,0.62285,0.06707,0.54699,0.72198,0.30166,0.43712,0.73252,0.22049
3,0.52676,0.31278,-0.06734,0.44094,0.30896,-0.06872,0.51940,0.40187,-0.16996,0.44293,...,0.06903,0.52625,0.62221,0.06903,0.54653,0.72183,0.30450,0.43704,0.73247,0.22491
4,0.52685,0.31279,-0.06762,0.44098,0.30853,-0.06737,0.51895,0.40580,-0.16560,0.44245,...,0.07167,0.52631,0.62216,0.07167,0.54634,0.72161,0.31094,0.43701,0.73240,0.23368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,0.52640,0.30708,-0.05894,0.44124,0.30548,-0.06457,0.52611,0.39522,-0.16428,0.44677,...,0.03973,0.52824,0.62316,0.03973,0.54999,0.72612,0.28484,0.43792,0.73271,0.23692
282,0.52638,0.30709,-0.05917,0.44124,0.30549,-0.06640,0.52613,0.39511,-0.16465,0.44673,...,0.03989,0.52824,0.62314,0.03989,0.54990,0.72611,0.28574,0.43788,0.73284,0.23648
283,0.52638,0.30712,-0.05971,0.44123,0.30549,-0.06761,0.52612,0.39510,-0.16522,0.44670,...,0.04084,0.52824,0.62316,0.04084,0.54984,0.72611,0.28828,0.43783,0.73296,0.23630
284,0.52706,0.30713,-0.06036,0.44127,0.30548,-0.06683,0.52614,0.39584,-0.16519,0.44674,...,0.04419,0.52830,0.62315,0.04419,0.54986,0.72589,0.29209,0.43786,0.73296,0.23622


In [70]:
sqaut_bad_105_3 = pd.read_csv('datacsv\\sqaut_bad_105_3.csv')
sqaut_bad_105_3

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.50844,0.30964,-0.00887,0.42888,0.30769,-0.07209,0.52392,0.38882,-0.09763,0.43944,...,0.13324,0.50972,0.60875,0.13324,0.52530,0.70628,0.35835,0.42764,0.74315,0.14796
1,0.50833,0.30980,-0.00569,0.42882,0.30770,-0.08735,0.52391,0.38929,-0.09123,0.44100,...,0.10981,0.50920,0.60854,0.10981,0.52538,0.70691,0.33123,0.42764,0.74314,0.17522
2,0.50831,0.30980,0.00095,0.42876,0.30767,-0.08542,0.52382,0.38976,-0.08058,0.44194,...,0.10298,0.50898,0.60800,0.10298,0.52557,0.70717,0.32357,0.42764,0.74285,0.18062
3,0.50829,0.30980,0.00444,0.42870,0.30764,-0.08462,0.52375,0.39001,-0.07674,0.44232,...,0.10496,0.50880,0.60731,0.10496,0.52572,0.70723,0.32467,0.42764,0.74279,0.17635
4,0.50819,0.30963,0.00405,0.42858,0.30730,-0.09336,0.52357,0.39004,-0.08066,0.44341,...,0.10898,0.50876,0.60685,0.10898,0.52584,0.70727,0.32912,0.42756,0.74279,0.17438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,0.51883,0.31664,-0.00267,0.43702,0.30665,-0.06007,0.52736,0.39583,-0.09482,0.44598,...,0.07636,0.51168,0.61343,0.07636,0.52815,0.70746,0.33740,0.43019,0.74037,0.18745
251,0.52181,0.31688,0.00730,0.43992,0.30684,-0.04762,0.53020,0.39622,-0.07915,0.44888,...,0.08024,0.51382,0.61215,0.08024,0.52852,0.70662,0.33556,0.43040,0.73849,0.18388
252,0.52349,0.31740,0.01215,0.44206,0.30749,-0.04748,0.53206,0.39681,-0.08194,0.45078,...,0.07942,0.51523,0.61227,0.07942,0.52866,0.70654,0.33441,0.43040,0.73847,0.18202
253,0.52400,0.31758,0.00277,0.44242,0.30795,-0.05997,0.53279,0.39687,-0.09117,0.45063,...,0.09653,0.51560,0.61236,0.09653,0.52903,0.70627,0.34612,0.43040,0.73837,0.18058


In [71]:
sqaut_bad_135_3 = pd.read_csv('datacsv\\sqaut_bad_135_3.csv')
sqaut_bad_135_3

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.52357,0.41829,-0.11996,0.45357,0.40421,-0.29232,0.53206,0.48243,-0.22982,0.46506,...,-0.09213,0.50100,0.61595,-0.09213,0.51290,0.69863,0.17832,0.42918,0.73238,-0.02914
1,0.52427,0.42708,-0.18837,0.45385,0.41250,-0.29089,0.53255,0.50241,-0.29434,0.46330,...,-0.09402,0.50399,0.61629,-0.09402,0.51192,0.69431,0.17797,0.42919,0.73572,-0.01959
2,0.52428,0.43739,-0.17791,0.45519,0.42936,-0.30586,0.53344,0.51018,-0.26837,0.46320,...,-0.08854,0.50876,0.61692,-0.08854,0.51190,0.69461,0.17310,0.43012,0.73765,-0.05239
3,0.52459,0.44186,-0.20352,0.45552,0.43100,-0.31759,0.53391,0.51524,-0.29466,0.45701,...,-0.04644,0.51068,0.61730,-0.04644,0.51194,0.69426,0.23331,0.43090,0.73977,-0.03552
4,0.52552,0.45498,-0.19002,0.45563,0.44277,-0.27281,0.53497,0.52335,-0.23819,0.44929,...,-0.11207,0.51250,0.61719,-0.11207,0.51234,0.69604,0.09672,0.43221,0.74163,-0.10096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,0.49574,0.31230,0.06580,0.42279,0.30896,-0.07942,0.51546,0.39320,-0.02162,0.44077,...,0.10726,0.49602,0.59904,0.10726,0.51447,0.70072,0.29260,0.43216,0.74379,0.10885
218,0.49546,0.31262,0.06550,0.42285,0.30882,-0.08050,0.51578,0.39305,-0.02203,0.44107,...,0.10582,0.49533,0.59897,0.10582,0.51446,0.70079,0.28970,0.43195,0.74510,0.10911
219,0.49532,0.31278,0.06671,0.42290,0.30890,-0.08150,0.51583,0.39319,-0.01995,0.44176,...,0.10874,0.49522,0.59896,0.10874,0.51441,0.70075,0.29445,0.43194,0.74547,0.11195
220,0.49481,0.31257,0.07739,0.42294,0.30816,-0.08420,0.51534,0.39331,-0.00981,0.44294,...,0.11951,0.49530,0.59875,0.11951,0.51435,0.70070,0.29639,0.43177,0.74557,0.09497


In [72]:
sqaut_bad_155_3 = pd.read_csv('datacsv\\sqaut_bad_155_3.csv')
sqaut_bad_155_3

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.50165,0.32698,0.06910,0.45268,0.31189,-0.16112,0.52625,0.40291,-0.01014,0.46856,...,0.20596,0.48934,0.60220,0.20596,0.48892,0.69558,0.39529,0.43396,0.74392,-0.03565
1,0.50194,0.32866,0.07178,0.45358,0.31484,-0.17738,0.52650,0.40457,-0.01514,0.47260,...,0.24013,0.48888,0.60285,0.24013,0.48863,0.69630,0.42391,0.43376,0.74592,-0.04786
2,0.50280,0.33471,0.07175,0.45594,0.32110,-0.18135,0.52648,0.40768,-0.01920,0.48508,...,0.22077,0.48887,0.60311,0.22077,0.48845,0.69624,0.40624,0.43376,0.74720,-0.04753
3,0.50366,0.33592,0.07148,0.45735,0.32209,-0.17794,0.52651,0.40940,-0.02397,0.48489,...,0.20562,0.48881,0.60311,0.20562,0.48845,0.69583,0.38490,0.43375,0.74745,-0.05056
4,0.50462,0.34166,0.07158,0.45853,0.32857,-0.17150,0.52652,0.41167,-0.02330,0.49092,...,0.10982,0.48884,0.60168,0.10982,0.48829,0.69381,0.28381,0.43362,0.74736,-0.04927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,0.48173,0.31284,0.09312,0.43224,0.30954,-0.15098,0.50493,0.39443,0.00180,0.46325,...,0.22202,0.48545,0.60274,0.22202,0.49613,0.69940,0.37656,0.43319,0.74804,0.01469
259,0.48036,0.31160,0.09496,0.43303,0.30947,-0.15146,0.50367,0.39439,0.00582,0.46530,...,0.22248,0.48557,0.60276,0.22248,0.49637,0.69947,0.37640,0.43334,0.74797,0.01436
260,0.47945,0.31075,0.09592,0.43372,0.30926,-0.15152,0.50292,0.39425,0.00842,0.46611,...,0.22437,0.48573,0.60276,0.22437,0.49661,0.69947,0.37907,0.43351,0.74797,0.01072
261,0.47950,0.31216,0.09176,0.43320,0.31009,-0.15042,0.50308,0.39489,0.00733,0.46608,...,0.22665,0.48561,0.60405,0.22665,0.49656,0.69976,0.38560,0.43355,0.74814,0.01559


In [73]:
sqaut_bad_180_3 = pd.read_csv('datacsv\\sqaut_bad_180_3.csv')
sqaut_bad_180_3

,LS_x,LS_y,LS_z,RS_x,RS_y,RS_z,LFA_x,LFA_y,LFA_z,RFA_x,...,LL_z,RL_x,RL_y,RL_z,LF_x,LF_y,LF_z,RF_x,RF_y,RF_z
0,0.50699,0.35915,0.14718,0.50069,0.34950,-0.17041,0.51488,0.42269,0.11583,0.52143,...,0.18433,0.46663,0.60847,0.18433,0.46848,0.71859,0.27933,0.47521,0.74668,-0.04214
1,0.50760,0.36047,0.14198,0.50222,0.35865,-0.18382,0.51591,0.41630,0.20479,0.52433,...,0.20706,0.46433,0.61489,0.20706,0.46873,0.72477,0.33903,0.47378,0.74884,0.02409
2,0.50774,0.36731,0.13906,0.50330,0.36931,-0.18263,0.51630,0.42287,0.21213,0.52658,...,0.21232,0.46424,0.61935,0.21232,0.46948,0.72874,0.34379,0.47297,0.74889,0.03327
3,0.50804,0.37005,0.13976,0.50369,0.37273,-0.18756,0.51711,0.42327,0.23118,0.52735,...,0.21975,0.46393,0.61996,0.21975,0.46949,0.72925,0.34728,0.47170,0.75002,0.04310
4,0.50990,0.37403,0.14151,0.50576,0.37764,-0.17204,0.51758,0.43506,0.22265,0.52746,...,0.21985,0.46453,0.61902,0.21985,0.47012,0.72849,0.34694,0.47096,0.75014,-0.03132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,0.47023,0.30962,0.16249,0.46680,0.29804,-0.18879,0.46310,0.38024,0.24673,0.46386,...,0.19939,0.48547,0.61606,0.19939,0.47975,0.72729,0.30912,0.46726,0.75141,0.03446
261,0.47068,0.30966,0.16253,0.46710,0.29793,-0.18887,0.46373,0.38024,0.24655,0.46404,...,0.20047,0.48551,0.61564,0.20047,0.47975,0.72692,0.30987,0.46723,0.75133,0.03472
262,0.47110,0.30967,0.16471,0.46771,0.29757,-0.18715,0.46532,0.38015,0.25319,0.46417,...,0.19861,0.48545,0.61548,0.19861,0.47941,0.72637,0.30457,0.46725,0.75107,0.03406
263,0.47119,0.30966,0.16610,0.46808,0.29735,-0.18658,0.46582,0.38016,0.25247,0.46493,...,0.19797,0.48540,0.61541,0.19797,0.47922,0.72604,0.30294,0.46727,0.75092,0.03358


In [74]:
# 컬럼의 이름 확인, z 값은 불필요하므로 ...
sqaut_bad_180_3.columns

Index(['LS_x', 'LS_y', 'LS_z', 'RS_x', 'RS_y', 'RS_z', 'LFA_x', 'LFA_y',
       'LFA_z', 'RFA_x', 'RFA_y', 'RFA_z', 'LH_x', 'LH_y', 'LH_z', 'RH_x',
       'RH_y', 'RH_z', 'LUL_x', 'LUL_y', 'LUL_z', 'RUL_x', 'RUL_y', 'RUL_z',
       'LL_x', 'LL_y', 'LL_z', 'RL_x', 'RL_y', 'RL_z', 'LF_x', 'LF_y', 'LF_z',
       'RF_x', 'RF_y', 'RF_z'],
      dtype='object')

In [75]:
# 이름에 '_z' 가 포함된 컬럼은 삭제한다
columns_to_drop = [col for col in sqaut_bad_180_3 if '_z' in col]
len(columns_to_drop)  # 12

12

In [76]:
# 위에서 구한 '_z'가 포함된 컬럼명을 사용하여 DF에서 해당 컬럼 삭제
sqaut_hand0_0.drop(columns_to_drop, axis=1, inplace=True)
sqaut_hand25_0.drop(columns_to_drop, axis=1, inplace=True)
sqaut_hand45_0.drop(columns_to_drop, axis=1, inplace=True)
sqaut_hand60_0.drop(columns_to_drop, axis=1, inplace=True)
sqaut_hand90_0.drop(columns_to_drop, axis=1, inplace=True)
sqaut_hand105_0.drop(columns_to_drop, axis=1, inplace=True)
sqaut_hand135_0.drop(columns_to_drop, axis=1, inplace=True)
sqaut_hand155_0.drop(columns_to_drop, axis=1, inplace=True)
sqaut_hand180_0.drop(columns_to_drop, axis=1, inplace=True)
sqaut_0_0.drop(columns_to_drop, axis=1, inplace=True)
sqaut_25_0.drop(columns_to_drop, axis=1, inplace=True)
sqaut_45_0.drop(columns_to_drop, axis=1, inplace=True)
sqaut_60_0.drop(columns_to_drop, axis=1, inplace=True)
sqaut_90_0.drop(columns_to_drop, axis=1, inplace=True)
sqaut_105_0.drop(columns_to_drop, axis=1, inplace=True)
sqaut_135_0.drop(columns_to_drop, axis=1, inplace=True)
sqaut_155_0.drop(columns_to_drop, axis=1, inplace=True)
sqaut_180_0.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand0_1.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand25_1.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand45_1.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand60_1.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand90_1.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand105_1.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand135_1.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand155_1.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand180_1.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_0_1.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_25_1.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_45_1.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_60_1.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_90_1.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_105_1.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_135_1.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_155_1.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_180_1	.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand0_2.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand25_2.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand45_2.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand60_2.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand90_2.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand105_2.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand135_2.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand155_2.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand180_2.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_0_2.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_25_2.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_45_2.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_60_2	.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_90_2.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_105_2.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_135_2.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_155_2.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_180_2.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand0_3.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand25_3.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand45_3.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand60_3.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand90_3.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand105_3.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand135_3.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand155_3.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_hand180_3.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_0_3.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_25_3.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_45_3.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_60_3.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_90_3.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_105_3.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_135_3.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_155_3.drop(columns_to_drop, axis=1, inplace=True)
sqaut_bad_180_3.drop(columns_to_drop, axis=1, inplace=True)

len(sqaut_bad_155_3.columns), len(sqaut_bad_180_3.columns)

(24, 24)

In [77]:
# Sliding Window 방식으로 Sequence 생성
arr_sqaut_hand0_0 = sqaut_hand0_0.values
arr_sqaut_hand25_0 = sqaut_hand25_0.values
arr_sqaut_hand45_0 = sqaut_hand45_0.values
arr_sqaut_hand60_0 = sqaut_hand60_0.values
arr_sqaut_hand90_0 = sqaut_hand90_0.values
arr_sqaut_hand105_0 = sqaut_hand105_0.values
arr_sqaut_hand135_0 = sqaut_hand135_0.values
arr_sqaut_hand155_0 = sqaut_hand155_0.values
arr_sqaut_hand180_0 = sqaut_hand180_0.values
arr_sqaut_0_0 = sqaut_0_0.values
arr_sqaut_25_0 = sqaut_25_0.values
arr_sqaut_45_0 = sqaut_45_0.values
arr_sqaut_60_0 = sqaut_60_0.values
arr_sqaut_90_0 = sqaut_90_0.values
arr_sqaut_105_0 = sqaut_105_0.values
arr_sqaut_135_0 = sqaut_135_0.values
arr_sqaut_155_0 = sqaut_155_0.values
arr_sqaut_180_0 = sqaut_180_0.values
arr_sqaut_bad_hand0_1 = sqaut_bad_hand0_1.values
arr_sqaut_bad_hand25_1 = sqaut_bad_hand25_1.values
arr_sqaut_bad_hand45_1 = sqaut_bad_hand45_1.values
arr_sqaut_bad_hand60_1 = sqaut_bad_hand60_1.values
arr_sqaut_bad_hand90_1 = sqaut_bad_hand90_1.values
arr_sqaut_bad_hand105_1 = sqaut_bad_hand105_1.values
arr_sqaut_bad_hand135_1 = sqaut_bad_hand135_1.values
arr_sqaut_bad_hand155_1 = sqaut_bad_hand155_1.values
arr_sqaut_bad_hand180_1 = sqaut_bad_hand180_1.values
arr_sqaut_bad_0_1 = sqaut_bad_0_1.values
arr_sqaut_bad_25_1 = sqaut_bad_25_1.values
arr_sqaut_bad_45_1 = sqaut_bad_45_1.values
arr_sqaut_bad_60_1 = sqaut_bad_60_1.values
arr_sqaut_bad_90_1 = sqaut_bad_90_1.values
arr_sqaut_bad_105_1 = sqaut_bad_105_1.values
arr_sqaut_bad_135_1 = sqaut_bad_135_1.values
arr_sqaut_bad_155_1 = sqaut_bad_155_1.values
arr_sqaut_bad_180_1 = sqaut_bad_180_1.values
arr_sqaut_bad_hand0_2 = sqaut_bad_hand0_2.values
arr_sqaut_bad_hand25_2 = sqaut_bad_hand25_2.values
arr_sqaut_bad_hand45_2 = sqaut_bad_hand45_2.values
arr_sqaut_bad_hand60_2 = sqaut_bad_hand60_2.values
arr_sqaut_bad_hand90_2 = sqaut_bad_hand90_2.values
arr_sqaut_bad_hand105_2 = sqaut_bad_hand105_2.values
arr_sqaut_bad_hand135_2 = sqaut_bad_hand135_2.values
arr_sqaut_bad_hand155_2 = sqaut_bad_hand155_2.values
arr_sqaut_bad_hand180_2 = sqaut_bad_hand180_2.values
arr_sqaut_bad_0_2 = sqaut_bad_0_2.values
arr_sqaut_bad_25_2 = sqaut_bad_25_2.values
arr_sqaut_bad_45_2 = sqaut_bad_45_2.values
arr_sqaut_bad_60_2 = sqaut_bad_60_2.values
arr_sqaut_bad_90_2 = sqaut_bad_90_2.values
arr_sqaut_bad_105_2 = sqaut_bad_105_2.values
arr_sqaut_bad_135_2 = sqaut_bad_135_2.values
arr_sqaut_bad_155_2 = sqaut_bad_155_2.values
arr_sqaut_bad_180_2 = sqaut_bad_180_2.values
arr_sqaut_bad_hand0_3 = sqaut_bad_hand0_3.values
arr_sqaut_bad_hand25_3 = sqaut_bad_hand25_3.values
arr_sqaut_bad_hand45_3 = sqaut_bad_hand45_3.values
arr_sqaut_bad_hand60_3 = sqaut_bad_hand60_3.values
arr_sqaut_bad_hand90_3 = sqaut_bad_hand90_3.values
arr_sqaut_bad_hand105_3 = sqaut_bad_hand105_3.values
arr_sqaut_bad_hand135_3 = sqaut_bad_hand135_3.values
arr_sqaut_bad_hand155_3 = sqaut_bad_hand155_3.values
arr_sqaut_bad_hand180_3 = sqaut_bad_hand180_3.values
arr_sqaut_bad_0_3 = sqaut_bad_0_3.values
arr_sqaut_bad_25_3 = sqaut_bad_25_3.values
arr_sqaut_bad_45_3 = sqaut_bad_45_3.values
arr_sqaut_bad_60_3 = sqaut_bad_60_3.values
arr_sqaut_bad_90_3 = sqaut_bad_90_3.values
arr_sqaut_bad_105_3 = sqaut_bad_105_3.values
arr_sqaut_bad_135_3 = sqaut_bad_135_3.values
arr_sqaut_bad_155_3 = sqaut_bad_155_3.values
arr_sqaut_bad_180_3 = sqaut_bad_180_3.values

In [78]:
# 스쿼트 데이터를 시퀀스로 구성
seq_len = 30
arr_sqaut_hand0_0_X = []
for i in range(len(arr_sqaut_hand0_0) - seq_len):
    arr_sqaut_hand0_0_X.append(arr_sqaut_hand0_0[i:i + seq_len])
arr_sqaut_hand0_0_seq = np.array(arr_sqaut_hand0_0_X)
arr_sqaut_hand0_0_seq.shape   # (205, 30, 24)

(205, 30, 24)

In [79]:
seq_len = 30
arr_sqaut_hand25_0_X = []
for i in range(len(arr_sqaut_hand25_0) - seq_len):
    arr_sqaut_hand25_0_X.append(arr_sqaut_hand25_0[i:i + seq_len])
arr_sqaut_hand25_0_seq = np.array(arr_sqaut_hand25_0_X)
arr_sqaut_hand25_0_seq.shape  

(425, 30, 24)

In [80]:
seq_len = 30
arr_sqaut_hand45_0_X = []
for i in range(len(arr_sqaut_hand45_0) - seq_len):
    arr_sqaut_hand45_0_X.append(arr_sqaut_hand45_0[i:i + seq_len])
arr_sqaut_hand45_0_seq = np.array(arr_sqaut_hand45_0_X)
arr_sqaut_hand45_0_seq.shape 

(278, 30, 24)

In [81]:
seq_len = 30
arr_sqaut_hand60_0_X = []
for i in range(len(arr_sqaut_hand60_0) - seq_len):
    arr_sqaut_hand60_0_X.append(arr_sqaut_hand60_0[i:i + seq_len])
arr_sqaut_hand60_0_seq = np.array(arr_sqaut_hand60_0_X)
arr_sqaut_hand60_0_seq.shape 

(270, 30, 24)

In [82]:
seq_len = 30
arr_sqaut_hand90_0_X = []
for i in range(len(arr_sqaut_hand90_0) - seq_len):
    arr_sqaut_hand90_0_X.append(arr_sqaut_hand90_0[i:i + seq_len])
arr_sqaut_hand90_0_seq = np.array(arr_sqaut_hand90_0_X)
arr_sqaut_hand90_0_seq.shape 

(264, 30, 24)

In [83]:
seq_len = 30
arr_sqaut_hand105_0_X = []
for i in range(len(arr_sqaut_hand105_0) - seq_len):
    arr_sqaut_hand105_0_X.append(arr_sqaut_hand105_0[i:i + seq_len])
arr_sqaut_hand105_0_seq = np.array(arr_sqaut_hand105_0_X)
arr_sqaut_hand105_0_seq.shape 

(270, 30, 24)

In [84]:
seq_len = 30
arr_sqaut_hand135_0_X = []
for i in range(len(arr_sqaut_hand135_0) - seq_len):
    arr_sqaut_hand135_0_X.append(arr_sqaut_hand135_0[i:i + seq_len])
arr_sqaut_hand135_0_seq = np.array(arr_sqaut_hand135_0_X)
arr_sqaut_hand135_0_seq.shape 

(278, 30, 24)

In [85]:
seq_len = 30
arr_sqaut_hand155_0_X = []
for i in range(len(arr_sqaut_hand155_0) - seq_len):
    arr_sqaut_hand155_0_X.append(arr_sqaut_hand155_0[i:i + seq_len])
arr_sqaut_hand155_0_seq = np.array(arr_sqaut_hand155_0_X)
arr_sqaut_hand155_0_seq.shape 

(425, 30, 24)

In [86]:
seq_len = 30
arr_sqaut_hand180_0_X = []
for i in range(len(arr_sqaut_hand180_0) - seq_len):
    arr_sqaut_hand180_0_X.append(arr_sqaut_hand180_0[i:i + seq_len])
arr_sqaut_hand180_0_seq = np.array(arr_sqaut_hand180_0_X)
arr_sqaut_hand180_0_seq.shape 

(205, 30, 24)

In [87]:
seq_len = 30
arr_sqaut_0_0_X = []
for i in range(len(arr_sqaut_0_0) - seq_len):
    arr_sqaut_0_0_X.append(arr_sqaut_0_0[i:i + seq_len])
arr_sqaut_0_0_seq = np.array(arr_sqaut_0_0_X)
arr_sqaut_0_0_seq.shape 

(295, 30, 24)

In [88]:
seq_len = 30
arr_sqaut_25_0_X = []
for i in range(len(arr_sqaut_25_0) - seq_len):
    arr_sqaut_25_0_X.append(arr_sqaut_25_0[i:i + seq_len])
arr_sqaut_25_0_seq = np.array(arr_sqaut_25_0_X)
arr_sqaut_25_0_seq.shape 

(423, 30, 24)

In [89]:
seq_len = 30
arr_sqaut_45_0_X = []
for i in range(len(arr_sqaut_45_0) - seq_len):
    arr_sqaut_45_0_X.append(arr_sqaut_45_0[i:i + seq_len])
arr_sqaut_45_0_seq = np.array(arr_sqaut_45_0_X)
arr_sqaut_45_0_seq.shape 

(316, 30, 24)

In [90]:
seq_len = 30
arr_sqaut_60_0_X = []
for i in range(len(arr_sqaut_60_0) - seq_len):
    arr_sqaut_60_0_X.append(arr_sqaut_60_0[i:i + seq_len])
arr_sqaut_60_0_seq = np.array(arr_sqaut_60_0_X)
arr_sqaut_60_0_seq.shape 

(378, 30, 24)

In [91]:
seq_len = 30
arr_sqaut_90_0_X = []
for i in range(len(arr_sqaut_90_0) - seq_len):
    arr_sqaut_90_0_X.append(arr_sqaut_90_0[i:i + seq_len])
arr_sqaut_90_0_seq = np.array(arr_sqaut_90_0_X)
arr_sqaut_90_0_seq.shape

(261, 30, 24)

In [92]:
seq_len = 30
arr_sqaut_105_0_X = []
for i in range(len(arr_sqaut_105_0) - seq_len):
    arr_sqaut_105_0_X.append(arr_sqaut_105_0[i:i + seq_len])
arr_sqaut_105_0_seq = np.array(arr_sqaut_105_0_X)
arr_sqaut_105_0_seq.shape 

(378, 30, 24)

In [93]:
seq_len = 30
arr_sqaut_135_0_X = []
for i in range(len(arr_sqaut_135_0) - seq_len):
    arr_sqaut_135_0_X.append(arr_sqaut_135_0[i:i + seq_len])
arr_sqaut_135_0_seq = np.array(arr_sqaut_135_0_X)
arr_sqaut_135_0_seq.shape 

(316, 30, 24)

In [94]:
seq_len = 30
arr_sqaut_155_0_X = []
for i in range(len(arr_sqaut_155_0) - seq_len):
    arr_sqaut_155_0_X.append(arr_sqaut_155_0[i:i + seq_len])
arr_sqaut_155_0_seq = np.array(arr_sqaut_155_0_X)
arr_sqaut_155_0_seq.shape 

(423, 30, 24)

In [95]:
seq_len = 30
arr_sqaut_180_0_X = []
for i in range(len(arr_sqaut_180_0) - seq_len):
    arr_sqaut_180_0_X.append(arr_sqaut_180_0[i:i + seq_len])
arr_sqaut_180_0_seq = np.array(arr_sqaut_180_0_X)
arr_sqaut_180_0_seq.shape 

(376, 30, 24)

In [96]:
seq_len = 30
arr_sqaut_bad_hand0_1_X = []
for i in range(len(arr_sqaut_bad_hand0_1) - seq_len):
    arr_sqaut_bad_hand0_1_X.append(arr_sqaut_bad_hand0_1[i:i + seq_len])
arr_sqaut_bad_hand0_1_seq = np.array(arr_sqaut_bad_hand0_1_X)
arr_sqaut_bad_hand0_1_seq.shape 

(267, 30, 24)

In [97]:
seq_len = 30
arr_sqaut_bad_hand25_1_X = []
for i in range(len(arr_sqaut_bad_hand25_1) - seq_len):
    arr_sqaut_bad_hand25_1_X.append(arr_sqaut_bad_hand25_1[i:i + seq_len])
arr_sqaut_bad_hand25_1_seq = np.array(arr_sqaut_bad_hand25_1_X)
arr_sqaut_bad_hand25_1_seq.shape

(254, 30, 24)

In [98]:
seq_len = 30
arr_sqaut_bad_hand45_1_X = []
for i in range(len(arr_sqaut_bad_hand45_1) - seq_len):
    arr_sqaut_bad_hand45_1_X.append(arr_sqaut_bad_hand45_1[i:i + seq_len])
arr_sqaut_bad_hand45_1_seq = np.array(arr_sqaut_bad_hand45_1_X)
arr_sqaut_bad_hand45_1_seq.shape

(281, 30, 24)

In [99]:
seq_len = 30
arr_sqaut_bad_hand60_1_X = []
for i in range(len(arr_sqaut_bad_hand60_1) - seq_len):
    arr_sqaut_bad_hand60_1_X.append(arr_sqaut_bad_hand60_1[i:i + seq_len])
arr_sqaut_bad_hand60_1_seq = np.array(arr_sqaut_bad_hand60_1_X)
arr_sqaut_bad_hand60_1_seq.shape

(293, 30, 24)

In [100]:
seq_len = 30
arr_sqaut_bad_hand90_1_X = []
for i in range(len(arr_sqaut_bad_hand90_1) - seq_len):
    arr_sqaut_bad_hand90_1_X.append(arr_sqaut_bad_hand90_1[i:i + seq_len])
arr_sqaut_bad_hand90_1_seq = np.array(arr_sqaut_bad_hand90_1_X)
arr_sqaut_bad_hand90_1_seq.shape

(317, 30, 24)

In [101]:
seq_len = 30
arr_sqaut_bad_hand105_1_X = []
for i in range(len(arr_sqaut_bad_hand105_1) - seq_len):
    arr_sqaut_bad_hand105_1_X.append(arr_sqaut_bad_hand105_1[i:i + seq_len])
arr_sqaut_bad_hand105_1_seq = np.array(arr_sqaut_bad_hand105_1_X)
arr_sqaut_bad_hand105_1_seq.shape

(293, 30, 24)

In [102]:
seq_len = 30
arr_sqaut_bad_hand135_1_X = []
for i in range(len(arr_sqaut_bad_hand135_1) - seq_len):
    arr_sqaut_bad_hand135_1_X.append(arr_sqaut_bad_hand135_1[i:i + seq_len])
arr_sqaut_bad_hand135_1_seq = np.array(arr_sqaut_bad_hand135_1_X)
arr_sqaut_bad_hand135_1_seq.shape

(281, 30, 24)

In [103]:
seq_len = 30
arr_sqaut_bad_hand155_1_X = []
for i in range(len(arr_sqaut_bad_hand155_1) - seq_len):
    arr_sqaut_bad_hand155_1_X.append(arr_sqaut_bad_hand155_1[i:i + seq_len])
arr_sqaut_bad_hand155_1_seq = np.array(arr_sqaut_bad_hand155_1_X)
arr_sqaut_bad_hand155_1_seq.shape

(254, 30, 24)

In [104]:
seq_len = 30
arr_sqaut_bad_hand180_1_X = []
for i in range(len(arr_sqaut_bad_hand180_1) - seq_len):
    arr_sqaut_bad_hand180_1_X.append(arr_sqaut_bad_hand180_1[i:i + seq_len])
arr_sqaut_bad_hand180_1_seq = np.array(arr_sqaut_bad_hand180_1_X)
arr_sqaut_bad_hand180_1_seq.shape

(267, 30, 24)

In [105]:
seq_len = 30
arr_sqaut_bad_0_1_X = []
for i in range(len(arr_sqaut_bad_0_1) - seq_len):
    arr_sqaut_bad_0_1_X.append(arr_sqaut_bad_0_1[i:i + seq_len])
arr_sqaut_bad_0_1_seq = np.array(arr_sqaut_bad_0_1_X)
arr_sqaut_bad_0_1_seq.shape

(316, 30, 24)

In [106]:
seq_len = 30
arr_sqaut_bad_25_1_X = []
for i in range(len(arr_sqaut_bad_25_1) - seq_len):
    arr_sqaut_bad_25_1_X.append(arr_sqaut_bad_25_1[i:i + seq_len])
arr_sqaut_bad_25_1_seq = np.array(arr_sqaut_bad_25_1_X)
arr_sqaut_bad_25_1_seq.shape

(297, 30, 24)

In [107]:
seq_len = 30
arr_sqaut_bad_45_1_X = []
for i in range(len(arr_sqaut_bad_45_1) - seq_len):
    arr_sqaut_bad_45_1_X.append(arr_sqaut_bad_45_1[i:i + seq_len])
arr_sqaut_bad_45_1_seq = np.array(arr_sqaut_bad_45_1_X)
arr_sqaut_bad_45_1_seq.shape

(301, 30, 24)

In [108]:
seq_len = 30
arr_sqaut_bad_60_1_X = []
for i in range(len(arr_sqaut_bad_60_1) - seq_len):
    arr_sqaut_bad_60_1_X.append(arr_sqaut_bad_60_1[i:i + seq_len])
arr_sqaut_bad_60_1_seq = np.array(arr_sqaut_bad_60_1_X)
arr_sqaut_bad_60_1_seq.shape

(308, 30, 24)

In [109]:
seq_len = 30
arr_sqaut_bad_90_1_X = []
for i in range(len(arr_sqaut_bad_90_1) - seq_len):
    arr_sqaut_bad_90_1_X.append(arr_sqaut_bad_90_1[i:i + seq_len])
arr_sqaut_bad_90_1_seq = np.array(arr_sqaut_bad_90_1_X)
arr_sqaut_bad_90_1_seq.shape

(316, 30, 24)

In [110]:
seq_len = 30
arr_sqaut_bad_105_1_X = []
for i in range(len(arr_sqaut_bad_105_1) - seq_len):
    arr_sqaut_bad_105_1_X.append(arr_sqaut_bad_105_1[i:i + seq_len])
arr_sqaut_bad_105_1_seq = np.array(arr_sqaut_bad_105_1_X)
arr_sqaut_bad_105_1_seq.shape

(308, 30, 24)

In [111]:
seq_len = 30
arr_sqaut_bad_135_1_X = []
for i in range(len(arr_sqaut_bad_135_1) - seq_len):
    arr_sqaut_bad_135_1_X.append(arr_sqaut_bad_135_1[i:i + seq_len])
arr_sqaut_bad_135_1_seq = np.array(arr_sqaut_bad_135_1_X)
arr_sqaut_bad_135_1_seq.shape

(301, 30, 24)

In [112]:
seq_len = 30
arr_sqaut_bad_155_1_X = []
for i in range(len(arr_sqaut_bad_155_1) - seq_len):
    arr_sqaut_bad_155_1_X.append(arr_sqaut_bad_155_1[i:i + seq_len])
arr_sqaut_bad_155_1_seq = np.array(arr_sqaut_bad_155_1_X)
arr_sqaut_bad_155_1_seq.shape

(297, 30, 24)

In [113]:
seq_len = 30
arr_sqaut_bad_180_1_X = []
for i in range(len(arr_sqaut_bad_180_1) - seq_len):
    arr_sqaut_bad_180_1_X.append(arr_sqaut_bad_180_1[i:i + seq_len])
arr_sqaut_bad_180_1_seq = np.array(arr_sqaut_bad_180_1_X)
arr_sqaut_bad_180_1_seq.shape

(316, 30, 24)

In [114]:
seq_len = 30
arr_sqaut_bad_hand0_2_X = []
for i in range(len(arr_sqaut_bad_hand0_2) - seq_len):
    arr_sqaut_bad_hand0_2_X.append(arr_sqaut_bad_hand0_2[i:i + seq_len])
arr_sqaut_bad_hand0_2_seq = np.array(arr_sqaut_bad_hand0_2_X)
arr_sqaut_bad_hand0_2_seq.shape

(203, 30, 24)

In [115]:
seq_len = 30
arr_sqaut_bad_hand25_2_X = []
for i in range(len(arr_sqaut_bad_hand25_2) - seq_len):
    arr_sqaut_bad_hand25_2_X.append(arr_sqaut_bad_hand25_2[i:i + seq_len])
arr_sqaut_bad_hand25_2_seq = np.array(arr_sqaut_bad_hand25_2_X)
arr_sqaut_bad_hand25_2_seq.shape

(253, 30, 24)

In [116]:
seq_len = 30
arr_sqaut_bad_hand45_2_X = []
for i in range(len(arr_sqaut_bad_hand45_2) - seq_len):
    arr_sqaut_bad_hand45_2_X.append(arr_sqaut_bad_hand45_2[i:i + seq_len])
arr_sqaut_bad_hand45_2_seq = np.array(arr_sqaut_bad_hand45_2_X)
arr_sqaut_bad_hand25_2_seq.shape

(253, 30, 24)

In [117]:
seq_len = 30
arr_sqaut_bad_hand60_2_X = []
for i in range(len(arr_sqaut_bad_hand60_2) - seq_len):
    arr_sqaut_bad_hand60_2_X.append(arr_sqaut_bad_hand60_2[i:i + seq_len])
arr_sqaut_bad_hand60_2_seq = np.array(arr_sqaut_bad_hand60_2_X)
arr_sqaut_bad_hand60_2_seq.shape

(218, 30, 24)

In [118]:
seq_len = 30
arr_sqaut_bad_hand90_2_X = []
for i in range(len(arr_sqaut_bad_hand90_2) - seq_len):
    arr_sqaut_bad_hand90_2_X.append(arr_sqaut_bad_hand90_2[i:i + seq_len])
arr_sqaut_bad_hand90_2_seq = np.array(arr_sqaut_bad_hand90_2_X)
arr_sqaut_bad_hand90_2_seq.shape

(213, 30, 24)

In [119]:
seq_len = 30
arr_sqaut_bad_hand105_2_X = []
for i in range(len(arr_sqaut_bad_hand105_2) - seq_len):
    arr_sqaut_bad_hand105_2_X.append(arr_sqaut_bad_hand105_2[i:i + seq_len])
arr_sqaut_bad_hand105_2_seq = np.array(arr_sqaut_bad_hand105_2_X)
arr_sqaut_bad_hand105_2_seq.shape

(218, 30, 24)

In [120]:
seq_len = 30
arr_sqaut_bad_hand135_2_X = []
for i in range(len(arr_sqaut_bad_hand135_2) - seq_len):
    arr_sqaut_bad_hand135_2_X.append(arr_sqaut_bad_hand135_2[i:i + seq_len])
arr_sqaut_bad_hand135_2_seq = np.array(arr_sqaut_bad_hand135_2_X)
arr_sqaut_bad_hand135_2_seq.shape

(213, 30, 24)

In [121]:
seq_len = 30
arr_sqaut_bad_hand155_2_X = []
for i in range(len(arr_sqaut_bad_hand155_2) - seq_len):
    arr_sqaut_bad_hand155_2_X.append(arr_sqaut_bad_hand155_2[i:i + seq_len])
arr_sqaut_bad_hand155_2_seq = np.array(arr_sqaut_bad_hand155_2_X)
arr_sqaut_bad_hand155_2_seq.shape

(253, 30, 24)

In [122]:
seq_len = 30
arr_sqaut_bad_hand180_2_X = []
for i in range(len(arr_sqaut_bad_hand180_2) - seq_len):
    arr_sqaut_bad_hand180_2_X.append(arr_sqaut_bad_hand180_2[i:i + seq_len])
arr_sqaut_bad_hand180_2_seq = np.array(arr_sqaut_bad_hand180_2_X)
arr_sqaut_bad_hand180_2_seq.shape

(203, 30, 24)

In [123]:
seq_len = 30
arr_sqaut_bad_0_2_X = []
for i in range(len(arr_sqaut_bad_0_2) - seq_len):
    arr_sqaut_bad_0_2_X.append(arr_sqaut_bad_0_2[i:i + seq_len])
arr_sqaut_bad_0_2_seq = np.array(arr_sqaut_bad_0_2_X)
arr_sqaut_bad_0_2_seq.shape

(255, 30, 24)

In [124]:
seq_len = 30
arr_sqaut_bad_25_2_X = []
for i in range(len(arr_sqaut_bad_25_2) - seq_len):
    arr_sqaut_bad_25_2_X.append(arr_sqaut_bad_25_2[i:i + seq_len])
arr_sqaut_bad_25_2_seq = np.array(arr_sqaut_bad_25_2_X)
arr_sqaut_bad_25_2_seq.shape

(213, 30, 24)

In [125]:
seq_len = 30
arr_sqaut_bad_45_2_X = []
for i in range(len(arr_sqaut_bad_45_2) - seq_len):
    arr_sqaut_bad_45_2_X.append(arr_sqaut_bad_45_2[i:i + seq_len])
arr_sqaut_bad_45_2_seq = np.array(arr_sqaut_bad_45_2_X)
arr_sqaut_bad_45_2_seq.shape

(232, 30, 24)

In [126]:
seq_len = 30
arr_sqaut_bad_60_2_X = []
for i in range(len(arr_sqaut_bad_60_2) - seq_len):
    arr_sqaut_bad_60_2_X.append(arr_sqaut_bad_60_2[i:i + seq_len])
arr_sqaut_bad_60_2_seq = np.array(arr_sqaut_bad_60_2_X)
arr_sqaut_bad_60_2_seq.shape

(201, 30, 24)

In [127]:
seq_len = 30
arr_sqaut_bad_90_2_X = []
for i in range(len(arr_sqaut_bad_90_2) - seq_len):
    arr_sqaut_bad_90_2_X.append(arr_sqaut_bad_90_2[i:i + seq_len])
arr_sqaut_bad_90_2_seq = np.array(arr_sqaut_bad_90_2_X)
arr_sqaut_bad_90_2_seq.shape

(181, 30, 24)

In [128]:
seq_len = 30
arr_sqaut_bad_105_2_X = []
for i in range(len(arr_sqaut_bad_105_2) - seq_len):
    arr_sqaut_bad_105_2_X.append(arr_sqaut_bad_105_2[i:i + seq_len])
arr_sqaut_bad_105_2_seq = np.array(arr_sqaut_bad_105_2_X)
arr_sqaut_bad_105_2_seq.shape

(201, 30, 24)

In [129]:
seq_len = 30
arr_sqaut_bad_135_2_X = []
for i in range(len(arr_sqaut_bad_135_2) - seq_len):
    arr_sqaut_bad_135_2_X.append(arr_sqaut_bad_135_2[i:i + seq_len])
arr_sqaut_bad_135_2_seq = np.array(arr_sqaut_bad_135_2_X)
arr_sqaut_bad_135_2_seq.shape

(232, 30, 24)

In [130]:
seq_len = 30
arr_sqaut_bad_155_2_X = []
for i in range(len(arr_sqaut_bad_155_2) - seq_len):
    arr_sqaut_bad_155_2_X.append(arr_sqaut_bad_155_2[i:i + seq_len])
arr_sqaut_bad_155_2_seq = np.array(arr_sqaut_bad_155_2_X)
arr_sqaut_bad_155_2_seq.shape

(213, 30, 24)

In [131]:
seq_len = 30
arr_sqaut_bad_180_2_X = []
for i in range(len(arr_sqaut_bad_180_2) - seq_len):
    arr_sqaut_bad_180_2_X.append(arr_sqaut_bad_180_2[i:i + seq_len])
arr_sqaut_bad_180_2_seq = np.array(arr_sqaut_bad_180_2_X)
arr_sqaut_bad_180_2_seq.shape

(255, 30, 24)

In [132]:
seq_len = 30
arr_sqaut_bad_hand0_3_X = []
for i in range(len(arr_sqaut_bad_hand0_3) - seq_len):
    arr_sqaut_bad_hand0_3_X.append(arr_sqaut_bad_hand0_3[i:i + seq_len])
arr_sqaut_bad_hand0_3_seq = np.array(arr_sqaut_bad_hand0_3_X)
arr_sqaut_bad_hand0_3_seq.shape

(233, 30, 24)

In [133]:
seq_len = 30
arr_sqaut_bad_hand25_3_X = []
for i in range(len(arr_sqaut_bad_hand25_3) - seq_len):
    arr_sqaut_bad_hand25_3_X.append(arr_sqaut_bad_hand25_3[i:i + seq_len])
arr_sqaut_bad_hand25_3_seq = np.array(arr_sqaut_bad_hand25_3_X)
arr_sqaut_bad_hand25_3_seq.shape

(312, 30, 24)

In [134]:
seq_len = 30
arr_sqaut_bad_hand45_3_X = []
for i in range(len(arr_sqaut_bad_hand45_3) - seq_len):
    arr_sqaut_bad_hand45_3_X.append(arr_sqaut_bad_hand45_3[i:i + seq_len])
arr_sqaut_bad_hand45_3_seq = np.array(arr_sqaut_bad_hand45_3_X)
arr_sqaut_bad_hand45_3_seq.shape

(255, 30, 24)

In [135]:
seq_len = 30
arr_sqaut_bad_hand60_3_X = []
for i in range(len(arr_sqaut_bad_hand60_3) - seq_len):
    arr_sqaut_bad_hand60_3_X.append(arr_sqaut_bad_hand60_3[i:i + seq_len])
arr_sqaut_bad_hand60_3_seq = np.array(arr_sqaut_bad_hand60_3_X)
arr_sqaut_bad_hand60_3_seq.shape

(293, 30, 24)

In [136]:
seq_len = 30
arr_sqaut_bad_hand90_3_X = []
for i in range(len(arr_sqaut_bad_hand90_3) - seq_len):
    arr_sqaut_bad_hand90_3_X.append(arr_sqaut_bad_hand90_3[i:i + seq_len])
arr_sqaut_bad_hand90_3_seq = np.array(arr_sqaut_bad_hand90_3_X)
arr_sqaut_bad_hand90_3_seq.shape

(303, 30, 24)

In [137]:
seq_len = 30
arr_sqaut_bad_hand105_3_X = []
for i in range(len(arr_sqaut_bad_hand105_3) - seq_len):
    arr_sqaut_bad_hand105_3_X.append(arr_sqaut_bad_hand105_3[i:i + seq_len])
arr_sqaut_bad_hand105_3_seq = np.array(arr_sqaut_bad_hand105_3_X)
arr_sqaut_bad_hand105_3_seq.shape

(293, 30, 24)

In [138]:
seq_len = 30
arr_sqaut_bad_hand135_3_X = []
for i in range(len(arr_sqaut_bad_hand135_3) - seq_len):
    arr_sqaut_bad_hand135_3_X.append(arr_sqaut_bad_hand135_3[i:i + seq_len])
arr_sqaut_bad_hand135_3_seq = np.array(arr_sqaut_bad_hand135_3_X)
arr_sqaut_bad_hand135_3_seq.shape

(255, 30, 24)

In [139]:
seq_len = 30
arr_sqaut_bad_hand155_3_X = []
for i in range(len(arr_sqaut_bad_hand155_3) - seq_len):
    arr_sqaut_bad_hand155_3_X.append(arr_sqaut_bad_hand155_3[i:i + seq_len])
arr_sqaut_bad_hand155_3_seq = np.array(arr_sqaut_bad_hand155_3_X)
arr_sqaut_bad_hand155_3_seq.shape

(312, 30, 24)

In [140]:
seq_len = 30
arr_sqaut_bad_hand180_3_X = []
for i in range(len(arr_sqaut_bad_hand180_3) - seq_len):
    arr_sqaut_bad_hand180_3_X.append(arr_sqaut_bad_hand180_3[i:i + seq_len])
arr_sqaut_bad_hand180_3_seq = np.array(arr_sqaut_bad_hand180_3_X)
arr_sqaut_bad_hand180_3_seq.shape

(233, 30, 24)

In [141]:
seq_len = 30
arr_sqaut_bad_0_3_X = []
for i in range(len(arr_sqaut_bad_0_3) - seq_len):
    arr_sqaut_bad_0_3_X.append(arr_sqaut_bad_0_3[i:i + seq_len])
arr_sqaut_bad_0_3_seq = np.array(arr_sqaut_bad_0_3_X)
arr_sqaut_bad_0_3_seq.shape

(235, 30, 24)

In [142]:
seq_len = 30
arr_sqaut_bad_25_3_X = []
for i in range(len(arr_sqaut_bad_25_3) - seq_len):
    arr_sqaut_bad_25_3_X.append(arr_sqaut_bad_25_3[i:i + seq_len])
arr_sqaut_bad_25_3_seq = np.array(arr_sqaut_bad_25_3_X)
arr_sqaut_bad_25_3_seq.shape

(233, 30, 24)

In [143]:
seq_len = 30
arr_sqaut_bad_45_3_X = []
for i in range(len(arr_sqaut_bad_45_3) - seq_len):
    arr_sqaut_bad_45_3_X.append(arr_sqaut_bad_45_3[i:i + seq_len])
arr_sqaut_bad_45_3_seq = np.array(arr_sqaut_bad_45_3_X)
arr_sqaut_bad_45_3_seq.shape

(192, 30, 24)

In [144]:
seq_len = 30
arr_sqaut_bad_60_3_X = []
for i in range(len(arr_sqaut_bad_60_3) - seq_len):
    arr_sqaut_bad_60_3_X.append(arr_sqaut_bad_60_3[i:i + seq_len])
arr_sqaut_bad_60_3_seq = np.array(arr_sqaut_bad_60_3_X)
arr_sqaut_bad_60_3_seq.shape

(225, 30, 24)

In [145]:
seq_len = 30
arr_sqaut_bad_90_3_X = []
for i in range(len(arr_sqaut_bad_90_3) - seq_len):
    arr_sqaut_bad_90_3_X.append(arr_sqaut_bad_90_3[i:i + seq_len])
arr_sqaut_bad_90_3_seq = np.array(arr_sqaut_bad_90_3_X)
arr_sqaut_bad_90_3_seq.shape

(256, 30, 24)

In [146]:
seq_len = 30
arr_sqaut_bad_105_3_X = []
for i in range(len(arr_sqaut_bad_105_3) - seq_len):
    arr_sqaut_bad_105_3_X.append(arr_sqaut_bad_105_3[i:i + seq_len])
arr_sqaut_bad_105_3_seq = np.array(arr_sqaut_bad_105_3_X)
arr_sqaut_bad_105_3_seq.shape

(225, 30, 24)

In [147]:
seq_len = 30
arr_sqaut_bad_135_3_X = []
for i in range(len(arr_sqaut_bad_135_3) - seq_len):
    arr_sqaut_bad_135_3_X.append(arr_sqaut_bad_135_3[i:i + seq_len])
arr_sqaut_bad_135_3_seq = np.array(arr_sqaut_bad_135_3_X)
arr_sqaut_bad_135_3_seq.shape

(192, 30, 24)

In [148]:
seq_len = 30
arr_sqaut_bad_155_3_X = []
for i in range(len(arr_sqaut_bad_155_3) - seq_len):
    arr_sqaut_bad_155_3_X.append(arr_sqaut_bad_155_3[i:i + seq_len])
arr_sqaut_bad_155_3_seq = np.array(arr_sqaut_bad_155_3_X)
arr_sqaut_bad_155_3_seq.shape

(233, 30, 24)

In [149]:
seq_len = 30
arr_sqaut_bad_180_3_X = []
for i in range(len(arr_sqaut_bad_180_3) - seq_len):
    arr_sqaut_bad_180_3_X.append(arr_sqaut_bad_180_3[i:i + seq_len])
arr_sqaut_bad_180_3_seq = np.array(arr_sqaut_bad_180_3_X)
arr_sqaut_bad_180_3_seq.shape

(235, 30, 24)

In [150]:
# 스쿼트 번호 별로 Sequence를 하나로 합친다
sqaut_0_seq = np.concatenate((arr_sqaut_hand0_0_seq,
arr_sqaut_hand25_0_seq,
arr_sqaut_hand45_0_seq,
arr_sqaut_hand60_0_seq,
arr_sqaut_hand90_0_seq,
arr_sqaut_hand105_0_seq,
arr_sqaut_hand135_0_seq,
arr_sqaut_hand155_0_seq,
arr_sqaut_hand180_0_seq,
arr_sqaut_0_0_seq,
arr_sqaut_25_0_seq,
arr_sqaut_45_0_seq,
arr_sqaut_60_0_seq,
arr_sqaut_90_0_seq,
arr_sqaut_105_0_seq,
arr_sqaut_135_0_seq,
arr_sqaut_155_0_seq,
arr_sqaut_180_0_seq), axis=0)
sqaut_0_seq.shape  # (5786, 30, 24)

(5786, 30, 24)

In [151]:
sqaut_1_seq = np.concatenate((arr_sqaut_bad_hand0_1_seq,
arr_sqaut_bad_hand25_1_seq,
arr_sqaut_bad_hand45_1_seq,
arr_sqaut_bad_hand60_1_seq,
arr_sqaut_bad_hand90_1_seq,
arr_sqaut_bad_hand105_1_seq,
arr_sqaut_bad_hand135_1_seq,
arr_sqaut_bad_hand155_1_seq,
arr_sqaut_bad_hand180_1_seq,
arr_sqaut_bad_0_1_seq,
arr_sqaut_bad_25_1_seq,
arr_sqaut_bad_45_1_seq,
arr_sqaut_bad_60_1_seq,
arr_sqaut_bad_90_1_seq,
arr_sqaut_bad_105_1_seq,
arr_sqaut_bad_135_1_seq,
arr_sqaut_bad_155_1_seq,
arr_sqaut_bad_180_1_seq), axis=0)
sqaut_1_seq.shape   

(5267, 30, 24)

In [152]:
sqaut_2_seq = np.concatenate((arr_sqaut_bad_hand0_2_seq,
arr_sqaut_bad_hand25_2_seq,
arr_sqaut_bad_hand45_2_seq,
arr_sqaut_bad_hand60_2_seq,
arr_sqaut_bad_hand90_2_seq,
arr_sqaut_bad_hand105_2_seq,
arr_sqaut_bad_hand135_2_seq,
arr_sqaut_bad_hand155_2_seq,
arr_sqaut_bad_hand180_2_seq,
arr_sqaut_bad_0_2_seq,
arr_sqaut_bad_25_2_seq,
arr_sqaut_bad_45_2_seq,
arr_sqaut_bad_60_2_seq,
arr_sqaut_bad_90_2_seq,
arr_sqaut_bad_105_2_seq,
arr_sqaut_bad_135_2_seq,
arr_sqaut_bad_155_2_seq,
arr_sqaut_bad_180_2_seq), axis=0)
sqaut_2_seq.shape   

(3970, 30, 24)

In [153]:
sqaut_3_seq = np.concatenate((arr_sqaut_bad_hand0_3_seq,
arr_sqaut_bad_hand25_3_seq,
arr_sqaut_bad_hand45_3_seq,
arr_sqaut_bad_hand60_3_seq,
arr_sqaut_bad_hand90_3_seq,
arr_sqaut_bad_hand105_3_seq,
arr_sqaut_bad_hand135_3_seq,
arr_sqaut_bad_hand155_3_seq,
arr_sqaut_bad_hand180_3_seq,
arr_sqaut_bad_0_3_seq,
arr_sqaut_bad_25_3_seq,
arr_sqaut_bad_45_3_seq,
arr_sqaut_bad_60_3_seq,
arr_sqaut_bad_90_3_seq,
arr_sqaut_bad_105_3_seq,
arr_sqaut_bad_135_3_seq,
arr_sqaut_bad_155_3_seq,
arr_sqaut_bad_180_3_seq), axis=0)
sqaut_3_seq.shape   

(4515, 30, 24)

In [154]:
# seq에 대한 Label을 준비한다
sqaut_0_labels = np.zeros(len(sqaut_0_seq)) 
len(sqaut_0_labels)   

sqaut_1_labels = np.ones(len(sqaut_1_seq))
len(sqaut_1_labels)

sqaut_2_labels = np.full(len(sqaut_2_seq), 2)
len(sqaut_2_labels)

sqaut_3_labels = np.full(len(sqaut_3_seq), 3)
len(sqaut_3_labels)

4515

In [155]:
# 학습용 데이터를 합쳐서 total_X 생성
# Label를 합쳐서 train_y 생성
total_X = np.concatenate((sqaut_0_seq, sqaut_1_seq, sqaut_2_seq, sqaut_3_seq), axis=0)
total_y = np.concatenate((sqaut_0_labels, sqaut_1_labels, sqaut_2_labels, sqaut_3_labels), axis=0)

total_X.shape,   total_y.shape    

((19538, 30, 24), (19538,))

In [156]:
!pip install scikit-learn

In [157]:
# total_X, total_y 데이터를 학습용과 검증용으로 분리한다
from sklearn.model_selection import train_test_split

In [158]:
train_X, test_X, train_y, test_y = train_test_split(total_X, total_y, test_size=0.2, random_state=0)
train_X.shape, test_X.shape, train_y.shape, test_y.shape

((15630, 30, 24), (3908, 30, 24), (15630,), (3908,))

In [159]:
# Label 데이터가 정확한지 테스트
unique, counts = np.unique(total_y, return_counts=True)
unique, counts    # array([5786, 5267, 3970, 4515], dtype=int64))

(array([0., 1., 2., 3.]), array([5786, 5267, 3970, 4515], dtype=int64))

In [ ]:
!pip install tensorflow

## LSTM 신경망 구성 및 학습

In [181]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
import random as rn

rn.seed(0)
np.random.seed(0)
tf.random.set_seed(3)

model = Sequential([
    Input(shape=(seq_len, 24)),
    LSTM(256, return_sequences=True),    
    Dropout(0.4),    
    LSTM(128, return_sequences=True),    
    Dropout(0.3),                        
    LSTM(64, return_sequences=True),     
    Dropout(0.3),
    LSTM(32, return_sequences=True),     
    Dropout(0.3),
    LSTM(16, return_sequences=False),
    Dropout(0.3),
    Dense(4, activation='softmax') # 다중분류시 activation은 softmax output3개면 3으로면경
])

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) #  loss = sparse_categorical_crossentropy 라벨숫자로

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_24 (LSTM)              (None, 30, 256)           287744    
                                                                 
 dropout_24 (Dropout)        (None, 30, 256)           0         
                                                                 
 lstm_25 (LSTM)              (None, 30, 128)           197120    
                                                                 
 dropout_25 (Dropout)        (None, 30, 128)           0         
                                                                 
 lstm_26 (LSTM)              (None, 30, 64)            49408     
                                                                 
 dropout_26 (Dropout)        (None, 30, 64)            0         
                                                                 
 lstm_27 (LSTM)              (None, 30, 32)           

In [182]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# EarlyStopping 설정
early_stopping = EarlyStopping(
    monitor='val_loss',  # 모니터링할 지표
    patience=15,          # 개선되지 않더라도 몇 epoch을 더 지켜볼지
    verbose=1,           # 조기 종료 시 출력 메시지
    restore_best_weights=True # 가장 좋은 성능의 가중치를 복원
)

checkpoint = ModelCheckpoint(
    filepath='datakeras\\sqaut_model_checkpoint3.keras', # 모델이 저장될 경로(*.keras 요구됨)
    monitor='val_loss',             # 모니터링할 지표(지표가 상승할 때만 파일에 저장됨)
    save_best_only=True,            # 가장 좋은 성능의 모델만 저장
    verbose=1
)

In [183]:
# 모델 학습
hist = model.fit(train_X, train_y, epochs=10000, batch_size=32, validation_data=(test_X, test_y), 
verbose=1, callbacks=[early_stopping, checkpoint])

Epoch 1/10000
489/489 [==============================] - ETA: 0s - loss: 1.2841 - accuracy: 0.3690
Epoch 1: val_loss improved from inf to 1.13973, saving model to datakeras\sqaut_model_checkpoint3.keras
489/489 [==============================] - 25s 41ms/step - loss: 1.2841 - accuracy: 0.3690 - val_loss: 1.1397 - val_accuracy: 0.4839
Epoch 2/10000
489/489 [==============================] - ETA: 0s - loss: 1.0682 - accuracy: 0.5269
Epoch 2: val_loss improved from 1.13973 to 0.92859, saving model to datakeras\sqaut_model_checkpoint3.keras
489/489 [==============================] - 20s 40ms/step - loss: 1.0682 - accuracy: 0.5269 - val_loss: 0.9286 - val_accuracy: 0.6067
Epoch 3/10000
489/489 [==============================] - ETA: 0s - loss: 0.9347 - accuracy: 0.5937
Epoch 3: val_loss improved from 0.92859 to 0.68346, saving model to datakeras\sqaut_model_checkpoint3.keras
489/489 [==============================] - 19s 39ms/step - loss: 0.9347 - accuracy: 0.5937 - val_loss: 0.6835 - val_a

In [ ]:
 import matplotlib.pyplot as plt

plt.figure(figsize=(5,3))
plt.plot(hist.history['loss'], label='loss')
plt.plot(hist.history['accuracy'], label='accuracy')
plt.plot(hist.history['val_loss'], label='val_loss')
plt.plot(hist.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epochs')
plt.ylabel('Error')
plt.legend()
plt.show()

## 모델 평가

In [187]:
from tensorflow.keras.models import load_model

# 저장된 모델을 불러오기
model = load_model('datakeras\\sqaut_model_checkpoint2.keras')

# 모델 요약 출력 (모델 구조 확인)
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_30 (LSTM)              (None, 30, 128)           78336     
                                                                 
 dropout_27 (Dropout)        (None, 30, 128)           0         
                                                                 
 lstm_31 (LSTM)              (None, 30, 64)            49408     
                                                                 
 dropout_28 (Dropout)        (None, 30, 64)            0         
                                                                 
 lstm_32 (LSTM)              (None, 30, 32)            12416     
                                                                 
 dropout_29 (Dropout)        (None, 30, 32)            0         
                                                                 
 lstm_33 (LSTM)              (None, 16)               

In [188]:
# 모델 평가
loss, accuracy = model.evaluate(test_X, test_y)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


123/123 [==============================] - 2s 11ms/step - loss: 0.0030 - accuracy: 0.9992
Test Loss: 0.002952232724055648
Test Accuracy: 0.9992323517799377


In [215]:
# 모델 예측
predictions = model.predict(test_X)

# 예측 결과 출력
print(predictions)


123/123 [==============================] - 2s 11ms/step
[[2.9814466e-05 4.1898675e-05 9.9987900e-01 4.9267725e-05]
 [4.7198435e-05 8.7185894e-05 9.9979728e-01 6.8329246e-05]
 [1.8201172e-05 4.4052467e-06 8.0586860e-06 9.9996924e-01]
 ...
 [9.9981672e-01 1.4466449e-04 5.8796672e-06 3.2747459e-05]
 [1.1760959e-04 9.9986506e-01 1.2602399e-05 4.6688642e-06]
 [6.7293375e-05 9.9992025e-01 9.5290770e-06 2.8455097e-06]]


## 학습된 모델을 파일에 저장

In [73]:
# 학습된 모델을 파일에 저장
model.save('data\\lstm_walk_run_classify_model.keras')
print('모델을 파일에 저장함')

모델을 파일에 저장함


## 저장된 모델을 파일에서 로드

In [2]:
from tensorflow.keras.models import load_model

loaded_lstm_model = load_model('datakeras\\sqaut_model_checkpoint2.keras')   # 원본 비디오 학습모델
loaded_lstm_model
print('모델 로드함')

모델 로드함


## 실제 비디오 파일 분류

In [ ]:
%%writefile fastApiServer.py
import os
import re
import cv2
import joblib
import shutil
import openai
import tempfile
import requests
import numpy as np
import pandas as pd
import mediapipe as mp 
import tensorflow as tf
from collections import deque
from dotenv import load_dotenv
from pydantic import BaseModel
from tensorflow.keras.models import load_model
from fastapi.responses import JSONResponse
from fastapi.staticfiles import StaticFiles
from fastapi.middleware.cors import CORSMiddleware
from fastapi.exceptions import RequestValidationError
from fastapi import FastAPI, HTTPException, File, UploadFile, Request
from langchain_openai import ChatOpenAI   # instruct 모델은 OpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

app= FastAPI()
app

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4o-mini")
llm

LOCAL_DIRECTORY = os.path.join(os.path.dirname(__file__), "fitizenmovie")
app.mount("/videos", StaticFiles(directory=LOCAL_DIRECTORY), name="videos")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # 모든 도메인에서의 요청을 허용하려면 "*" 사용
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# 서버 상태 확인을 위한 기본 엔드포인트
@app.get("/")
def read_root():
    return {"message": "Welcome to test FastAPI!"}

class VideoRequest(BaseModel):
    video_url: str

loaded_lstm_model = load_model('datakeras\\sqaut_model_checkpoint2.keras')  # 모델 경로 설정

@app.post("/analyzevideo")
async def analyze_video(request: VideoRequest):
    try:
        # URL로부터 비디오 스트림 가져오기
        try:
            response = requests.get(request.video_url, stream=True,  verify=False)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Step 2 Error: {str(e)}")
            raise HTTPException(status_code=500, detail=f"비디오 URL 요청 오류: {str(e)}")
        
        # 임시 파일에 비디오 저장
        try:
            with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as tmp_file:
                tmp_path = tmp_file.name
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        tmp_file.write(chunk)
        except Exception as e:
            print(f"Step 3 Error: {str(e)}")
            raise HTTPException(status_code=500, detail=f"임시 파일 저장 중 오류 발생: {str(e)}")

        # 비디오 파일을 열어서 프레임별로 처리
        try:
            cap = cv2.VideoCapture(tmp_path)
            if not cap.isOpened():
                raise HTTPException(status_code=500, detail="비디오 파일을 열 수 없습니다.")
        except Exception as e:
            print(f"Step 4 Error: {str(e)}")
            raise HTTPException(status_code=500, detail=f"비디오 파일 열기 오류: {str(e)}")

        # 모델 로드 (경로 확인)
        try: 
            loaded_lstm_model
        except Exception as e:
            print(f"Step 5 Error: {str(e)}")
            raise HTTPException(status_code=500, detail=f"LSTM 모델 로드 오류: {str(e)}")
        
        try:
            mp_drawing = mp.solutions.drawing_utils
            mp_pose = mp.solutions.pose
            pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
        except Exception as e:
            print(f"Step 6 Error: {str(e)}")
            raise HTTPException(status_code=500, detail=f"mediapipe pose 초기화 오류: {str(e)}")

         # 분석된 비디오를 저장할 경로 설정
        w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        # 'H264' 또는 'avc1' Ai서버로 올릴시 openh264-1.8.0-win64.dll 다운후 파이썬스크립트에 적용
        fourcc = cv2.VideoWriter_fourcc(*'H264')  
        output_video_path = 'fitizenmovie/processed_' + os.path.basename(tmp_path)
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (w, h))

        frame_cnt = 0
        sequence_dq = deque(maxlen=30)
        analysis_results = []
        bone_idx = [11, 12, 13, 14, 15, 16, 23, 24, 25, 25, 27, 28]  # 관절 인덱스

        # 비디오 분석 루프
        try:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break  
                frame_cnt += 1
                print(f"Step 7: Processing frame {frame_cnt}")  # 프레임 처리 중

                # 이미지 전처리 및 포즈 추출
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)  # 랜드마크 정보를 사용하여 연결선을 그림
                out.write(image)

                if results.pose_landmarks is None:
                    print(f"No landmarks detected for frame {frame_cnt}")
                    analysis_results.append({"frame": frame_cnt, "prediction": "No landmarks"})
                    continue

                # 관절 위치 데이터 수집
                row = []
                for idx in bone_idx:
                    row.extend([round(results.pose_landmarks.landmark[idx].x, 5),
                                round(results.pose_landmarks.landmark[idx].y, 5)])
                sequence_dq.append(row)

                # 시퀀스 길이가 충분히 쌓였을 때 예측 수행
                if len(sequence_dq) == 30:
                    pred = loaded_lstm_model.predict(np.array(list(sequence_dq)).reshape(1, 30, 24))
                    pred_value = pred[0].argmax()
                    pose_code = str(pred_value)
                    analysis_results.append(f"{frame_cnt}:{pose_code}")
        except Exception as e:
            print(f"Step 7 Error: {str(e)}")
            raise HTTPException(status_code=500, detail=f"비디오 분석 오류: {str(e)}")

        cap.release()
        out.release()
        pose.close()

        # 임시 파일 삭제
        if os.path.exists(tmp_path):
            os.remove(tmp_path)
        
        ai_server_url = f"{os.path.basename(output_video_path)}"
        
        analysis_result_str = ",".join(analysis_results)
        aianswer = await analyze_video_ask_chatgpt(analysis_result_str)
        
        return {
            "analysis_results": analysis_result_str,
            "aivideourl": ai_server_url,
            "aianswer": aianswer.get("answer", "응답 없음")
        }

    except Exception as e:
        print(f"알 수 없는 오류 발생: {str(e)}")  # 전체적인 예외 처리
        raise HTTPException(status_code=500, detail=f"알 수 없는 오류 발생: {str(e)}")

async def analyze_video_ask_chatgpt(question_str: str):
    try:
        vector_store = get_or_create_vector_store()

        # 관련 문서 검색
        relevant_docs = search_relevant_docs(vector_store, question_str)


        # 관련 문서를 하나의 텍스트로 합치기
        relevant_texts = " ".join([doc.page_content for doc in relevant_docs])

        # GPT 프롬프트 생성
        gpt_prompt = (
            f"다음 스쿼트 자세 프레임 데이터를 분석하고 피드백을 제공하세요.\n\n"
            f"프레임 데이터: {question_str}\n\n"
            "자세 값:\n"
            "  0: 올바른 자세.\n"
            "  1: 무릎 문제 (예: 무릎이 안쪽으로 움직이거나, 발끝을 초과함).\n"
            "  2: 깊이 문제 (예: 너무 얕거나 너무 깊음).\n"
            "  3: 허리 각도 문제 (예: 둥글거나 과도하게 펴짐).\n\n"
            "규칙:\n"
            "- 반드시 한국어로만 작성하세요. 영어로 응답하지 마세요.\n"
            "- 제공된 프레임 데이터에 없는 자세 값에 대해서는 피드백을 작성하지 마세요.\n"
            "- 특정 자세 값이 많은 경우 해당 문제에 대한 피드백에 집중하세요.\n"
            "- 여러 자세 값이 섞여 있는 경우, 각 유형별로 제공된 프레임 데이터에 있는 자세 값에 대해 분석하고 피드백을 작성하세요.\n"
            "- 프레임 데이터 갯수를 정확하게 작성하세요.\n"
            "- 문제를 요약하고, 해결 방법을 제안하세요.\n\n"
            "분석 결과를 아래에 한국어로만 작성하세요:"
        )

        # GPT 모델 호출
        gpt_response = llm.invoke(gpt_prompt)

        # 응답에서 내용을 추출
        answer = gpt_response.content

        # 결과 반환
        return {"answer": answer}
    except Exception as e:
        print(f"Error in analyze_video_ask_chatgpt: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))

# 요청 바디 모델 정의
class QuestionRequest(BaseModel):
    question: str

VECTOR_STORE_FILE = "vector_store.faiss"
PDF_FILE = "<파일이름>.pdf"

# 벡터 스토어 생성 함수
def create_vector_store_from_pdf(pdf_file):
    try:
        loader = PyPDFLoader(pdf_file)
        documents = loader.load()
    except Exception as e:
        print(f"Error while loading PDF file: {e}")
        raise HTTPException(status_code=500, detail=f"PDF 파일을 로드할 수 없습니다: {e}")

    try:
        text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
        docs = text_splitter.split_documents(documents)
    except Exception as e:
        print(f"Error while splitting documents: {e}")
        raise HTTPException(status_code=500, detail=f"문서 청크 처리 중 오류 발생: {e}")

    try:
        embeddings = OpenAIEmbeddings()
        vector_store = FAISS.from_documents(docs, embeddings)
        return vector_store
    except Exception as e:
        print(f"Error while creating vector store: {e}")
        raise HTTPException(status_code=500, detail=f"벡터 스토어 생성 중 오류 발생: {e}")


# 벡터 스토어 저장 함수
def save_vector_store(vector_store, file_path):
    vector_store.save_local(file_path)

# 벡터 스토어 로드 함수
def load_vector_store(file_path):
    embeddings = OpenAIEmbeddings()
    return FAISS.load_local(file_path, embeddings, allow_dangerous_deserialization=True)

# 사용자의 질문(query)과 유사한 문서를 검색
def search_relevant_docs(vector_store, query):
    embeddings = OpenAIEmbeddings()
    query_embedding = embeddings.embed_query(query)
    results = vector_store.similarity_search_by_vector(query_embedding, k=3)
    return results

# ChatGPT API 호출 함수 (검색된 문서에 기반한 답변 생성)
def call_chatgpt_api(prompt):
    response = openai.Completion.create(
        model="gpt-4o-mini",  # 사용할 GPT 모델
        prompt=prompt,
        max_tokens=100
    )
    return response.choices[0].text.strip()
def get_or_create_vector_store():
    if os.path.exists(VECTOR_STORE_FILE):
        return load_vector_store(VECTOR_STORE_FILE)
    else:
        vector_store = create_vector_store_from_pdf(PDF_FILE)
        save_vector_store(vector_store, VECTOR_STORE_FILE)
        return vector_store    

#ssl_certfile="server.crt"
# FastAPI 앱 실행을 위한 코드
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)